In [1]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_to_release = ['eae5ee86-b6ee-4112-941b-311d6026acb7']
store, uuids = record_object_es(set_to_release, my_auth, schema_name, store_frame='raw', add_pc_wfr=True, store={}, item_uuids=[])
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1)/60)
   

Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem encountered - skipped - check
Problem enco

KeyboardInterrupt: 

In [27]:
from dcicutils import ff_utils
from functions.notebook_functions import *
import time

def record_object_es23(uuid_list, con_key, schema_name, store_frame='raw', add_pc_wfr=False):    
    """starting from a single uuid, tracks all linked items,
    keeps a list of uuids, and dictionary of items for each schema in the given store_frame"""
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    if add_pc_wfr:
        add_from_embedded = {'file_fastq': ['workflow_run_inputs', 'workflow_run_outputs'],
                             'file_processed': ['workflow_run_inputs', 'workflow_run_outputs']
                             }
    else:
        add_from_embedded = {}
        
    store={}
    item_uuids=[]
    
    while uuid_list:
        #find schema name, store as obj_key, create empty list if missing in store
        print('uuidlist length', len(uuid_list))
        chunked_uuids = [uuid_list[i:i + 10] for i in range(0, len(uuid_list), 10)]
        all_responses = []
        for a_chunk in chunked_uuids:
            all_responses.extend(ff_utils.get_es_metadata(a_chunk, key=con_key))
        print('ES_items length', len(all_responses))
  
        uuids_to_check = []
        for ES_item in all_responses:
            uuid = ES_item['uuid']
            object_resp = ES_item['object']
            obj_type = object_resp['@type'][0]
            obj_key = schema_name[obj_type]
            if obj_key not in store:
                store[obj_key] = []
            raw_resp = ES_item['properties']
            raw_resp['uuid'] = uuid
            # add raw frame to store and uuid to list
            if uuid not in item_uuids:
                if store_frame == 'object':
                    store[obj_key].append(object_resp)
                else:
                    store[obj_key].append(raw_resp)
                item_uuids.append(uuid)
            # this case should not happen actually
            else:
                print('Problem encountered - skipped - check')
                continue

            #get linked items from es
            for key in ES_item['links']:
                uuids_to_check.extend(ES_item['links'][key])

            # check if any field from the embedded frame is required
            add_fields = add_from_embedded.get(obj_key)
            if add_fields:
                emb_resp = ES_item['embedded']
                for a_field in add_fields:
                    field_val = emb_resp.get(a_field)
                    if field_val:
                        #turn it into string
                        field_val = str(field_val)
                        # check if any of embedded uuids is in the field value
                        for a_uuid in ES_item['embedded_uuids']:
                            if a_uuid in field_val:
                                uuids_to_check.append(a_uuid)
        # get uniques
        uuids_to_check = list(set(uuids_to_check))
        uuid_list = []
        for an_uuid in uuids_to_check:
            if an_uuid not in item_uuids:
                uuid_list.append(an_uuid)    
    return store, item_uuids

time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 
set_to_release = ['eae5ee86-b6ee-4112-941b-311d6026acb7']

store, uuids = record_object_es23(set_to_release, my_auth, schema_name, store_frame='raw', add_pc_wfr=True)
print()
print(len(uuids))
print(len(list(set(uuids))))
time2 = time.time()
print((time2-time1)/60)

uuidlist length 1
ES_items length 1
uuidlist length 51
ES_items length 51
uuidlist length 300
ES_items length 300
uuidlist length 441
ES_items length 441
uuidlist length 144
ES_items length 144
uuidlist length 49
ES_items length 49
uuidlist length 17
ES_items length 17
uuidlist length 1
ES_items length 1

1004
1004
0.7759940346082052


In [8]:
ES_item = {'embedded_uuids': ['0003ed8f-64ed-4bd7-8331-3fa739507671', '00e22704-f080-4073-91e7-f04c0c2acd7c', '00ff4c9e-31ef-478a-82e6-c43503bc4cff', '01bc8c8c-083b-4119-bfde-e3272dfe9e5b', '01c28366-8dd7-4769-87d0-ce2cb0894139', '01ca19b6-4652-4eca-9f38-f0e93acb58fe', '01d94530-2ebc-4449-b0f0-a2fdb5e304ee', '01fab0f2-e76a-4a14-a77e-e5239dd391dc', '02051c92-cbb4-4b95-a7a8-1d65897ce112', '027019c5-88e7-4357-8363-dde26483b8cb', '029e34cd-a40f-41b3-8066-d2ff514e7d3a', '02b5bd81-1702-4066-8231-7df334b0156f', '02e584e1-8e53-4415-9503-cb2ed8f2c07e', '02e8d673-2ed9-4082-a027-2da57f056f0b', '02ec55b7-2961-48bc-bd54-9cbfc1e2320e', '03168ced-bb74-4438-8cd4-e3f1d348be8b', '03418f85-9977-4c00-ba9f-36b1c3ab2e4d', '03af4d9f-3171-44db-93f5-ca7326b7a6a2', '03bb9328-c6d3-477e-93ac-a72232e9b93e', '03f64d2f-9be4-47ea-aa53-25cece95b8a8', '0429b923-dc29-4514-a1a2-2d6d1d76d973', '04afedd0-1dcf-429c-ad77-341855b6746d', '04b2c874-95df-48e9-a072-cf383cd4e0a5', '0613a2e1-b543-4da9-90aa-9dad054e496d', '063961e2-891e-4b2f-be49-97a400f9a1e9', '06618a6a-2001-4033-843f-fb56797b71f7', '0683b6d6-5fcb-4135-ae23-ea1cb9dc2d92', '06982263-0ec8-4c25-b8f6-04b08abc0bbf', '06ea837b-093e-4cc5-bc1d-87ac9f19f6c2', '073667f1-ceef-4b3d-a4db-c48ef11a7ffb', '0775a179-151f-4531-a89c-01ff2429acdb', '07b05436-e2da-46f2-90ae-f460dd7abddf', '07c8be77-ab7d-4e6c-8fc4-85a346cf0ebb', '07f60950-c5e8-411e-b6b4-dfdf9526b53f', '08122668-07a4-45ec-9864-1a5460394e0a', '0842df74-c469-418b-956e-ba47183e14bb', '08772fb9-b432-4f75-b01e-2c2aca7f9c41', '087a6256-a52a-40de-99aa-f5ab2416cb2f', '09ee64fe-0c97-440b-bf81-7106ac5620fc', '0a44ff64-237d-4e9c-bd18-68657194d643', '0b5cbce7-71dc-4e27-8246-f2b636102b85', '0b7cb7d2-dd17-4d2c-a80a-d2b6228abce8', '0bcf3871-67bf-482d-9a89-ab99a47cdb0e', '0d9b34fc-1399-42d4-ab91-999b166b0701', '0d9c0bdc-05f3-4f73-8340-9774ca55408e', '0dbdaebc-67ee-46f4-b041-15044a741bae', '0ddedcbe-5167-4b3b-a223-42d6bce3127b', '0e329024-c611-4537-b26d-d26519f94fe7', '0e4bf888-08e9-40f2-a7da-a33efd00aeee', '0ea74454-5d62-4eec-9994-9864d7e15b51', '0eb67115-70ff-4a35-ba22-1803839aefbe', '0eb98dae-4970-4716-8145-5c3c5c4b1073', '0ee90d06-3a98-45e6-ba37-0ba6214a878b', '0f01d1a5-2469-4a66-ab39-e47fae555d10', '0f020b60-545f-4677-b0fe-428fa405cf65', '0f3b47e4-2365-4b5b-94fe-35b0fcc0b3c4', '0f606c1c-bfcb-4b5d-a9e9-ee774080c5d3', '0f984601-84aa-4754-8ab9-ba11412487e9', '0ff4313b-53df-4e74-b129-a3f5692b0088', '1025af37-366d-49e5-b492-120023fae219', '10dc8592-5e4b-4e5c-ab7e-2444217a567d', '10e47432-8191-4908-b402-6e9e4837ace7', '10f3f998-b87f-4998-8d5e-144e0eb363c6', '111189bc-8535-4448-903e-854af460a233', '11392c03-15d4-494f-a748-62cdc780dc83', '114f658f-8245-432c-8b8d-56e06bdb7507', '11d5e29c-fb38-4640-a55c-9bf90bd17542', '11ed4e56-3f47-4cd3-8b14-aedb403189a9', '11fa57ea-95a2-4a59-ad69-5f4f70e6093c', '120b4ff9-350e-44f0-bff5-254d71751b1e', '123c5108-0073-4e57-87ed-31db8ec22fd7', '12a70373-d35e-4dd6-b3f1-1b30ae6a1a53', '12c0e1a8-c3fb-44bb-8c92-f6cc9d288752', '12c9e3f3-d382-4fb2-b024-e7c3c1dcd02d', '130d5ee3-067b-46fa-af40-4b715e5c42ac', '14949ebc-4257-4fe3-a375-fe687034f7e7', '149b37bd-eff8-448a-832e-ae592c52fd2e', '14edd7fe-90fe-4072-94ef-8328c3f3e735', '153b1e20-7bca-4890-8d21-d2b8a29f60dd', '1584efb7-9d05-4ca8-9822-9e17cba720b2', '15918da6-2476-4a01-a736-621e01e636b4', '18324f91-fdc4-4d9c-85d1-6656e67d22b5', '18336371-b9ae-4255-942e-32415f1253b6', '185e765f-aeec-4277-81e2-70dcf8b56893', '18668ac8-a5ea-403b-b7f0-8cf62faf70f1', '18852a4b-cece-4549-894e-2160465e01c6', '18d5c79e-a424-4bad-ada5-a471f7b02ea6', '18f9c9f7-1099-4ed4-8071-aad033c5eab8', '193f2cea-6904-419f-a7ff-94f224001a56', '19e075ef-24de-4b0e-b08a-a2b343541aee', '19f10ff2-e95b-4901-8365-158ccfc001c6', '1a60b3ab-fd39-4877-8af4-357359627b71', '1a95f073-c8ac-4b56-b11d-2c6a02cc7f30', '1b01a20f-9edb-4427-ab74-ee4ba82bf08f', '1b3e2cf4-a5a8-4e53-8449-71cc8dc17ddc', '1bb25dd6-676c-4f7d-a1df-c59705faf03f', '1bf9403c-0513-4616-acda-84f2b2fa20cc', '1d135c5c-4b43-4f64-a596-42393bac455c', '1d72e443-1975-41d4-8ef9-6dbbbb0ff30b', '1da0e678-e610-46ab-aefb-9c466f7cb4ce', '1dc2472e-32a1-49ea-bf6e-6a722061e4c5', '1ddf2fb7-dd07-4364-9154-636ae293c68a', '1de7f3ed-4bfb-43b5-b63b-2ab28f039ac4', '1df0c71e-b8a3-49cf-9bcb-c342c0736270', '1e06fd23-07e1-4200-94c2-952bbcb57a91', '1e1bb0a2-017c-431f-8e69-c3bc5deb4c77', '1e240fa4-10d0-4fa2-b281-3cdbf0ed3584', '1e24dd56-c884-4d78-9145-acaf4b1c0c79', '1e73764c-81f6-460f-96b5-a2771601fad5', '1f400f8b-9561-4b46-9e09-0c4c83c09691', '1f4012c8-8b5e-4c5b-9604-c35d8a5e3c21', '1f47cb15-c562-4b06-8cd8-0ef5c8ece340', '1f860ec8-24e0-484e-bb19-67e22360b632', '1f930c17-3b84-4c7b-ac44-b484ffdc19d2', '1fdb014a-e880-48a2-a341-cdf0acd5de0d', '1fee3029-42fb-433e-bc35-c62a0b8d73b0', '20a136df-054e-4159-b9be-3a7a613e6b4c', '20f2c990-4a30-4599-9058-44e66fee686d', '210b3480-c15a-49c8-ae42-c8f066eb1db2', '214cfbfc-1139-4905-b54f-1b48c15d5f95', '216911ee-35ce-43a3-9279-c42763d70172', '21b03f97-05ad-40f5-be8a-b6a054941a88', '21b9c655-043b-408f-8495-c3dcdd399c8d', '21ee7949-6449-4fc0-a0b4-28f4d958962e', '22351015-90bb-4121-bc1f-5b5d99c65769', '2299254d-2a5f-48d1-aac5-1138fb10a967', '229bed6b-5122-4be8-b359-d3587fcb04b2', '22ec4614-ecee-4d41-8f3a-d2fdb4c95483', '2324ad76-ff37-4157-8bcc-3ce72b7dace9', '235a431b-4e61-4755-95e8-47305735c374', '237569f1-f169-45d7-97af-6026f128d47b', '23ad5350-60e3-4665-8e0c-d7cc82814b2f', '24043c6b-25a5-4be3-a2aa-b467342e6e83', '2437af1f-f6ec-487d-be44-c4a0700ac11c', '24482a30-3f46-4e0d-911e-c76beaa1e19f', '246a6fe0-ed10-49d0-b673-adf56b7ed662', '2673dc69-29cf-474b-8fd9-f3e7129b21e0', '26a2860b-9db8-40dc-b4bd-4b47a7224c9a', '271132b7-6986-4fe9-8716-49144c52e501', '27459cfc-be97-4218-b832-54e39609f81a', '27719efc-6b2f-4c53-b11e-48879ee807bc', '278d8e94-0021-4008-9c12-74956d21ab4c', '281bc792-e782-4308-9d18-2d96bfcf6389', '284d5402-0107-44e4-bc2b-1aeb578eff84', '28b12db0-1f11-4a6d-ac89-1a4a44218c16', '2900fd25-c89c-4202-913d-37969357f2be', '2931fa71-cb22-4cd7-bce8-d26cae0b518e', '29b9f6a5-e31c-4c05-b1bb-31437866a679', '29ce9b5c-7c2f-4368-9c0a-ce66a7adfea3', '29da4ae5-07d0-4521-808b-4836d6aee9ab', '2a4979c5-3f44-4d5d-8ffe-5f17ba15b793', '2a4fb7f9-ff43-4666-b932-915584b4b441', '2a77258f-8aac-4a60-8bf9-2c595eae3ddf', '2ab85c11-14da-49d2-b428-385b6ee341bf', '2ad3484f-d2cc-4fd5-95a4-c1323fbb511f', '2ae11d64-4e02-40f8-b448-a396dcabb856', '2b217c78-edeb-4e2a-98b0-8b9154f3e70f', '2b315bd0-d852-4841-85d3-9ee1e23fdd95', '2bb873e3-b5a8-4c96-b143-3716a898b322', '2bc83039-aa06-4c96-9876-14ec2ad4e0c1', '2bcb9dcc-7dbe-4c1e-b063-7c691db4ce22', '2bd4fede-151d-4273-b2bf-6fe2ef51429e', '2c038815-bd38-47d4-bc00-333bd0696316', '2c0c7103-5a56-410f-b6c4-d91dd0e16bcf', '2c378e44-aa04-4f02-ba85-b679fc9b5f4a', '2c851365-b54c-4f0a-8b24-c8881e856f89', '2c8bedd9-6797-4bb7-b918-8783aff2a9c9', '2d512da0-bcba-44d8-8e44-7f60ee367e2d', '2d60ef3d-6bcb-47dc-8fed-7ba88401ddc1', '2de851ec-16c3-4133-b590-e9a87552eb82', '2eab3194-c7ba-40ff-8c5e-206c365045d2', '2eac4499-d26d-40be-8f67-4e98b7f50752', '2ec4b195-617d-40d9-9db9-ca4447834b9e', '2f2ffcee-df9c-4a10-966c-4dcb5dd5485a', '2f3a81c3-553c-44b3-9431-4b7e4b5a0e7b', '2f7dcbf8-e8de-4319-a44a-f383e446b027', '3062a2f4-dc06-4130-aa1d-8d0d87140764', '30b22cf1-e3d0-44db-8e27-cd79aabb3d6d', '31082bad-ade3-47fe-8337-56c45876d903', '31be8f2b-0f03-45c7-8e50-2ec5cda94b0e', '327abce6-8158-466e-ac84-720297e89776', '32a077d8-584d-4223-bf11-9f5df1d2797b', '32a0c6b1-9d2d-49c8-a806-c74df5686712', '32b69fe8-4c38-4351-a334-14cd92cd0691', '344fd27f-2406-4171-9c0f-f63c32df5abf', '34772abe-2d63-4b46-a0e9-6e4a2bc60c52', '3575ead4-0fa4-4fd1-af6a-46d5105628a8', '366a888c-1415-41db-9078-f5f0f32ec63f', '36727775-f63d-4bc7-b05a-1115af9af8f9', '36cdb32b-a34b-43c2-bb1b-97a56ccae90f', '373f37d7-43f0-49b4-b307-b9a56da6e2a6', '37486dcb-eeef-4132-8a75-760ae0850fef', '376e358c-48be-48ac-8a6c-0d313d21f0e0', '38d09d2b-5f25-4f05-acdb-555f46bb3bfa', '391d73ad-c768-48b1-aeae-622bf982fb30', '3962e891-9f91-48b9-9658-5b80e387af3e', '39716245-2513-479f-b881-82e16f4d452f', '398dad50-f67b-4595-9c45-41ca3c979ff3', '39a4a3ac-d577-4598-b763-79b121ea5271', '3a1dd6cd-b763-4304-bb24-2ac877b9cda5', '3a2f7cea-9e9a-4c8d-b501-f2f27776b425', '3a3aca22-7824-4179-ba02-9fdc68abcc2f', '3aa74d1f-f93b-4b79-b685-52d3b66dee77', '3acc368f-7ce8-44f2-bd28-c737c12aa639', '3addbef2-1a5e-40d8-92aa-d861c51a31c3', '3b558807-a86f-41b1-827f-9e7e5e831c5d', '3b7983e4-0bbc-4435-8a94-74d7c4ec753e', '3ba895e1-c270-407d-8c8c-49f105336082', '3bfa1c2a-602c-420b-8d28-5b1c1c954d25', '3bfcbd27-4b29-4989-9d76-7ea78d004d3f', '3c35fbcd-2c00-4c12-a75a-d1f340177739', '3c72a3ef-2863-4079-ac71-d3cbac232e15', '3ca2f1f4-98cf-4316-b156-49aaeafc10d4', '3cb303be-fb26-4359-951f-9e7c27c7f297', '3ce88a02-14ad-489f-a75d-48ea341e8c51', '3d365a04-9285-43d7-89ab-c5ea29391d6a', '3d43a37c-af1b-42ee-9d2c-d3843f63214c', '3d4d696e-0939-4461-97d4-867930011372', '3dcc1971-9c0f-42d6-933f-e4770147b827', '3e2e4a0a-ea00-46b6-bb8d-3867b9e3498f', '3e5dc140-4de3-4cf1-a2fc-11f9bce4f22d', '3e7d8342-c051-45d5-b04d-8b1bd62bdbf3', '3e8087c9-1311-484d-9cea-8dac908b0aee', '3ea0252a-b9f0-46aa-895b-df9615996e45', '3f2e38ec-79bd-48f4-b243-627836c78910', '3fc0291e-14ff-4f73-a3c6-25230f668b55', '401e101d-55c5-45d1-942b-d036f7d3e70f', '40b69cc2-d14a-43d1-a41e-fdd38c6dd3f2', '40b70f90-9f0e-464f-bfba-7616603a64ad', '40f4120e-7c12-4a25-9b2f-efc28b09e63b', '40ff719e-0f4f-4e4e-a283-57a87cc0902d', '411ce2e5-68bd-487d-993c-b01fd8ab7ff9', '41784f69-697d-42c2-82b1-b52bc5ea3519', '417c774f-0ca4-464e-8f29-44cbd7f299fe', '41d1f0be-32e1-4e23-a9fc-6210ee7665ff', '4330a4a1-143e-48dd-8acf-98f3c3a1b3a5', '437632e5-f7af-4a5a-ae80-eebf51dcce20', '4384a60d-4440-4e8f-9c0d-428a54dc43a1', '439e1ffb-35f5-445e-a968-e5414d0a6e4c', '43a1214f-f9be-4314-9805-29f9cbdc35fd', '43b0a759-761e-466d-bc66-7845ef275389', '444c27dc-484c-4784-98f9-5d41e04ee05e', '44da5b18-190d-4f94-9f50-f23208f0fc92', '44ea12d1-7346-4244-a6b8-06549f1fa181', '44eaa227-827c-4940-9ee0-9becfdd1b5f3', '453461e7-bf1a-43a0-9177-8010f65f620c', '4577d3f7-9e2d-4a03-97de-61335539534b', '45854623-e14e-4c0a-ba52-c478e32cd86c', '45f8d7f1-bded-4298-9d46-9e35edad4460', '4613ff48-2e4a-401c-b0da-137b666109c9', '4666b46c-4b7a-4454-b968-9ab20c08ec46', '46a075c4-2717-49cf-ae8c-6c108d090a5e', '46bcd592-6558-4bce-bab9-799b21b4034b', '46e3377c-6caf-4ade-af4e-ca6f81ac2f9e', '4704ffa1-b916-4c0d-86e8-677a182ce2b6', '47abd88c-f56c-4ddc-b959-486b3bc16e9a', '47d6f544-6405-401e-9bad-e41ad0f43ec3', '483695a1-b525-401a-beb3-2decbe1884a7', '48b3a641-f51c-4438-b2b6-c94ab2a1864f', '48b88e33-bc15-477a-8943-0b5fe65af623', '48cdb7b4-1cc3-483b-b95a-dc276ac21d07', '48d9dc72-4f60-4dba-b20f-6551f1624aaa', '495f8b40-1177-4ed9-bc53-3737acea8ae2', '49937616-a384-484c-bd5f-fb9dd992984b', '4a0abcd5-a99b-4b97-b026-d8758fe75c42', '4a62480f-aa50-45df-8881-da2c95be9774', '4ad020cc-4f7b-4af6-b952-cb72b86daf38', '4b10a69b-146e-4e74-ac98-b2342a698522', '4b11d240-5acf-4577-a18b-9526b17665b6', '4b43218f-af8b-4395-a130-027be3cd82e0', '4bce48c8-23ce-45ef-8dc0-80225c8e973e', '4be6a125-defb-423c-a140-37cb967c93e2', '4c0dd361-e0e1-46dd-bfa0-e82344bd3959', '4c1bbb1e-75e2-4331-880c-0a47692c6b45', '4c7ec1c1-1693-49dd-b1c0-48a4b75a4e15', '4caa37f3-b01f-4034-bcd4-3fa7079602fe', '4ce53fb2-987e-4164-9c33-029a55d08cbf', '4d028573-d2cc-49a0-a5ca-80906a269152', '4d430e0d-b79d-4bfb-be54-b5795838174e', '4de30270-87dd-42b9-b5a7-fd8fb7d091da', '4e19b7ac-86fc-45dd-9a64-ccc1ba61ec41', '4e31b135-9a59-46a6-a9df-8b7bbf2b194e', '4e331085-75b8-495b-ab08-23110ace1d30', '4ea34fb7-a41e-4e47-8d87-9bc2becf1e99', '4ef7308a-cde2-44c3-8a13-6c2b10d37404', '4efa4b0a-5587-4876-9d33-813eb1e7f491', '4f70783d-063a-4651-846c-6e83cbaa7bce', '4fe066da-7f61-4754-878d-5628f6585e03', '507163ca-1d4f-4a3a-a9a3-4bfae5017eb0', '50eecc37-6569-4910-bf62-1c1ecf1b5a57', '50f8300d-400f-4ce1-8163-42f417cbbada', '51af7800-1541-41c1-9c5c-a793311d4781', '51be3a0c-90bd-4e40-844c-d95900b61756', '5228058b-398b-4bbf-92ec-304279ab1d89', '528d6fc3-7760-4c36-b550-b1cb6910461e', '52cd7f15-a526-4fb0-9442-562c9e2c3696', '52ce302d-745d-4bfc-99e8-f8092a9f5cee', '530006bc-8535-4448-903e-854af460b254', '5343effb-62ca-4f49-98ba-28747670cb54', '538b06ad-ebfd-4ef5-8e19-56e4b982e9fc', '5396f6de-15f3-49a7-a031-e7e81c122195', '53d9ce5a-75db-4aa5-b96f-09da09bd3ba1', '5402ee85-d568-4d40-b86d-fec615f5b419', '548acb46-ca54-4b6c-9597-02a8e4f5e4d9', '54a69337-7ba4-4c78-8a80-c9d0675b4850', '54f877c3-9e06-4d4a-93ed-5e3d5d0ddcfc', '54fa576f-98cc-4ec9-a6c6-d988328a81bb', '55042510-50ff-45b1-a972-04496709e5bb', '55cb0f75-30c0-4493-9b5f-e561da2a5642', '55f132c9-7fdb-492c-81ce-fd55be956246', '55f42202-e4b8-4ef6-b91b-c4c1820f96cc', '568b426b-866d-4817-ba43-6efe2071a85f', '56a58e85-cb4f-41d3-be16-cf95111fa5d1', '56b88122-94ab-4a10-b28e-8ef2e71a9a81', '56bcd0d9-0f7a-42fc-b949-f6eba2a561a2', '56cb7caf-1f93-42ad-9237-39ed7166bd0f', '5704b3a9-f889-48cc-b19d-64ab5b0564b1', '57501104-cbc6-47af-8149-92a977f9240f', '576cd077-0010-464a-ba15-70546e725531', '57727ebd-425c-4d57-a2fe-f57bdac0195f', '580f3d5d-cfbe-4536-8cb3-b226e76d0c45', '580f8c0f-e5a1-4c68-94e6-d89c16870ffa', '590e9a44-537c-4799-8779-24b14b3302f5', '599c5d2f-eddd-4f76-97e3-554d6e539e4f', '59da6335-ef3e-47f0-a80a-0f09352a25a9', '59eaf14a-1e59-4936-a68d-0c29f8d59843', '5ac960b2-9758-406c-94f3-c914f812c46d', '5c20d4db-add8-4db9-9c29-5ad642f183cb', '5c6d1a0a-ff87-4e6c-b234-f648b851273e', '5c8c9652-7eb4-4a7e-bccd-a0f96f965f81', '5ea3a819-5f27-4a01-a916-e5d258759672', '5efd514a-f072-4537-bbfc-0707ad127577', '5f1719ce-8e36-4d79-81e6-cb370739d0d5', '5f33da65-0211-418c-bb5f-5510b1b2aab8', '5f53f149-f653-42b0-a903-d12647fd4af6', '5fa22568-3b1b-4fcb-a200-7a3ad2e28d63', '5fb2966a-1807-4699-96dd-62a1e4066669', '5fc63fe6-b29e-4681-96ff-052c872af9f7', '60188dcf-2c60-4b56-9856-114c60937e7f', '60446cf3-3b1a-49f6-bad2-c492e20eae7c', '61ad62d0-cf15-4859-aefe-c52bfc4767c6', '6301c7cc-5ae7-43d1-85db-610f7973e9bb', '6328e19f-d31c-4572-bc36-f757035844b6', '6343d57a-213e-4ac7-89ec-8b5e74c21bd2', '636fb808-d2c1-4dec-98cd-f36e4b1061f6', '63a43224-cd34-4ff7-bf3e-0cc66ebf381a', '63ab19c4-86b6-44ec-8b1e-5912dad01cd6', '64213148-f0b7-47d1-9d57-fbdaad09c97f', '643007f9-e7bc-4a2d-ba49-1fd0ba484b3d', '64381efc-9e14-4bde-9817-0b5d8d5f9003', '65015fc1-ce32-4c31-821c-d105bb0a82be', '6520dff6-cc1c-4a2a-a67f-2fdfa889e582', '65504487-18db-4229-a4c3-8b10cec51f86', '65bcc2f7-ab2d-4439-a28e-0a753a609835', '66029258-f7b9-47e0-8c72-01eead159a98', '6630f5a6-567c-4799-8c9c-c5fdad74ba61', '66526e51-0ce0-4417-acc7-1564dd20324b', '66b5bd83-6b3b-4a19-b372-a4f88ff3b9ef', '66d83ee4-2757-4a6b-a3b9-cbde81eccb21', '66ef3799-ec50-4dbf-9753-36bea5b1e76d', '67f86a25-7ba2-4c5d-9b11-ca7363f0e0e4', '67f99df9-4183-454e-986d-7576a4f62309', '68172441-97c4-40cc-b73f-d0f5dbc5cc05', '6851452d-1227-4e68-bf9e-b5a00fffd22d', '68b0ff45-92c3-4aad-8887-3606d9c9d21d', '68b9c110-9c22-4003-b3d2-93c467dfd077', '68defae2-f421-4e89-942e-f421e67f336c', '69494da1-fa34-4aca-9ff4-639073e3e10b', '69658dec-ce10-4f34-a117-13781bd571c6', '697f8022-7bf3-484c-87c1-3d77ffa67439', '6abe0604-39a7-48ef-b653-4d23fc5c6298', '6b08421f-f635-4a01-aac3-f0d2e6f900d8', '6b0d2ffc-5fb3-4c77-befa-b82b0ad0e91e', '6bf42006-2220-4c14-bf70-ac6c3f94b00b', '6c3586da-4567-4d91-b479-7fe33330256b', '6c4f5dd0-3982-4e95-8027-f89709f3d6a7', '6c664b74-2794-4f8a-829a-754c18ef427e', '6ce025db-dc9b-4c66-ac74-0b23efd6da71', '6ce17b03-55d1-41d7-96eb-9203b15b2c77', '6d086a47-e980-4f2b-b5d3-407a53eb6638', '6d269c56-87d7-41cf-8a4d-b178db7ae157', '6d8aac76-93bf-4423-ab15-aae9c66d7ecc', '6d9acc40-be0a-4ed0-a893-df5c4db25b52', '6da1a1b2-58aa-4bb4-a2af-131891d79384', '6e06c80b-23ef-4957-85f7-b94ba0420984', '6e378c15-bf09-4304-bd45-e8a444c2a5fa', '6f31119f-e3cb-4052-a8c7-c74f0f474676', '6f6c9cf5-4e4c-40e4-b2bd-f5e2360c1aa8', '6f9714dc-2138-41fe-915a-9d40902efa67', '6fc80dc4-0e1e-47df-a9cf-a19ba740fa5a', '6fd5b473-3f46-4219-a5ec-0db93342a728', '7013a367-924f-4002-8682-bfc9a1ab0904', '708af918-6bb4-4a7d-b0dc-2a3706953364', '7153c1b9-4d08-46dd-866e-ef8333dba917', '71cf239c-25c0-4fcc-b265-f88dcbd53809', '720c6ec6-f5e1-4f6d-8a60-9a45c7cdc013', '726fc6f3-9cf2-4ee0-93b1-d51d1b1df1d4', '727b9399-92f9-41f2-a61e-8532fef538dd', '7299dd07-220c-44f1-9e56-bff6b453460b', '72af586f-4d8f-44b1-b0b6-feb2eaf6993e', '7321fb29-e465-4baf-ae3a-cf88defcd9fc', '7442030a-be29-4d3a-951b-014f6958fe8f', '7463ec99-c203-4c4c-a221-b899fbbfe14d', '748db193-9256-4976-ac6f-c4e9e823ffbc', '74a4af26-0e2f-4818-969e-ae772af46cba', '74e09d91-0329-45d1-899d-7b9c450926b4', '7521f981-0027-4121-8afd-bec7eae22efe', '75754d4a-0b50-4281-8975-c4f3936cf1c0', '757f213c-d802-471d-b857-74258bbe037d', '75e19a0d-f2b4-42fb-8716-70a79e14fd69', '7601d9a0-34de-477c-b7e5-6ac26f1a855f', '76450ea8-d15c-4572-ace8-a7ea947de5a8', '76ec7d64-7f75-43fb-b112-3d05925377e9', '7745b647-ff15-4ff3-9ced-b897d4e2983c', '77b6306e-d3e3-493e-a242-c6b81e971db5', '7876e906-1898-4097-8490-d657cfc76330', '78775159-a731-44d0-bb9e-281787541407', '78a67c3e-6274-4f13-b998-7dd0d8ab9f5f', '79112ab5-1aa5-4bca-a89d-3949fe4fb661', '79384268-41cd-4840-8ff7-1de6cd47a19a', '79797967-00ba-4c13-ba2d-eaa32a44df96', '798b8174-fcfc-4acd-8cdc-8a40e2b84cab', '7b611fb0-8339-402c-9281-b7fa13f0c4be', '7b88cdd6-233b-4400-bff3-76ea4668881a', '7b8c2345-1b50-4fe4-bac9-5e9f61fdfc2f', '7be9606e-7a12-4a0b-b204-8478b51873f9', '7c3b073f-a110-427e-aab9-3a51050692ac', '7c5d5239-7fe9-4060-9e66-6572849ef296', '7c60a1bf-efec-4315-9500-7e854a5c12ce', '7c718f52-3bde-4175-a485-0d522d62b043', '7c91002c-bd0e-46f9-ab15-a11226604604', '7ca9afd8-106b-4a8e-b403-8e110f2447fe', '7d02daa1-5fda-442d-890d-7473ed261331', '7da005eb-81e1-4526-92e0-7fe8ff2917ae', '7dbfffab-470f-44c1-a554-052065ec6047', '7de8cb5c-579c-4cdf-8690-f4c9aac2d4ad', '7e2b2747-2210-4d31-bcad-104d545d15a5', '7e45306a-ed29-4065-a25b-acc3aaf112d5', '7ee5d8af-16e5-4376-a7e8-e59bb11e4ed1', '7fa3e6da-c9a5-4a5d-aaf5-b45cd4480aea', '7fd6fb2c-f2a1-4e5b-b45f-f9c5a55b3d08', '801429b7-a742-4c62-ad8f-5ea755656669', '804d8f54-07b7-4f82-8c2b-71539b508aca', '80858597-c2a9-4cb7-a4ba-c893decb8af1', '80bca017-6891-4cf2-a0fe-26c90aa3b4d8', '81324f54-3127-4d68-8c7a-997c1727e0a9', '813a81cd-59e1-4068-a1da-6e02a1e6b45c', '814b6aa2-6868-4590-8011-90de1100e54b', '81f71af6-8307-4c09-8a49-a74c23c544a1', '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989', '82f5aec7-4803-4ad5-8225-ee31a04fd632', '8300c32f-9a81-4af1-855d-8ba3d9e09edf', '8310a3b2-e421-4656-84dd-f7b0abfea638', '832736bc-2995-4b64-86cb-99f68df0d464', '8395504d-0d4c-48a0-9cee-584fc258348d', '839a4264-6499-4f11-b349-bdc93236cb28', '8453ba2b-6cd6-49b4-afd2-b740dc9c7296', '847739c0-3af0-4be8-9fa0-16c3071b4e29', '84943bfc-4167-465c-aa0f-7443c88d92e5', '84e34ca6-8a91-4eca-8aa4-fd7e5f546395', '85490436-b099-4760-9d4e-621e2dea4572', '859badf7-4a3a-4a1d-a97d-020f54a2a103', '865ef54c-1b77-447e-8e0d-de8c744f86be', '86643679-3d06-4684-816e-06d6a473e1bb', '868895c8-2497-4fc6-9fae-854345da5d8d', '86d44292-e30e-40d8-9ef3-899105f3af1f', '877bcafd-c796-4aa2-b8a1-bbfcdaa144c7', '878d02c6-72b1-4429-97c0-2c9a6a142279', '878db655-beff-4652-a201-e71864b7cd36', '884ab4e8-d169-4a9d-8414-4f85cec5c0ac', '885673f9-e004-4748-938a-74e7bf2f7d11', '8896846b-9c19-493a-ad78-b29162eea666', '88e9de99-1269-441e-a2b2-2db6624f1643', '890104a1-6dde-4d09-b847-dcd71c908852', '890ae6ed-cc5e-4472-9370-ff2bf939950b', '891b53a3-e6f0-4d28-b370-227e45051f46', '8b29449e-55b4-4767-9399-18aa9612f310', '8bd453dd-902c-4e00-98d9-69dcfb5d96ea', '8bdc8002-c4cf-43ca-b0f0-0a35eb688101', '8bdfd1c7-09e3-4d14-969e-9cedf03003f1', '8c545779-f107-476e-92c0-3fc3e9418edf', '8c8882b2-eeac-4594-b1c5-9bfc39532abd', '8cb4f8f2-ed9f-4875-9b1f-6dbacb26d0d2', '8d178b40-ffb7-4766-94c4-0dce43501230', '8d3f8614-18da-4717-ab5a-90e369d508b1', '8d4773c8-5573-4f20-ac34-2f7aa1eb449e', '8dc994e3-2aa8-4caa-be8a-20d413e6a518', '8e0e4a91-ed32-4ef6-9a0b-23fc04555774', '8e4edcfe-c3eb-45d6-bada-c9cafe4faefb', '8e70c8a8-6c07-4f31-a7c6-2c2dedf3f153', '8ede0774-7923-476e-b589-a453d32b2a9a', '8eff8235-f121-4417-8005-8cd35ea56e88', '8f355919-6a46-4928-a223-b8f5358210c3', '8fb10e27-7b0a-412a-962f-ab703740e197', '8fe88d17-a0fd-4366-8f4a-ec931aea4a61', '90225d20-3737-4eee-87fb-d2ea7a42d135', '9039221d-be97-4d67-8e74-c591c8bdb088', '904455a5-d8cd-42ee-8c61-a91acd503a8e', '9062f200-f8dc-4096-97b5-b6ff9ed15e33', '90ab23f5-8db0-453d-b958-e56daf8a173b', '90bc7ff3-bf6c-430f-9a07-1026e28c2d60', '90beeed8-e401-4758-a543-f8dc2433a203', '90c84c90-553a-45be-bb3d-2d11801d05df', '90e7c0d5-35f2-45d9-9f42-1db6082c5319', '912d26ef-d305-459a-908d-808a88aee5ec', '91b694c3-f4d7-4ddd-8278-16f94e15c1c5', '91c4ce9e-1620-4cf6-9127-23107829bc60', '91dda5b8-7818-4f21-b81f-2d2655d4e03c', '9274cbcf-8ebb-4626-8756-5ffbbce5b4a5', '92b65796-33cb-4b1a-98a8-b25f27e92cfb', '92d85c9d-885f-4c9f-bdc8-273eab66babd', '92f90aed-7df1-4bd9-9e74-a472cb50d663', '930fbb26-06a0-4051-b3ac-4effa9ee6924', '931d1b4c-3015-4b86-af1c-d2b5a6fd09a9', '93325ccf-72c8-44c4-8292-c79b2aadcf40', '934b8a92-3adc-4991-983c-dcec0191f5c8', '93849d35-d246-424b-aa08-cce3dfa94530', '93946918-5126-4ddb-adac-eb171cef0bde', '939f7cd8-e01f-4fd6-8d35-15e3328661fa', '93e55e41-90ad-4c99-844d-d25e60303003', '9485316e-7137-4ca6-ad1f-2894a59eaee6', '94ce0c14-9ad8-498f-848f-5d6dc1d80153', '950172af-87f1-44d2-aac4-22112322efac', '9510e997-27a2-4260-861c-efca9efa542a', '95a56624-fd10-48c2-b7eb-12b1b056d50a', '95bee459-3ca1-4421-afb6-c7f09475d827', '95d81a56-7de0-4113-af7b-62ea60dd4cdd', '95deb527-6d19-45fa-ace9-4e5d4ed1e023', '95e5a239-3010-4d3b-ba53-c1439ea3f89b', '9604c8eb-96f1-4008-b5cc-7055c1016bdc', '9632cfd3-e140-4b18-830b-a61474b8248d', '96b02cf1-f8b2-448a-b858-e99492c737d2', '972c0f31-244f-40ac-9961-308a22babe35', '97acc49e-fd3b-46e5-adf2-cf9112b52cb0', '97b5785c-b2d7-4b59-b8bc-448cc9a58f89', '97b74f37-00f9-4636-b384-d466a32432af', '97f3f525-30ab-4ec9-b31b-6ef008112ab7', '986b362f-4eb6-4a9c-8173-3ab267111888', '986b362f-4eb6-4a9c-8173-3ab267227777', '986b362f-4eb6-4a9c-8173-3ab267307e3a', '98c4cba2-8cdc-4b37-97ff-d0974f6b4729', '9a6675f8-3dcc-44c9-b4d9-d954acdf69b9', '9a667a78-abfe-4983-bcb1-3aba11347ca7', '9a7fe685-b500-451a-a53b-f945880ece63', '9a9ebf83-1ad1-44ff-8a70-e0d581785c87', '9ae79d4c-0a7b-4bd3-9439-34bbd06a76dc', '9b43d6dd-e505-4a48-97df-e354458dc12c', '9bb81b4e-1f0c-45d8-bc06-15f3b8fd6de8', '9bd01d51-3aee-4535-9071-752499ca6a70', '9cc0442d-9a51-4e7a-985a-aa345d0e0322', '9cd05f6f-d339-473d-a566-8eec65ce51e6', '9d374f95-9859-4ad0-90c2-9758e1259a04', '9e995add-688a-42ec-b86a-1571324b923f', '9eb7a058-e831-47c4-bd97-b8938f979728', '9ebb2014-6979-4e60-9783-96b673453fe5', '9f4430d3-c70f-4f68-8b0a-57ab22476dc7', '9f53bc06-50e6-42ab-8ad5-a29d6b21b71f', '9f6db11c-eac4-4e6d-9c40-011d1127bb4a', '9fb452a2-b24d-4f5e-914c-80b73e7653ff', '9fbad3fd-3c10-4c27-bbe6-0ec536f9e963', 'a01ec5fc-5016-43d1-a3fa-f435146aab02', 'a0a2bee4-4204-47fc-b945-c696b7e12b15', 'a0d4e212-d1f9-455e-8410-b5b533112097', 'a0f5399e-57c2-41a5-bd76-8872c46b8fa4', 'a13b6328-4aad-42d4-81dd-6d07c39600b4', 'a167963f-7e11-4dce-b5f8-24e9c8cbc4dd', 'a1703df0-8fd4-491e-aee4-26403decf498', 'a17bbfc6-6955-4e7f-a150-937dd2d65fc8', 'a20ead51-2d13-4fb9-b781-c2b172beb4b0', 'a247e119-11ab-467d-b648-ab4e92926dda', 'a252b998-1ff9-47b5-bf1d-b10bf8005ce5', 'a255a3e8-ef0c-4d8d-a54c-5efb6256b45a', 'a263eb95-a1ac-4c39-a043-f407cc2ebb65', 'a2911f72-9aba-45c4-9024-14018792a0e7', 'a2fe8bc2-3e14-4cfd-96c2-1c2bb395a3b1', 'a30a8601-f28c-4f47-9b66-eb1cc722183f', 'a313969d-81f7-450c-bb17-09195f723422', 'a31f8fe3-01be-4fae-9ca3-9fc8de4b7193', 'a36726ea-da51-4736-9f10-f13fd7560ca7', 'a3f1f14e-dd1d-439d-8bc6-32a1228a48cf', 'a401eee2-3dcd-4512-a51d-b29069ef3ee3', 'a4104023-d705-4a11-ad13-705c950ed59e', 'a4827f07-f508-401a-b7ef-f675d0ebb343', 'a496efb0-ade8-4243-9258-9458e248c5e2', 'a4af5721-6432-4cce-9a50-c6f499a75202', 'a4b83dbc-5a6b-4268-b45c-795f77c56eb9', 'a4c5dabf-a3fe-4894-a49c-efefcc4a541b', 'a582c75c-ec7e-40ff-95c7-a0d510caf95d', 'a5cde8c5-5172-457f-9fe5-9e2073fb800c', 'a62e5d35-ce10-44e0-9075-e3697dd60b3b', 'a6308778-f9ed-407a-a422-e71da88a8dae', 'a65740ee-db45-46ff-ad72-122bbb0d108a', 'a662782d-174d-41d3-b0e4-f9e0f3fea7ec', 'a6839631-d38c-4b21-ba44-b99e9d8c0c07', 'a6eb02e7-e543-42ef-9f7c-fa75c4552011', 'a6fc9d41-cf80-4eef-b090-2352ad99409c', 'a7731fd1-7763-4865-8ccd-ab1eb92cd9e0', 'a7a90458-4243-4cf8-bef0-82c3f7d6ab41', 'a81e847c-7016-41c2-abde-5d016c4afccf', 'a82023b3-080f-4cd7-aa92-fdc30be405c8', 'a8310275-9f12-4103-9029-964e67a83c89', 'a85ceaa5-6348-4911-b88b-d40f64f48ecc', 'a915a342-5cf6-4d61-89cc-4cf5ef6006cb', 'a9802b54-3836-4cc1-a343-174b799bfe11', 'aa370560-9977-4298-9147-db3162af5b8f', 'ab161bb1-71ab-405c-b2d4-29c412548a7a', 'ab371aab-3fb7-425e-8328-6199902a6b78', 'ab933a1f-7f5c-4c4c-a7ec-4ef37b2f0e6b', 'abfa318f-15e6-462d-9b54-de24e5f15f74', 'ac4ed406-3708-4879-a057-830ef4e97856', 'ac654424-dfae-4dbc-bfd6-264c2969ab71', 'ac7d0ee9-ae2c-4002-8539-82a1e1918c14', 'acc3ae42-f070-4759-9885-aab25797e4cf', 'acf3e2bd-59b7-43cb-b9fc-0aaeaad35263', 'ad4b9a48-51bc-4d45-b4e5-bb870089f98a', 'ae12f563-7a2c-4767-b4c1-99e3d2458199', 'ae233230-7dcd-46f7-a282-813173990d35', 'ae6141eb-c45a-44ba-9abc-6cc05787bad7', 'ae6ff264-aa1e-41c3-b8df-02ef90bafcb2', 'afbbbce3-992d-46ff-bd36-ec54e97cf0ce', 'b06035c8-3014-4965-9fbf-23b7933c1938', 'b09e0dc2-00f0-4546-8893-0007877cf741', 'b0b9c607-f8b4-4f02-93f4-9895b461334b', 'b0c917cc-65a0-41f9-82d4-7a731e636ed3', 'b156a751-31bf-410a-86e2-fe4f897061e3', 'b159360b-8681-4baa-91c8-d495714424dd', 'b179105c-56b9-4a4b-9c36-b3c8409f0c9d', 'b18568b9-8d84-45d3-9fcd-e5f5c59b8c76', 'b1929671-8076-4e24-8db4-144e6fd81397', 'b1a8be28-c1f8-4505-bbd5-1c06c982c5fb', 'b1f7529c-59a9-47d9-aa12-9319275d27c0', 'b2b5bda5-a83a-4a31-bf07-e9c24b38dd23', 'b2b97fd0-7284-4bfd-b73c-714f9894f739', 'b2b9e518-14ef-4bb9-8aa3-a63a3b5c8673', 'b2c2deeb-e883-4ac0-b9e2-906e598884d6', 'b2f2fc89-07a8-4fdf-a594-5d0e9a92f3eb', 'b352a9e0-30b2-4837-a2e8-5a404ddb7d20', 'b371c45a-d24f-48bf-8220-9e76103467d2', 'b3859df0-da57-40cd-9b76-6ef5efc9b537', 'b3fc4fd0-7843-441d-8d04-02e6f18d54bd', 'b40aa1bf-224b-458f-9a6a-835177aebc57', 'b4778c57-5d69-424b-b287-9b505aaa6f6a', 'b48a1a5e-38cc-4e7b-8652-8fb0fe8bf7ee', 'b5267b15-f5a9-40b9-9816-d8138d9bf17f', 'b5457a2b-5e33-41ec-89f1-c37fd1686532', 'b5e13c68-4f56-4353-a769-4c09e530c483', 'b5f0e395-c150-46e7-b114-7f9be2f8a05f', 'b6760bb2-5a83-4133-b4a5-2eae62d0d079', 'b68d4494-43bf-4845-a81d-06586466dbeb', 'b68e223b-edf0-44d6-93e4-a4c6225266b8', 'b69dfc34-20ec-4a08-9975-2afaa59988b1', 'b6c8e21a-fb2c-4464-a89d-41408cd76ac6', 'b6d45df2-122f-49ba-ac1f-e93d71ca2b71', 'b6f630b5-95df-4292-86a6-8cdc250ee200', 'b83d9e9c-64d7-4c24-881b-64117aad2a20', 'b841ad00-61b0-4d09-96f6-d75f7479c366', 'b87dbb08-21b8-4765-b21d-6425ca84a132', 'b8f7974b-73e6-4b5a-ba68-2f7deac95bec', 'b9036992-9d8c-46cd-95d2-80714d22dc8f', 'b9280ca8-5a9a-458d-b4fd-157b7a099f99', 'b94e0001-19a4-4bd5-a383-9abc486c2734', 'b956d5d5-6475-4ade-86ef-3578f98b0142', 'b9aca08f-b2dd-45ae-adf9-04d97dfe7ff6', 'b9b15e4c-3efb-4ed3-b45e-876fbc9a6fdb', 'b9d84586-d23c-44c0-86aa-55bfbf428b10', 'b9e004fd-02ff-4439-b529-c12ae9a0592e', 'ba0db777-83bc-4bc4-beb5-f37f2828ad5c', 'ba2cbef0-7d30-44b3-84c0-168561062142', 'bb11d739-b081-4d09-b9cb-650f69edd911', 'bb489025-b026-4384-9e0f-9657ccd1e273', 'bb66f2d5-b60a-4607-8683-e99358ce52d5', 'bc16fea9-e36e-46a4-90a1-0ffdd4c29be0', 'bc3a884d-4b5d-4b34-964c-93073b2e0b71', 'bc3cde54-6c43-4b53-9a38-3991bd0fb3cf', 'bc96a34d-3150-4de7-9662-5acf71415a91', 'bd127d1b-fa3b-4687-8b31-2dd7e669d2f7', 'bd15695b-76c8-4c8a-aab4-f6a7b8a8b7e1', 'bd2e45e4-96fe-4b64-ba61-0550e4173b09', 'bd881a96-d1ba-473a-8afb-ff59ae4dba9d', 'bd8df9e5-4b82-4a8b-b3df-2eaa821b575c', 'be512b56-7f02-4587-a04d-1a338732262e', 'be64985d-012d-433f-87b0-0efefec06a94', 'be7600cd-dea3-4410-a37d-f140b173727a', 'bef50397-4d72-4ed1-9c78-100e14e5c47f', 'bf3d8c8b-fe3d-4c40-8d83-1897a4da5551', 'bf6979eb-0930-4a25-8ad3-de70eb7bc097', 'bf7e1a00-f2eb-4eea-8497-fefdeedfdc18', 'bf898e14-3937-43ad-9f3c-3c7557cbb293', 'bfa688b3-3497-41dd-b33f-378c868e6c32', 'bfca5c5e-64dd-48db-9acb-ad2b70f1f719', 'c02220c9-1f09-4e61-be2e-06364412d396', 'c07ba226-cdfd-4f61-b436-8b1b165d44ee', 'c096d50d-665c-41dc-b440-42057cfecdad', 'c0b4cadf-6e92-425e-bb77-8e13c68245c7', 'c11a4ed9-42d2-451f-bcb7-921fe058e6e9', 'c12cd770-0871-4f15-88c4-69f571e2a829', 'c1547eb8-e962-40ab-bcf5-bfa66ddd655c', 'c18f400e-9838-4b04-97b3-a42ef9e66e52', 'c24ac0c8-e0ea-44c6-bbbd-6068c4501c1c', 'c2544b4d-756c-44b9-b7a4-c8abb92c8842', 'c27e1314-5807-4fbf-ab36-3083480b08fd', 'c356608e-6448-4212-b554-839603ebcd44', 'c3ac36e0-4220-490d-a909-51ae62ae3baf', 'c3f50ed1-1a93-4597-ac31-ab6ab0b19a18', 'c3f6e06f-900e-45f9-9c23-983c5673f58d', 'c425f577-465c-4681-b0ab-6e73325b0fc7', 'c46bf024-6195-43b4-8cbd-6b33f589e570', 'c46d34e8-4165-4de4-a7c5-0cb8ce888415', 'c48e3db5-5415-4490-a5d1-261a1ebbd9a2', 'c5b19a04-c7ec-4450-b0f6-8c5aa2b5525e', 'c5d5ca3a-d06a-4f12-a81e-0d165cdaf907', 'c656f155-d543-49d0-8dda-e751084bd682', 'c66a036c-d0ea-4999-98f2-772fb319156d', 'c6861395-70a5-4007-9256-00cda8f4b498', 'c6bd7f1f-a03b-4394-84e3-a740c43b188a', 'c6e1086e-dbd1-4e2b-90e3-7f12098032d1', 'c70c6200-a51e-4747-b0c9-ae683eba9c6b', 'c7610bf4-4d01-407a-a973-dfa76057d698', 'c76580e8-031c-4864-8d7a-7fd65d4ca7a9', 'c8196b2a-fe7b-4bbd-b567-1614f3c85cb1', 'c82e80e1-8ad5-4936-aeea-65b0b90dcda0', 'c8828eb2-1190-43d6-94b3-e06cb4ac0d12', 'c8a46306-07da-4888-8658-59e57fd61dc9', 'c9260c17-6ccc-497f-b653-5551fd7e2399', 'c9265062-3e4d-41ea-8a4d-e11b3e984eea', 'c92e4122-fe08-4ca6-9388-466a12322799', 'c97775b7-3d00-4132-9598-863797b8ef14', 'c99d94a8-844b-48a7-900f-8d1077635a85', 'c9c61fc9-93e5-470e-8aa0-d276755c4f37', 'c9e94080-4a8d-4902-9fdb-a94eb817ece8', 'c9f10528-9818-487e-9efc-9685b0998569', 'c9fcfab2-c94c-4a75-a2a3-f3362c42ae12', 'ca59dda2-ff0b-4d12-80cb-806bdb38ec5c', 'ca5ff878-d59b-43a0-a552-1fbfa80ea7e6', 'caa1c39b-56ad-4b2a-bce0-74db5625e100', 'cb0e75d1-8c2b-4e92-8d5c-64cdb102fa04', 'cb1f7317-d78c-45b2-9e89-6f00a0d3439d', 'cba70e24-9b1e-4bd4-8345-488f1bb7b60d', 'cbe76c34-c205-4062-b1dc-52eb3e667a46', 'cc081549-cc1d-40df-80f2-665a0393fa2a', 'cc0b81df-b04d-4e52-b51b-90fe6c82f384', 'cc26bf88-1d31-4f0b-b8d6-8c839fa9bfe8', 'cc4b5465-3689-474d-bed8-b14bb03e500f', 'cca1f343-39ef-4844-ba68-4e01465b1819', 'cd0da383-7284-453e-9865-961513f3c5d2', 'cd0fff1f-7619-4693-a7a2-ee17bea2eb5b', 'cd2eb717-fbee-4e8c-be80-bb3dee696ade', 'cd3b738d-f081-4de2-b313-3326935f1927', 'ce462d76-a2ae-4c94-b281-4c8c797a9d26', 'ce512508-ec09-4699-9384-55eac2c090e6', 'ce814cab-939a-4bdc-8d15-9019f50f23ef', 'ce8748d6-7a9f-4eb9-b7cc-fe0c450cac34', 'cea20b8c-938d-4099-9ed9-9cd689b5c5e7', 'ceaef097-e704-4f18-9473-c448b23d4894', 'cec13e12-1bd9-4194-89af-6306dfb6a451', 'cf239feb-529d-4b3a-a754-02f20e1e201d', 'd01e903e-60de-4084-b037-2af25342ba90', 'd0744fb9-824d-44ce-bb30-25cbaa8ff337', 'd1014058-2806-423b-af2f-19afc363e08a', 'd149cfb4-39c0-4e09-82c2-24c2dea4fab0', 'd1da185c-ca4c-41d8-8bf6-1d0003e60455', 'd1ea98f0-7604-440d-bc1a-44fc6b4e5dec', 'd1ed5137-9579-48be-8c48-6e8c822feef5', 'd225feda-d505-4ba4-a964-7de9fba47126', 'd2de899a-73d5-491d-9d9a-35946b57d2c6', 'd3b068f9-31fb-4ed1-b111-1e7556d2a906', 'd3f25cd3-e726-4b3c-a022-48f844474b41', 'd424854b-1812-405a-932a-fad3cea927ba', 'd462d1d6-e6a0-4751-ba94-de2ef1d6492c', 'd4d816ff-44ca-40cd-b800-5079a78b26bd', 'd4dee9b6-c5eb-4201-b99a-4e4e1e7e8739', 'd4fcbc95-98ee-4bc9-92e0-cb95f59c9ea4', 'd508a1c2-f4d4-4761-8257-aec6f5548a10', 'd5196f65-817e-4fd8-9373-c57cbf16e3be', 'd54d77ab-5422-4cc3-a436-824615ce4ead', 'd57a90f1-90ed-44ce-b424-a7f7faf96f95', 'd68d353a-7e73-44a1-8743-603d194cc4e7', 'd6eb6644-3e38-4e5d-811f-4c138bddb53b', 'd71f4d15-4a0e-463a-af1a-0b93818c4563', 'd74b4cc4-e9cd-4c6b-85ce-ab257ce3fee9', 'd7c6ad0f-3364-4a4a-a409-1da0c139f6d5', 'd7cd1a94-6e7b-49af-a7e6-b964a0bbf79b', 'd7fb317a-3e26-4607-a73c-cc23c511eb2d', 'd8818a97-84c7-4822-aa15-2704ba365bc0', 'd89c030c-adac-4298-a84c-70a07592b7ee', 'd8ac2807-c5a5-45e7-9ad3-c13548888090', 'd9021648-5f10-4fbf-9c2b-d2aaf7c60943', 'd922db91-d616-47f2-abba-9cf491686c10', 'd949d6bb-b98b-41e4-a3b5-fb1767d37db4', 'd9f064e6-6c98-41f0-bb61-03d3b7a327ed', 'da7ac4bf-2e8d-4a7f-883f-8cdc4d584b8f', 'db9096d6-9077-43a5-92cd-04f089963b02', 'dc2f88c8-ec2d-4539-99e1-f08023642c13', 'dcb80c8a-11cc-4140-ad70-924f17108ec4', 'dceba897-c946-40b8-bb64-8776cf0d72d9', 'dcf2832c-de63-45b5-b679-bf58641a8b0c', 'dd8b1cd6-93cd-41e5-b184-caf1369cb39f', 'ddd3e600-fe52-49cb-9f43-7a8341aac38a', 'ddec78d5-2cdb-47d2-b0f5-c88f84dab443', 'de2d1c4e-ea6d-4142-b808-d7880bdf6a20', 'de58d8c0-3645-4269-9dbc-071442a1a18f', 'e04f2218-201f-4919-803e-73dcfb00cb6c', 'e0fcd96e-3584-4307-8d74-df29c1e8a7d9', 'e12ae3e3-3eca-4e7d-ad7c-9ff77806e1ac', 'e162c270-6e09-4e25-883b-8c27d832ab93', 'e180ecae-99ae-4d20-8806-4a1473f5e82e', 'e185bb8b-ff26-44f9-9faa-7f1d3c431c7b', 'e18a8414-7e8e-4521-87b7-f8838d7a66ef', 'e1a326ae-70d8-4795-8ee0-b5f62e80c030', 'e1cc7675-027a-4f82-aefe-31b27e83ab59', 'e1e75450-e091-425f-b89b-9981aab92c5f', 'e2043947-2d53-4bb8-8fe2-8e29dd2c9ea8', 'e23b7427-a73e-428a-9ba9-758ef4692584', 'e3059723-9866-464c-b22e-274c6dc90dec', 'e308794d-f967-47b2-a014-897b337cb456', 'e34557b2-92b1-4a6c-8819-d1c59e59f674', 'e35d4c51-353c-4342-82d0-0cba70020775', 'e35d758c-ab16-4e19-94dc-404bea0659ee', 'e37debd2-5f04-43d1-ba3d-bc4bd9b0eeb0', 'e38fd611-b908-462b-a1c0-fdb9abc1f312', 'e395550a-c186-48a8-94ce-3519d8b6a7ea', 'e4fe3ee0-a97e-4329-9d0a-c81314581c91', 'e51be014-69ff-4cdd-beaf-38e6cfea75e6', 'e59b7d84-834d-456e-8ebd-7d60147ee365', 'e5c7d366-3a68-4c10-bd0a-f294af4a9482', 'e5ca627a-7673-4c26-8c1d-c7325889716a', 'e6654895-fcff-4651-a7df-e20ddc455e40', 'e66a2db9-cbcc-4fad-a194-9858449c9c73', 'e6b6aa3b-c776-49a3-a4ec-3ef68a321203', 'e6b9e27b-7205-490b-a82c-b3e3b27113ae', 'e7956292-5cdc-44fb-a3e6-ad0cad77ae6a', 'e7bdea0b-0df2-4e0f-af8d-e5e342a0831a', 'e7e200c6-dcae-4aa4-b0d6-08888eb8697e', 'e81634a4-fa04-4af9-8f5e-07edd78ca898', 'e8a09e70-6605-4f53-b032-c45cd52c7213', 'e8a11e7a-3783-451f-ba8a-cbe6cc37f77d', 'e90a17e5-0682-4cb0-ad8e-43f3e1381339', 'e95bc96b-6b33-46bb-bb33-b6ea82dfa269', 'e95dac9b-451a-453f-8259-63c027b42f69', 'e99de3e8-531e-4578-9a70-0f0893813311', 'ea017fee-57c8-4e00-afe2-10e7c2dffd0d', 'ea94c861-c86b-4052-b4f0-00f049a2626e', 'eacff5a1-bf5a-4fe8-8ec9-6e175a40a9fd', 'eae5ee86-b6ee-4112-941b-311d6026acb7', 'eb470753-670d-4bcf-a99c-1b2bb8f73111', 'eb6fbfca-1932-4406-8a16-51f26a48727b', 'ec2816fc-d05d-4208-a27e-069425d07b88', 'ec671232-dea4-4f41-88cb-cbccb53bd5f6', 'ecb8b512-94ef-49cb-b4a6-f1fb3845c8e0', 'ed4d8f47-473d-4c99-beb9-c7dcf65099c2', 'eda4f304-7d2c-4ea8-b68b-95e1096b98dc', 'edc7dbb1-3fad-4e6a-b11d-49f7d82bd490', 'ee1a118e-9598-49e2-bd91-2cc0463ac93b', 'ee1d66b6-5951-480b-a25b-b52a71d3b36a', 'ee8933ed-9dcd-4cb9-a3e2-04e62b74bc8c', 'eee866f2-0a7f-44aa-9cdb-acf4a6fb7ea0', 'ef09dca5-f6b9-472b-9c16-55ec53135b69', 'ef2001f9-2317-4bc6-9ff8-b440118c981b', 'ef604a98-4756-4328-be81-25ca0ef86754', 'ef7ce76f-83f3-4d8d-aa16-e0a9e53a8faa', 'efb22290-952d-410a-92c7-d470c678ee5c', 'f00977e7-adba-4f58-87d7-ea705ad2108f', 'f020fc78-5c69-481a-adf9-0c6a82f928ca', 'f025468a-fdad-4dfc-9eb1-2b8e4016ec66', 'f0a8e914-c8cd-46a6-988c-757c18753805', 'f0bdb890-14e1-4d0d-9244-8cb788c2d187', 'f0d3a911-ddbf-4b3d-8f9d-b2f37da68878', 'f131019a-f0f4-41b4-a9ee-64c20a95d83e', 'f15dcf69-9485-4d54-b3ac-dbf18fd2fad3', 'f18ef5d2-0aef-438c-9c1e-25913c5147b2', 'f1ae83c7-a426-47b2-bb8c-3f98d9ba1f10', 'f2060de0-83e1-458e-a41c-4672ea2b31b3', 'f22e5b48-6dbf-433a-855e-ca11878d4b0a', 'f2e65a2c-f37b-4810-83c9-764a5a84f160', 'f35ace9b-0dc5-4236-b54b-ee9a2c404971', 'f365f3b1-37f0-4ed5-be06-e88d7750449b', 'f36edf5e-1fe7-4df4-98c2-caa26c36b9d4', 'f390e159-8582-475e-9702-d7608f976ad4', 'f397f921-bddc-4ed2-9a4c-d285a6eeae38', 'f3c5f37f-2fd2-4794-84ad-71a3c41b2b84', 'f417a8f3-2660-4d66-bba7-7b0dcc002f6c', 'f43fe84f-842c-4bea-8d6f-08bb1d3b2049', 'f47e3e08-0912-4abf-a8a7-9010923dd5f6', 'f4936385-05ea-43ca-b771-9b4e3a1b598b', 'f49da828-7807-4a4c-af13-50163d70cf42', 'f5242b81-56a7-4fa7-9c5a-0760a6635dd8', 'f52f7e17-0bef-4249-80d2-41dd81bcbf58', 'f54b7013-98f7-4ca5-b90c-9dbfbb4480df', 'f554697e-f66a-4e47-bdf1-295d92ff45f3', 'f5f0100d-7a69-4690-9641-fb48356d93cb', 'f6088918-2688-4bf4-a28c-a4f2ef883f52', 'f64ec6e4-6f9a-407c-9982-b849e0375bf1', 'f683568b-d8f2-4150-9565-57907247ff02', 'f760cd79-ab9e-4d98-933b-a6c369af606c', 'f79a8700-0a77-44bc-aa82-c88801adb344', 'f7eb9d4a-191c-420c-8c96-f02460721ce3', 'f80cbbe8-64b2-4253-a8a0-c6e91074ffcd', 'f86d8da2-f7af-44dc-a7b3-197336041d94', 'f876438c-d77e-462f-9930-06cac21a7637', 'f87d9a9c-cf20-41c7-823e-2458a53ad386', 'f8812937-55c8-4d0d-9988-29c3d69915c2', 'f881668f-7d0c-44d9-a44c-75ae8393194d', 'f8fa62c4-9d06-4dff-b7a7-a4e2ca30837e', 'f8fd95dd-31ea-445f-84d0-9db431bff817', 'f94a7467-ec75-4df4-ba6c-440ea2572a29', 'faadb032-71f8-4742-80e0-8e3c58a8c9b4', 'facad216-daaa-4df7-a322-f15c2f0eaf87', 'facbb7f5-94df-4dfb-af28-27f37bf74b03', 'faf21fc3-eaf5-4e03-a29c-0add9faf43f5', 'fc24119c-eeb5-41e7-9b4a-c6c826dd1445', 'fc7a28f9-7f79-4b87-804f-d7e1e153b9ef', 'fca25ab6-ad07-470a-9462-236ceb03eb13', 'fcfa884e-a5df-47b2-a708-6c34f03ceb8c', 'fd44f0cb-c3b4-4a3e-a192-8d6e03ebe0f5', 'fd8e036e-9d3d-40e6-893b-2ec5095a9854', 'fdf30908-9968-42a6-855c-999745f8cf92', 'fe33cf1c-f323-4f96-9686-b66bd8044111', 'fe7641af-7ee9-49a3-a636-37fe7b16df51', 'fe7ad30c-0218-42fe-8214-e4f23d48dbbb', 'fea750f3-519d-48a1-8c16-35f4f97725ab', 'ffb9a363-1d4a-4479-b148-3809161ce86c'], 'paths': ['/4DNESXKBPZKQ', '/eae5ee86-b6ee-4112-941b-311d6026acb7', '/experiment-set-replicates/4DNESXKBPZKQ', '/experiment-set-replicates/eae5ee86-b6ee-4112-941b-311d6026acb7', '/experiment-set-replicates/vanSteensel-lab%3AH1_LmnB1', '/vanSteensel-lab%3AH1_LmnB1'], 'linked_uuids': ['0003ed8f-64ed-4bd7-8331-3fa739507671', '00e22704-f080-4073-91e7-f04c0c2acd7c', '00ff4c9e-31ef-478a-82e6-c43503bc4cff', '01bc8c8c-083b-4119-bfde-e3272dfe9e5b', '01ca19b6-4652-4eca-9f38-f0e93acb58fe', '01fab0f2-e76a-4a14-a77e-e5239dd391dc', '027019c5-88e7-4357-8363-dde26483b8cb', '029e34cd-a40f-41b3-8066-d2ff514e7d3a', '02e584e1-8e53-4415-9503-cb2ed8f2c07e', '02e8d673-2ed9-4082-a027-2da57f056f0b', '02ec55b7-2961-48bc-bd54-9cbfc1e2320e', '03418f85-9977-4c00-ba9f-36b1c3ab2e4d', '03bb9328-c6d3-477e-93ac-a72232e9b93e', '03f64d2f-9be4-47ea-aa53-25cece95b8a8', '0429b923-dc29-4514-a1a2-2d6d1d76d973', '04afedd0-1dcf-429c-ad77-341855b6746d', '04b2c874-95df-48e9-a072-cf383cd4e0a5', '0613a2e1-b543-4da9-90aa-9dad054e496d', '063961e2-891e-4b2f-be49-97a400f9a1e9', '06618a6a-2001-4033-843f-fb56797b71f7', '0683b6d6-5fcb-4135-ae23-ea1cb9dc2d92', '06982263-0ec8-4c25-b8f6-04b08abc0bbf', '06ea837b-093e-4cc5-bc1d-87ac9f19f6c2', '073667f1-ceef-4b3d-a4db-c48ef11a7ffb', '0775a179-151f-4531-a89c-01ff2429acdb', '07b05436-e2da-46f2-90ae-f460dd7abddf', '07c8be77-ab7d-4e6c-8fc4-85a346cf0ebb', '07f60950-c5e8-411e-b6b4-dfdf9526b53f', '0842df74-c469-418b-956e-ba47183e14bb', '087a6256-a52a-40de-99aa-f5ab2416cb2f', '09ee64fe-0c97-440b-bf81-7106ac5620fc', '0a44ff64-237d-4e9c-bd18-68657194d643', '0b5cbce7-71dc-4e27-8246-f2b636102b85', '0b7cb7d2-dd17-4d2c-a80a-d2b6228abce8', '0bcf3871-67bf-482d-9a89-ab99a47cdb0e', '0d9b34fc-1399-42d4-ab91-999b166b0701', '0d9c0bdc-05f3-4f73-8340-9774ca55408e', '0ddedcbe-5167-4b3b-a223-42d6bce3127b', '0e329024-c611-4537-b26d-d26519f94fe7', '0e4bf888-08e9-40f2-a7da-a33efd00aeee', '0ea74454-5d62-4eec-9994-9864d7e15b51', '0eb67115-70ff-4a35-ba22-1803839aefbe', '0eb98dae-4970-4716-8145-5c3c5c4b1073', '0ee90d06-3a98-45e6-ba37-0ba6214a878b', '0f020b60-545f-4677-b0fe-428fa405cf65', '0f3b47e4-2365-4b5b-94fe-35b0fcc0b3c4', '0f606c1c-bfcb-4b5d-a9e9-ee774080c5d3', '0f984601-84aa-4754-8ab9-ba11412487e9', '0ff4313b-53df-4e74-b129-a3f5692b0088', '1025af37-366d-49e5-b492-120023fae219', '10dc8592-5e4b-4e5c-ab7e-2444217a567d', '10e47432-8191-4908-b402-6e9e4837ace7', '10f3f998-b87f-4998-8d5e-144e0eb363c6', '111189bc-8535-4448-903e-854af460a233', '11392c03-15d4-494f-a748-62cdc780dc83', '114f658f-8245-432c-8b8d-56e06bdb7507', '11d5e29c-fb38-4640-a55c-9bf90bd17542', '11ed4e56-3f47-4cd3-8b14-aedb403189a9', '11fa57ea-95a2-4a59-ad69-5f4f70e6093c', '120b4ff9-350e-44f0-bff5-254d71751b1e', '123c5108-0073-4e57-87ed-31db8ec22fd7', '12c0e1a8-c3fb-44bb-8c92-f6cc9d288752', '12c9e3f3-d382-4fb2-b024-e7c3c1dcd02d', '130d5ee3-067b-46fa-af40-4b715e5c42ac', '14949ebc-4257-4fe3-a375-fe687034f7e7', '149b37bd-eff8-448a-832e-ae592c52fd2e', '14edd7fe-90fe-4072-94ef-8328c3f3e735', '153b1e20-7bca-4890-8d21-d2b8a29f60dd', '1584efb7-9d05-4ca8-9822-9e17cba720b2', '15918da6-2476-4a01-a736-621e01e636b4', '18324f91-fdc4-4d9c-85d1-6656e67d22b5', '18336371-b9ae-4255-942e-32415f1253b6', '185e765f-aeec-4277-81e2-70dcf8b56893', '18852a4b-cece-4549-894e-2160465e01c6', '18d5c79e-a424-4bad-ada5-a471f7b02ea6', '18f9c9f7-1099-4ed4-8071-aad033c5eab8', '1a95f073-c8ac-4b56-b11d-2c6a02cc7f30', '1b01a20f-9edb-4427-ab74-ee4ba82bf08f', '1b3e2cf4-a5a8-4e53-8449-71cc8dc17ddc', '1bf9403c-0513-4616-acda-84f2b2fa20cc', '1d135c5c-4b43-4f64-a596-42393bac455c', '1d72e443-1975-41d4-8ef9-6dbbbb0ff30b', '1dc2472e-32a1-49ea-bf6e-6a722061e4c5', '1ddf2fb7-dd07-4364-9154-636ae293c68a', '1df0c71e-b8a3-49cf-9bcb-c342c0736270', '1e06fd23-07e1-4200-94c2-952bbcb57a91', '1e1bb0a2-017c-431f-8e69-c3bc5deb4c77', '1e24dd56-c884-4d78-9145-acaf4b1c0c79', '1e73764c-81f6-460f-96b5-a2771601fad5', '1f400f8b-9561-4b46-9e09-0c4c83c09691', '1f4012c8-8b5e-4c5b-9604-c35d8a5e3c21', '1f47cb15-c562-4b06-8cd8-0ef5c8ece340', '1f860ec8-24e0-484e-bb19-67e22360b632', '1f930c17-3b84-4c7b-ac44-b484ffdc19d2', '1fdb014a-e880-48a2-a341-cdf0acd5de0d', '1fee3029-42fb-433e-bc35-c62a0b8d73b0', '20a136df-054e-4159-b9be-3a7a613e6b4c', '20f2c990-4a30-4599-9058-44e66fee686d', '210b3480-c15a-49c8-ae42-c8f066eb1db2', '214cfbfc-1139-4905-b54f-1b48c15d5f95', '216911ee-35ce-43a3-9279-c42763d70172', '21b03f97-05ad-40f5-be8a-b6a054941a88', '21b9c655-043b-408f-8495-c3dcdd399c8d', '21ee7949-6449-4fc0-a0b4-28f4d958962e', '22351015-90bb-4121-bc1f-5b5d99c65769', '2299254d-2a5f-48d1-aac5-1138fb10a967', '229bed6b-5122-4be8-b359-d3587fcb04b2', '22ec4614-ecee-4d41-8f3a-d2fdb4c95483', '235a431b-4e61-4755-95e8-47305735c374', '237569f1-f169-45d7-97af-6026f128d47b', '23ad5350-60e3-4665-8e0c-d7cc82814b2f', '24043c6b-25a5-4be3-a2aa-b467342e6e83', '2437af1f-f6ec-487d-be44-c4a0700ac11c', '24482a30-3f46-4e0d-911e-c76beaa1e19f', '2673dc69-29cf-474b-8fd9-f3e7129b21e0', '271132b7-6986-4fe9-8716-49144c52e501', '27459cfc-be97-4218-b832-54e39609f81a', '27719efc-6b2f-4c53-b11e-48879ee807bc', '281bc792-e782-4308-9d18-2d96bfcf6389', '284d5402-0107-44e4-bc2b-1aeb578eff84', '28b12db0-1f11-4a6d-ac89-1a4a44218c16', '2900fd25-c89c-4202-913d-37969357f2be', '29b9f6a5-e31c-4c05-b1bb-31437866a679', '29ce9b5c-7c2f-4368-9c0a-ce66a7adfea3', '29da4ae5-07d0-4521-808b-4836d6aee9ab', '2a4979c5-3f44-4d5d-8ffe-5f17ba15b793', '2a4fb7f9-ff43-4666-b932-915584b4b441', '2ab85c11-14da-49d2-b428-385b6ee341bf', '2ad3484f-d2cc-4fd5-95a4-c1323fbb511f', '2ae11d64-4e02-40f8-b448-a396dcabb856', '2b217c78-edeb-4e2a-98b0-8b9154f3e70f', '2bb873e3-b5a8-4c96-b143-3716a898b322', '2bc83039-aa06-4c96-9876-14ec2ad4e0c1', '2bcb9dcc-7dbe-4c1e-b063-7c691db4ce22', '2bd4fede-151d-4273-b2bf-6fe2ef51429e', '2c038815-bd38-47d4-bc00-333bd0696316', '2c0c7103-5a56-410f-b6c4-d91dd0e16bcf', '2c378e44-aa04-4f02-ba85-b679fc9b5f4a', '2c851365-b54c-4f0a-8b24-c8881e856f89', '2d512da0-bcba-44d8-8e44-7f60ee367e2d', '2eab3194-c7ba-40ff-8c5e-206c365045d2', '2eac4499-d26d-40be-8f67-4e98b7f50752', '2ec4b195-617d-40d9-9db9-ca4447834b9e', '2f3a81c3-553c-44b3-9431-4b7e4b5a0e7b', '3062a2f4-dc06-4130-aa1d-8d0d87140764', '30b22cf1-e3d0-44db-8e27-cd79aabb3d6d', '31be8f2b-0f03-45c7-8e50-2ec5cda94b0e', '327abce6-8158-466e-ac84-720297e89776', '32a077d8-584d-4223-bf11-9f5df1d2797b', '32a0c6b1-9d2d-49c8-a806-c74df5686712', '344fd27f-2406-4171-9c0f-f63c32df5abf', '34772abe-2d63-4b46-a0e9-6e4a2bc60c52', '3575ead4-0fa4-4fd1-af6a-46d5105628a8', '366a888c-1415-41db-9078-f5f0f32ec63f', '36727775-f63d-4bc7-b05a-1115af9af8f9', '36cdb32b-a34b-43c2-bb1b-97a56ccae90f', '37486dcb-eeef-4132-8a75-760ae0850fef', '38d09d2b-5f25-4f05-acdb-555f46bb3bfa', '391d73ad-c768-48b1-aeae-622bf982fb30', '3962e891-9f91-48b9-9658-5b80e387af3e', '39716245-2513-479f-b881-82e16f4d452f', '398dad50-f67b-4595-9c45-41ca3c979ff3', '3a1dd6cd-b763-4304-bb24-2ac877b9cda5', '3a2f7cea-9e9a-4c8d-b501-f2f27776b425', '3a3aca22-7824-4179-ba02-9fdc68abcc2f', '3acc368f-7ce8-44f2-bd28-c737c12aa639', '3addbef2-1a5e-40d8-92aa-d861c51a31c3', '3b558807-a86f-41b1-827f-9e7e5e831c5d', '3b7983e4-0bbc-4435-8a94-74d7c4ec753e', '3bfa1c2a-602c-420b-8d28-5b1c1c954d25', '3bfcbd27-4b29-4989-9d76-7ea78d004d3f', '3c35fbcd-2c00-4c12-a75a-d1f340177739', '3ca2f1f4-98cf-4316-b156-49aaeafc10d4', '3ce88a02-14ad-489f-a75d-48ea341e8c51', '3d365a04-9285-43d7-89ab-c5ea29391d6a', '3d43a37c-af1b-42ee-9d2c-d3843f63214c', '3d4d696e-0939-4461-97d4-867930011372', '3dcc1971-9c0f-42d6-933f-e4770147b827', '3e2e4a0a-ea00-46b6-bb8d-3867b9e3498f', '3e5dc140-4de3-4cf1-a2fc-11f9bce4f22d', '3e7d8342-c051-45d5-b04d-8b1bd62bdbf3', '3e8087c9-1311-484d-9cea-8dac908b0aee', '3ea0252a-b9f0-46aa-895b-df9615996e45', '3f2e38ec-79bd-48f4-b243-627836c78910', '3fc0291e-14ff-4f73-a3c6-25230f668b55', '401e101d-55c5-45d1-942b-d036f7d3e70f', '40b69cc2-d14a-43d1-a41e-fdd38c6dd3f2', '40b70f90-9f0e-464f-bfba-7616603a64ad', '40f4120e-7c12-4a25-9b2f-efc28b09e63b', '40ff719e-0f4f-4e4e-a283-57a87cc0902d', '411ce2e5-68bd-487d-993c-b01fd8ab7ff9', '41784f69-697d-42c2-82b1-b52bc5ea3519', '417c774f-0ca4-464e-8f29-44cbd7f299fe', '41d1f0be-32e1-4e23-a9fc-6210ee7665ff', '4330a4a1-143e-48dd-8acf-98f3c3a1b3a5', '437632e5-f7af-4a5a-ae80-eebf51dcce20', '4384a60d-4440-4e8f-9c0d-428a54dc43a1', '439e1ffb-35f5-445e-a968-e5414d0a6e4c', '43b0a759-761e-466d-bc66-7845ef275389', '444c27dc-484c-4784-98f9-5d41e04ee05e', '44da5b18-190d-4f94-9f50-f23208f0fc92', '44ea12d1-7346-4244-a6b8-06549f1fa181', '44eaa227-827c-4940-9ee0-9becfdd1b5f3', '4577d3f7-9e2d-4a03-97de-61335539534b', '45854623-e14e-4c0a-ba52-c478e32cd86c', '45f8d7f1-bded-4298-9d46-9e35edad4460', '4613ff48-2e4a-401c-b0da-137b666109c9', '4666b46c-4b7a-4454-b968-9ab20c08ec46', '46bcd592-6558-4bce-bab9-799b21b4034b', '46e3377c-6caf-4ade-af4e-ca6f81ac2f9e', '4704ffa1-b916-4c0d-86e8-677a182ce2b6', '47d6f544-6405-401e-9bad-e41ad0f43ec3', '483695a1-b525-401a-beb3-2decbe1884a7', '48b3a641-f51c-4438-b2b6-c94ab2a1864f', '48b88e33-bc15-477a-8943-0b5fe65af623', '48cdb7b4-1cc3-483b-b95a-dc276ac21d07', '48d9dc72-4f60-4dba-b20f-6551f1624aaa', '495f8b40-1177-4ed9-bc53-3737acea8ae2', '49937616-a384-484c-bd5f-fb9dd992984b', '4a0abcd5-a99b-4b97-b026-d8758fe75c42', '4a62480f-aa50-45df-8881-da2c95be9774', '4b10a69b-146e-4e74-ac98-b2342a698522', '4b11d240-5acf-4577-a18b-9526b17665b6', '4b43218f-af8b-4395-a130-027be3cd82e0', '4bce48c8-23ce-45ef-8dc0-80225c8e973e', '4c0dd361-e0e1-46dd-bfa0-e82344bd3959', '4c1bbb1e-75e2-4331-880c-0a47692c6b45', '4c7ec1c1-1693-49dd-b1c0-48a4b75a4e15', '4caa37f3-b01f-4034-bcd4-3fa7079602fe', '4ce53fb2-987e-4164-9c33-029a55d08cbf', '4d028573-d2cc-49a0-a5ca-80906a269152', '4e19b7ac-86fc-45dd-9a64-ccc1ba61ec41', '4e31b135-9a59-46a6-a9df-8b7bbf2b194e', '4e331085-75b8-495b-ab08-23110ace1d30', '4ea34fb7-a41e-4e47-8d87-9bc2becf1e99', '4ef7308a-cde2-44c3-8a13-6c2b10d37404', '4efa4b0a-5587-4876-9d33-813eb1e7f491', '4f70783d-063a-4651-846c-6e83cbaa7bce', '4fe066da-7f61-4754-878d-5628f6585e03', '507163ca-1d4f-4a3a-a9a3-4bfae5017eb0', '50eecc37-6569-4910-bf62-1c1ecf1b5a57', '50f8300d-400f-4ce1-8163-42f417cbbada', '51af7800-1541-41c1-9c5c-a793311d4781', '51be3a0c-90bd-4e40-844c-d95900b61756', '5228058b-398b-4bbf-92ec-304279ab1d89', '528d6fc3-7760-4c36-b550-b1cb6910461e', '52cd7f15-a526-4fb0-9442-562c9e2c3696', '52ce302d-745d-4bfc-99e8-f8092a9f5cee', '530006bc-8535-4448-903e-854af460b254', '5343effb-62ca-4f49-98ba-28747670cb54', '538b06ad-ebfd-4ef5-8e19-56e4b982e9fc', '5396f6de-15f3-49a7-a031-e7e81c122195', '53d9ce5a-75db-4aa5-b96f-09da09bd3ba1', '5402ee85-d568-4d40-b86d-fec615f5b419', '548acb46-ca54-4b6c-9597-02a8e4f5e4d9', '54a69337-7ba4-4c78-8a80-c9d0675b4850', '54f877c3-9e06-4d4a-93ed-5e3d5d0ddcfc', '54fa576f-98cc-4ec9-a6c6-d988328a81bb', '55cb0f75-30c0-4493-9b5f-e561da2a5642', '55f42202-e4b8-4ef6-b91b-c4c1820f96cc', '56a58e85-cb4f-41d3-be16-cf95111fa5d1', '56b88122-94ab-4a10-b28e-8ef2e71a9a81', '56bcd0d9-0f7a-42fc-b949-f6eba2a561a2', '5704b3a9-f889-48cc-b19d-64ab5b0564b1', '57501104-cbc6-47af-8149-92a977f9240f', '576cd077-0010-464a-ba15-70546e725531', '57727ebd-425c-4d57-a2fe-f57bdac0195f', '580f3d5d-cfbe-4536-8cb3-b226e76d0c45', '599c5d2f-eddd-4f76-97e3-554d6e539e4f', '59eaf14a-1e59-4936-a68d-0c29f8d59843', '5ac960b2-9758-406c-94f3-c914f812c46d', '5c20d4db-add8-4db9-9c29-5ad642f183cb', '5c6d1a0a-ff87-4e6c-b234-f648b851273e', '5c8c9652-7eb4-4a7e-bccd-a0f96f965f81', '5ea3a819-5f27-4a01-a916-e5d258759672', '5efd514a-f072-4537-bbfc-0707ad127577', '5f1719ce-8e36-4d79-81e6-cb370739d0d5', '5f33da65-0211-418c-bb5f-5510b1b2aab8', '5f53f149-f653-42b0-a903-d12647fd4af6', '5fa22568-3b1b-4fcb-a200-7a3ad2e28d63', '5fc63fe6-b29e-4681-96ff-052c872af9f7', '60188dcf-2c60-4b56-9856-114c60937e7f', '60446cf3-3b1a-49f6-bad2-c492e20eae7c', '61ad62d0-cf15-4859-aefe-c52bfc4767c6', '6301c7cc-5ae7-43d1-85db-610f7973e9bb', '6328e19f-d31c-4572-bc36-f757035844b6', '6343d57a-213e-4ac7-89ec-8b5e74c21bd2', '636fb808-d2c1-4dec-98cd-f36e4b1061f6', '63a43224-cd34-4ff7-bf3e-0cc66ebf381a', '64213148-f0b7-47d1-9d57-fbdaad09c97f', '643007f9-e7bc-4a2d-ba49-1fd0ba484b3d', '65015fc1-ce32-4c31-821c-d105bb0a82be', '6520dff6-cc1c-4a2a-a67f-2fdfa889e582', '65504487-18db-4229-a4c3-8b10cec51f86', '65bcc2f7-ab2d-4439-a28e-0a753a609835', '66029258-f7b9-47e0-8c72-01eead159a98', '6630f5a6-567c-4799-8c9c-c5fdad74ba61', '66526e51-0ce0-4417-acc7-1564dd20324b', '66b5bd83-6b3b-4a19-b372-a4f88ff3b9ef', '66d83ee4-2757-4a6b-a3b9-cbde81eccb21', '66ef3799-ec50-4dbf-9753-36bea5b1e76d', '67f86a25-7ba2-4c5d-9b11-ca7363f0e0e4', '68172441-97c4-40cc-b73f-d0f5dbc5cc05', '6851452d-1227-4e68-bf9e-b5a00fffd22d', '68b0ff45-92c3-4aad-8887-3606d9c9d21d', '68b9c110-9c22-4003-b3d2-93c467dfd077', '68defae2-f421-4e89-942e-f421e67f336c', '69494da1-fa34-4aca-9ff4-639073e3e10b', '69658dec-ce10-4f34-a117-13781bd571c6', '697f8022-7bf3-484c-87c1-3d77ffa67439', '6b08421f-f635-4a01-aac3-f0d2e6f900d8', '6bf42006-2220-4c14-bf70-ac6c3f94b00b', '6c3586da-4567-4d91-b479-7fe33330256b', '6c664b74-2794-4f8a-829a-754c18ef427e', '6ce025db-dc9b-4c66-ac74-0b23efd6da71', '6ce17b03-55d1-41d7-96eb-9203b15b2c77', '6d086a47-e980-4f2b-b5d3-407a53eb6638', '6d269c56-87d7-41cf-8a4d-b178db7ae157', '6d8aac76-93bf-4423-ab15-aae9c66d7ecc', '6d9acc40-be0a-4ed0-a893-df5c4db25b52', '6da1a1b2-58aa-4bb4-a2af-131891d79384', '6e06c80b-23ef-4957-85f7-b94ba0420984', '6e378c15-bf09-4304-bd45-e8a444c2a5fa', '6f31119f-e3cb-4052-a8c7-c74f0f474676', '6f6c9cf5-4e4c-40e4-b2bd-f5e2360c1aa8', '6f9714dc-2138-41fe-915a-9d40902efa67', '6fc80dc4-0e1e-47df-a9cf-a19ba740fa5a', '6fd5b473-3f46-4219-a5ec-0db93342a728', '7013a367-924f-4002-8682-bfc9a1ab0904', '708af918-6bb4-4a7d-b0dc-2a3706953364', '7153c1b9-4d08-46dd-866e-ef8333dba917', '71cf239c-25c0-4fcc-b265-f88dcbd53809', '720c6ec6-f5e1-4f6d-8a60-9a45c7cdc013', '727b9399-92f9-41f2-a61e-8532fef538dd', '7299dd07-220c-44f1-9e56-bff6b453460b', '72af586f-4d8f-44b1-b0b6-feb2eaf6993e', '7321fb29-e465-4baf-ae3a-cf88defcd9fc', '7442030a-be29-4d3a-951b-014f6958fe8f', '7463ec99-c203-4c4c-a221-b899fbbfe14d', '748db193-9256-4976-ac6f-c4e9e823ffbc', '74a4af26-0e2f-4818-969e-ae772af46cba', '74e09d91-0329-45d1-899d-7b9c450926b4', '75754d4a-0b50-4281-8975-c4f3936cf1c0', '757f213c-d802-471d-b857-74258bbe037d', '75e19a0d-f2b4-42fb-8716-70a79e14fd69', '7601d9a0-34de-477c-b7e5-6ac26f1a855f', '76450ea8-d15c-4572-ace8-a7ea947de5a8', '76ec7d64-7f75-43fb-b112-3d05925377e9', '7745b647-ff15-4ff3-9ced-b897d4e2983c', '7876e906-1898-4097-8490-d657cfc76330', '78775159-a731-44d0-bb9e-281787541407', '78a67c3e-6274-4f13-b998-7dd0d8ab9f5f', '79112ab5-1aa5-4bca-a89d-3949fe4fb661', '79384268-41cd-4840-8ff7-1de6cd47a19a', '79797967-00ba-4c13-ba2d-eaa32a44df96', '798b8174-fcfc-4acd-8cdc-8a40e2b84cab', '7b611fb0-8339-402c-9281-b7fa13f0c4be', '7b88cdd6-233b-4400-bff3-76ea4668881a', '7b8c2345-1b50-4fe4-bac9-5e9f61fdfc2f', '7be9606e-7a12-4a0b-b204-8478b51873f9', '7c5d5239-7fe9-4060-9e66-6572849ef296', '7c91002c-bd0e-46f9-ab15-a11226604604', '7ca9afd8-106b-4a8e-b403-8e110f2447fe', '7d02daa1-5fda-442d-890d-7473ed261331', '7da005eb-81e1-4526-92e0-7fe8ff2917ae', '7de8cb5c-579c-4cdf-8690-f4c9aac2d4ad', '7e2b2747-2210-4d31-bcad-104d545d15a5', '7e45306a-ed29-4065-a25b-acc3aaf112d5', '7fd6fb2c-f2a1-4e5b-b45f-f9c5a55b3d08', '801429b7-a742-4c62-ad8f-5ea755656669', '804d8f54-07b7-4f82-8c2b-71539b508aca', '80858597-c2a9-4cb7-a4ba-c893decb8af1', '80bca017-6891-4cf2-a0fe-26c90aa3b4d8', '81324f54-3127-4d68-8c7a-997c1727e0a9', '813a81cd-59e1-4068-a1da-6e02a1e6b45c', '814b6aa2-6868-4590-8011-90de1100e54b', '81f71af6-8307-4c09-8a49-a74c23c544a1', '828cd4fe-ebb0-4b36-a94a-d2e3a36cc989', '8300c32f-9a81-4af1-855d-8ba3d9e09edf', '8310a3b2-e421-4656-84dd-f7b0abfea638', '832736bc-2995-4b64-86cb-99f68df0d464', '8395504d-0d4c-48a0-9cee-584fc258348d', '839a4264-6499-4f11-b349-bdc93236cb28', '847739c0-3af0-4be8-9fa0-16c3071b4e29', '84943bfc-4167-465c-aa0f-7443c88d92e5', '84e34ca6-8a91-4eca-8aa4-fd7e5f546395', '85490436-b099-4760-9d4e-621e2dea4572', '86643679-3d06-4684-816e-06d6a473e1bb', '868895c8-2497-4fc6-9fae-854345da5d8d', '86d44292-e30e-40d8-9ef3-899105f3af1f', '877bcafd-c796-4aa2-b8a1-bbfcdaa144c7', '878db655-beff-4652-a201-e71864b7cd36', '884ab4e8-d169-4a9d-8414-4f85cec5c0ac', '885673f9-e004-4748-938a-74e7bf2f7d11', '8896846b-9c19-493a-ad78-b29162eea666', '890104a1-6dde-4d09-b847-dcd71c908852', '890ae6ed-cc5e-4472-9370-ff2bf939950b', '891b53a3-e6f0-4d28-b370-227e45051f46', '8b29449e-55b4-4767-9399-18aa9612f310', '8bd453dd-902c-4e00-98d9-69dcfb5d96ea', '8bdc8002-c4cf-43ca-b0f0-0a35eb688101', '8bdfd1c7-09e3-4d14-969e-9cedf03003f1', '8c8882b2-eeac-4594-b1c5-9bfc39532abd', '8cb4f8f2-ed9f-4875-9b1f-6dbacb26d0d2', '8d178b40-ffb7-4766-94c4-0dce43501230', '8d3f8614-18da-4717-ab5a-90e369d508b1', '8d4773c8-5573-4f20-ac34-2f7aa1eb449e', '8dc994e3-2aa8-4caa-be8a-20d413e6a518', '8e0e4a91-ed32-4ef6-9a0b-23fc04555774', '8e4edcfe-c3eb-45d6-bada-c9cafe4faefb', '8e70c8a8-6c07-4f31-a7c6-2c2dedf3f153', '8eff8235-f121-4417-8005-8cd35ea56e88', '8f355919-6a46-4928-a223-b8f5358210c3', '8fb10e27-7b0a-412a-962f-ab703740e197', '8fe88d17-a0fd-4366-8f4a-ec931aea4a61', '90225d20-3737-4eee-87fb-d2ea7a42d135', '9039221d-be97-4d67-8e74-c591c8bdb088', '904455a5-d8cd-42ee-8c61-a91acd503a8e', '9062f200-f8dc-4096-97b5-b6ff9ed15e33', '90ab23f5-8db0-453d-b958-e56daf8a173b', '90bc7ff3-bf6c-430f-9a07-1026e28c2d60', '90beeed8-e401-4758-a543-f8dc2433a203', '90c84c90-553a-45be-bb3d-2d11801d05df', '90e7c0d5-35f2-45d9-9f42-1db6082c5319', '912d26ef-d305-459a-908d-808a88aee5ec', '91b694c3-f4d7-4ddd-8278-16f94e15c1c5', '91c4ce9e-1620-4cf6-9127-23107829bc60', '91dda5b8-7818-4f21-b81f-2d2655d4e03c', '9274cbcf-8ebb-4626-8756-5ffbbce5b4a5', '92b65796-33cb-4b1a-98a8-b25f27e92cfb', '92d85c9d-885f-4c9f-bdc8-273eab66babd', '92f90aed-7df1-4bd9-9e74-a472cb50d663', '930fbb26-06a0-4051-b3ac-4effa9ee6924', '931d1b4c-3015-4b86-af1c-d2b5a6fd09a9', '93325ccf-72c8-44c4-8292-c79b2aadcf40', '934b8a92-3adc-4991-983c-dcec0191f5c8', '93946918-5126-4ddb-adac-eb171cef0bde', '939f7cd8-e01f-4fd6-8d35-15e3328661fa', '93e55e41-90ad-4c99-844d-d25e60303003', '9485316e-7137-4ca6-ad1f-2894a59eaee6', '94ce0c14-9ad8-498f-848f-5d6dc1d80153', '950172af-87f1-44d2-aac4-22112322efac', '9510e997-27a2-4260-861c-efca9efa542a', '95bee459-3ca1-4421-afb6-c7f09475d827', '95deb527-6d19-45fa-ace9-4e5d4ed1e023', '95e5a239-3010-4d3b-ba53-c1439ea3f89b', '9604c8eb-96f1-4008-b5cc-7055c1016bdc', '96b02cf1-f8b2-448a-b858-e99492c737d2', '972c0f31-244f-40ac-9961-308a22babe35', '97acc49e-fd3b-46e5-adf2-cf9112b52cb0', '97b5785c-b2d7-4b59-b8bc-448cc9a58f89', '97b74f37-00f9-4636-b384-d466a32432af', '97f3f525-30ab-4ec9-b31b-6ef008112ab7', '986b362f-4eb6-4a9c-8173-3ab267227777', '986b362f-4eb6-4a9c-8173-3ab267307e3a', '98c4cba2-8cdc-4b37-97ff-d0974f6b4729', '9a6675f8-3dcc-44c9-b4d9-d954acdf69b9', '9a667a78-abfe-4983-bcb1-3aba11347ca7', '9a7fe685-b500-451a-a53b-f945880ece63', '9ae79d4c-0a7b-4bd3-9439-34bbd06a76dc', '9b43d6dd-e505-4a48-97df-e354458dc12c', '9bb81b4e-1f0c-45d8-bc06-15f3b8fd6de8', '9bd01d51-3aee-4535-9071-752499ca6a70', '9cc0442d-9a51-4e7a-985a-aa345d0e0322', '9cd05f6f-d339-473d-a566-8eec65ce51e6', '9d374f95-9859-4ad0-90c2-9758e1259a04', '9e995add-688a-42ec-b86a-1571324b923f', '9eb7a058-e831-47c4-bd97-b8938f979728', '9ebb2014-6979-4e60-9783-96b673453fe5', '9f4430d3-c70f-4f68-8b0a-57ab22476dc7', '9f53bc06-50e6-42ab-8ad5-a29d6b21b71f', '9fb452a2-b24d-4f5e-914c-80b73e7653ff', '9fbad3fd-3c10-4c27-bbe6-0ec536f9e963', 'a01ec5fc-5016-43d1-a3fa-f435146aab02', 'a0a2bee4-4204-47fc-b945-c696b7e12b15', 'a0d4e212-d1f9-455e-8410-b5b533112097', 'a0f5399e-57c2-41a5-bd76-8872c46b8fa4', 'a13b6328-4aad-42d4-81dd-6d07c39600b4', 'a1703df0-8fd4-491e-aee4-26403decf498', 'a17bbfc6-6955-4e7f-a150-937dd2d65fc8', 'a20ead51-2d13-4fb9-b781-c2b172beb4b0', 'a247e119-11ab-467d-b648-ab4e92926dda', 'a252b998-1ff9-47b5-bf1d-b10bf8005ce5', 'a255a3e8-ef0c-4d8d-a54c-5efb6256b45a', 'a2911f72-9aba-45c4-9024-14018792a0e7', 'a2fe8bc2-3e14-4cfd-96c2-1c2bb395a3b1', 'a30a8601-f28c-4f47-9b66-eb1cc722183f', 'a313969d-81f7-450c-bb17-09195f723422', 'a31f8fe3-01be-4fae-9ca3-9fc8de4b7193', 'a36726ea-da51-4736-9f10-f13fd7560ca7', 'a3f1f14e-dd1d-439d-8bc6-32a1228a48cf', 'a401eee2-3dcd-4512-a51d-b29069ef3ee3', 'a4104023-d705-4a11-ad13-705c950ed59e', 'a4827f07-f508-401a-b7ef-f675d0ebb343', 'a496efb0-ade8-4243-9258-9458e248c5e2', 'a4af5721-6432-4cce-9a50-c6f499a75202', 'a4b83dbc-5a6b-4268-b45c-795f77c56eb9', 'a4c5dabf-a3fe-4894-a49c-efefcc4a541b', 'a582c75c-ec7e-40ff-95c7-a0d510caf95d', 'a5cde8c5-5172-457f-9fe5-9e2073fb800c', 'a62e5d35-ce10-44e0-9075-e3697dd60b3b', 'a6308778-f9ed-407a-a422-e71da88a8dae', 'a65740ee-db45-46ff-ad72-122bbb0d108a', 'a662782d-174d-41d3-b0e4-f9e0f3fea7ec', 'a6839631-d38c-4b21-ba44-b99e9d8c0c07', 'a6fc9d41-cf80-4eef-b090-2352ad99409c', 'a7a90458-4243-4cf8-bef0-82c3f7d6ab41', 'a81e847c-7016-41c2-abde-5d016c4afccf', 'a82023b3-080f-4cd7-aa92-fdc30be405c8', 'a8310275-9f12-4103-9029-964e67a83c89', 'a85ceaa5-6348-4911-b88b-d40f64f48ecc', 'a915a342-5cf6-4d61-89cc-4cf5ef6006cb', 'a9802b54-3836-4cc1-a343-174b799bfe11', 'aa370560-9977-4298-9147-db3162af5b8f', 'ab161bb1-71ab-405c-b2d4-29c412548a7a', 'ab371aab-3fb7-425e-8328-6199902a6b78', 'ab933a1f-7f5c-4c4c-a7ec-4ef37b2f0e6b', 'abfa318f-15e6-462d-9b54-de24e5f15f74', 'ac4ed406-3708-4879-a057-830ef4e97856', 'ac654424-dfae-4dbc-bfd6-264c2969ab71', 'acc3ae42-f070-4759-9885-aab25797e4cf', 'acf3e2bd-59b7-43cb-b9fc-0aaeaad35263', 'ad4b9a48-51bc-4d45-b4e5-bb870089f98a', 'ae12f563-7a2c-4767-b4c1-99e3d2458199', 'afbbbce3-992d-46ff-bd36-ec54e97cf0ce', 'b06035c8-3014-4965-9fbf-23b7933c1938', 'b09e0dc2-00f0-4546-8893-0007877cf741', 'b0b9c607-f8b4-4f02-93f4-9895b461334b', 'b156a751-31bf-410a-86e2-fe4f897061e3', 'b159360b-8681-4baa-91c8-d495714424dd', 'b18568b9-8d84-45d3-9fcd-e5f5c59b8c76', 'b1929671-8076-4e24-8db4-144e6fd81397', 'b1a8be28-c1f8-4505-bbd5-1c06c982c5fb', 'b1f7529c-59a9-47d9-aa12-9319275d27c0', 'b2b97fd0-7284-4bfd-b73c-714f9894f739', 'b2c2deeb-e883-4ac0-b9e2-906e598884d6', 'b2f2fc89-07a8-4fdf-a594-5d0e9a92f3eb', 'b352a9e0-30b2-4837-a2e8-5a404ddb7d20', 'b371c45a-d24f-48bf-8220-9e76103467d2', 'b3859df0-da57-40cd-9b76-6ef5efc9b537', 'b3fc4fd0-7843-441d-8d04-02e6f18d54bd', 'b4778c57-5d69-424b-b287-9b505aaa6f6a', 'b48a1a5e-38cc-4e7b-8652-8fb0fe8bf7ee', 'b5267b15-f5a9-40b9-9816-d8138d9bf17f', 'b5457a2b-5e33-41ec-89f1-c37fd1686532', 'b5e13c68-4f56-4353-a769-4c09e530c483', 'b5f0e395-c150-46e7-b114-7f9be2f8a05f', 'b68d4494-43bf-4845-a81d-06586466dbeb', 'b68e223b-edf0-44d6-93e4-a4c6225266b8', 'b69dfc34-20ec-4a08-9975-2afaa59988b1', 'b6c8e21a-fb2c-4464-a89d-41408cd76ac6', 'b6d45df2-122f-49ba-ac1f-e93d71ca2b71', 'b6f630b5-95df-4292-86a6-8cdc250ee200', 'b83d9e9c-64d7-4c24-881b-64117aad2a20', 'b87dbb08-21b8-4765-b21d-6425ca84a132', 'b8f7974b-73e6-4b5a-ba68-2f7deac95bec', 'b9036992-9d8c-46cd-95d2-80714d22dc8f', 'b9280ca8-5a9a-458d-b4fd-157b7a099f99', 'b94e0001-19a4-4bd5-a383-9abc486c2734', 'b956d5d5-6475-4ade-86ef-3578f98b0142', 'b9aca08f-b2dd-45ae-adf9-04d97dfe7ff6', 'b9d84586-d23c-44c0-86aa-55bfbf428b10', 'b9e004fd-02ff-4439-b529-c12ae9a0592e', 'ba0db777-83bc-4bc4-beb5-f37f2828ad5c', 'ba2cbef0-7d30-44b3-84c0-168561062142', 'bb11d739-b081-4d09-b9cb-650f69edd911', 'bb489025-b026-4384-9e0f-9657ccd1e273', 'bb66f2d5-b60a-4607-8683-e99358ce52d5', 'bc3a884d-4b5d-4b34-964c-93073b2e0b71', 'bc3cde54-6c43-4b53-9a38-3991bd0fb3cf', 'bc96a34d-3150-4de7-9662-5acf71415a91', 'bd127d1b-fa3b-4687-8b31-2dd7e669d2f7', 'bd15695b-76c8-4c8a-aab4-f6a7b8a8b7e1', 'bd2e45e4-96fe-4b64-ba61-0550e4173b09', 'bd881a96-d1ba-473a-8afb-ff59ae4dba9d', 'bd8df9e5-4b82-4a8b-b3df-2eaa821b575c', 'be64985d-012d-433f-87b0-0efefec06a94', 'be7600cd-dea3-4410-a37d-f140b173727a', 'bf3d8c8b-fe3d-4c40-8d83-1897a4da5551', 'bf7e1a00-f2eb-4eea-8497-fefdeedfdc18', 'bf898e14-3937-43ad-9f3c-3c7557cbb293', 'bfa688b3-3497-41dd-b33f-378c868e6c32', 'bfca5c5e-64dd-48db-9acb-ad2b70f1f719', 'c02220c9-1f09-4e61-be2e-06364412d396', 'c07ba226-cdfd-4f61-b436-8b1b165d44ee', 'c096d50d-665c-41dc-b440-42057cfecdad', 'c0b4cadf-6e92-425e-bb77-8e13c68245c7', 'c12cd770-0871-4f15-88c4-69f571e2a829', 'c1547eb8-e962-40ab-bcf5-bfa66ddd655c', 'c18f400e-9838-4b04-97b3-a42ef9e66e52', 'c24ac0c8-e0ea-44c6-bbbd-6068c4501c1c', 'c2544b4d-756c-44b9-b7a4-c8abb92c8842', 'c27e1314-5807-4fbf-ab36-3083480b08fd', 'c356608e-6448-4212-b554-839603ebcd44', 'c3ac36e0-4220-490d-a909-51ae62ae3baf', 'c3f50ed1-1a93-4597-ac31-ab6ab0b19a18', 'c3f6e06f-900e-45f9-9c23-983c5673f58d', 'c425f577-465c-4681-b0ab-6e73325b0fc7', 'c46bf024-6195-43b4-8cbd-6b33f589e570', 'c46d34e8-4165-4de4-a7c5-0cb8ce888415', 'c48e3db5-5415-4490-a5d1-261a1ebbd9a2', 'c5b19a04-c7ec-4450-b0f6-8c5aa2b5525e', 'c5d5ca3a-d06a-4f12-a81e-0d165cdaf907', 'c66a036c-d0ea-4999-98f2-772fb319156d', 'c6861395-70a5-4007-9256-00cda8f4b498', 'c6bd7f1f-a03b-4394-84e3-a740c43b188a', 'c70c6200-a51e-4747-b0c9-ae683eba9c6b', 'c7610bf4-4d01-407a-a973-dfa76057d698', 'c76580e8-031c-4864-8d7a-7fd65d4ca7a9', 'c8196b2a-fe7b-4bbd-b567-1614f3c85cb1', 'c8a46306-07da-4888-8658-59e57fd61dc9', 'c9260c17-6ccc-497f-b653-5551fd7e2399', 'c9265062-3e4d-41ea-8a4d-e11b3e984eea', 'c92e4122-fe08-4ca6-9388-466a12322799', 'c97775b7-3d00-4132-9598-863797b8ef14', 'c9c61fc9-93e5-470e-8aa0-d276755c4f37', 'c9e94080-4a8d-4902-9fdb-a94eb817ece8', 'c9f10528-9818-487e-9efc-9685b0998569', 'c9fcfab2-c94c-4a75-a2a3-f3362c42ae12', 'ca5ff878-d59b-43a0-a552-1fbfa80ea7e6', 'cb1f7317-d78c-45b2-9e89-6f00a0d3439d', 'cba70e24-9b1e-4bd4-8345-488f1bb7b60d', 'cbe76c34-c205-4062-b1dc-52eb3e667a46', 'cc081549-cc1d-40df-80f2-665a0393fa2a', 'cc0b81df-b04d-4e52-b51b-90fe6c82f384', 'cc4b5465-3689-474d-bed8-b14bb03e500f', 'cca1f343-39ef-4844-ba68-4e01465b1819', 'cd0da383-7284-453e-9865-961513f3c5d2', 'cd0fff1f-7619-4693-a7a2-ee17bea2eb5b', 'cd2eb717-fbee-4e8c-be80-bb3dee696ade', 'cd3b738d-f081-4de2-b313-3326935f1927', 'ce462d76-a2ae-4c94-b281-4c8c797a9d26', 'ce512508-ec09-4699-9384-55eac2c090e6', 'ce814cab-939a-4bdc-8d15-9019f50f23ef', 'cea20b8c-938d-4099-9ed9-9cd689b5c5e7', 'ceaef097-e704-4f18-9473-c448b23d4894', 'cec13e12-1bd9-4194-89af-6306dfb6a451', 'cf239feb-529d-4b3a-a754-02f20e1e201d', 'd01e903e-60de-4084-b037-2af25342ba90', 'd0744fb9-824d-44ce-bb30-25cbaa8ff337', 'd149cfb4-39c0-4e09-82c2-24c2dea4fab0', 'd1da185c-ca4c-41d8-8bf6-1d0003e60455', 'd225feda-d505-4ba4-a964-7de9fba47126', 'd2de899a-73d5-491d-9d9a-35946b57d2c6', 'd424854b-1812-405a-932a-fad3cea927ba', 'd462d1d6-e6a0-4751-ba94-de2ef1d6492c', 'd4d816ff-44ca-40cd-b800-5079a78b26bd', 'd4dee9b6-c5eb-4201-b99a-4e4e1e7e8739', 'd4fcbc95-98ee-4bc9-92e0-cb95f59c9ea4', 'd508a1c2-f4d4-4761-8257-aec6f5548a10', 'd5196f65-817e-4fd8-9373-c57cbf16e3be', 'd57a90f1-90ed-44ce-b424-a7f7faf96f95', 'd68d353a-7e73-44a1-8743-603d194cc4e7', 'd6eb6644-3e38-4e5d-811f-4c138bddb53b', 'd71f4d15-4a0e-463a-af1a-0b93818c4563', 'd74b4cc4-e9cd-4c6b-85ce-ab257ce3fee9', 'd7c6ad0f-3364-4a4a-a409-1da0c139f6d5', 'd7cd1a94-6e7b-49af-a7e6-b964a0bbf79b', 'd7fb317a-3e26-4607-a73c-cc23c511eb2d', 'd89c030c-adac-4298-a84c-70a07592b7ee', 'd8ac2807-c5a5-45e7-9ad3-c13548888090', 'd9021648-5f10-4fbf-9c2b-d2aaf7c60943', 'd922db91-d616-47f2-abba-9cf491686c10', 'd949d6bb-b98b-41e4-a3b5-fb1767d37db4', 'd9f064e6-6c98-41f0-bb61-03d3b7a327ed', 'da7ac4bf-2e8d-4a7f-883f-8cdc4d584b8f', 'db9096d6-9077-43a5-92cd-04f089963b02', 'dc2f88c8-ec2d-4539-99e1-f08023642c13', 'dcb80c8a-11cc-4140-ad70-924f17108ec4', 'dceba897-c946-40b8-bb64-8776cf0d72d9', 'dcf2832c-de63-45b5-b679-bf58641a8b0c', 'dd8b1cd6-93cd-41e5-b184-caf1369cb39f', 'ddd3e600-fe52-49cb-9f43-7a8341aac38a', 'ddec78d5-2cdb-47d2-b0f5-c88f84dab443', 'de2d1c4e-ea6d-4142-b808-d7880bdf6a20', 'de58d8c0-3645-4269-9dbc-071442a1a18f', 'e0fcd96e-3584-4307-8d74-df29c1e8a7d9', 'e12ae3e3-3eca-4e7d-ad7c-9ff77806e1ac', 'e162c270-6e09-4e25-883b-8c27d832ab93', 'e180ecae-99ae-4d20-8806-4a1473f5e82e', 'e185bb8b-ff26-44f9-9faa-7f1d3c431c7b', 'e18a8414-7e8e-4521-87b7-f8838d7a66ef', 'e1a326ae-70d8-4795-8ee0-b5f62e80c030', 'e1cc7675-027a-4f82-aefe-31b27e83ab59', 'e23b7427-a73e-428a-9ba9-758ef4692584', 'e37debd2-5f04-43d1-ba3d-bc4bd9b0eeb0', 'e38fd611-b908-462b-a1c0-fdb9abc1f312', 'e395550a-c186-48a8-94ce-3519d8b6a7ea', 'e4fe3ee0-a97e-4329-9d0a-c81314581c91', 'e51be014-69ff-4cdd-beaf-38e6cfea75e6', 'e59b7d84-834d-456e-8ebd-7d60147ee365', 'e5c7d366-3a68-4c10-bd0a-f294af4a9482', 'e5ca627a-7673-4c26-8c1d-c7325889716a', 'e6654895-fcff-4651-a7df-e20ddc455e40', 'e66a2db9-cbcc-4fad-a194-9858449c9c73', 'e6b6aa3b-c776-49a3-a4ec-3ef68a321203', 'e6b9e27b-7205-490b-a82c-b3e3b27113ae', 'e7bdea0b-0df2-4e0f-af8d-e5e342a0831a', 'e7e200c6-dcae-4aa4-b0d6-08888eb8697e', 'e81634a4-fa04-4af9-8f5e-07edd78ca898', 'e8a11e7a-3783-451f-ba8a-cbe6cc37f77d', 'e90a17e5-0682-4cb0-ad8e-43f3e1381339', 'e95bc96b-6b33-46bb-bb33-b6ea82dfa269', 'e95dac9b-451a-453f-8259-63c027b42f69', 'e99de3e8-531e-4578-9a70-0f0893813311', 'ea017fee-57c8-4e00-afe2-10e7c2dffd0d', 'eacff5a1-bf5a-4fe8-8ec9-6e175a40a9fd', 'eae5ee86-b6ee-4112-941b-311d6026acb7', 'eb470753-670d-4bcf-a99c-1b2bb8f73111', 'eb6fbfca-1932-4406-8a16-51f26a48727b', 'ec671232-dea4-4f41-88cb-cbccb53bd5f6', 'ecb8b512-94ef-49cb-b4a6-f1fb3845c8e0', 'ed4d8f47-473d-4c99-beb9-c7dcf65099c2', 'edc7dbb1-3fad-4e6a-b11d-49f7d82bd490', 'ee1a118e-9598-49e2-bd91-2cc0463ac93b', 'ee1d66b6-5951-480b-a25b-b52a71d3b36a', 'ee8933ed-9dcd-4cb9-a3e2-04e62b74bc8c', 'eee866f2-0a7f-44aa-9cdb-acf4a6fb7ea0', 'ef09dca5-f6b9-472b-9c16-55ec53135b69', 'ef604a98-4756-4328-be81-25ca0ef86754', 'f00977e7-adba-4f58-87d7-ea705ad2108f', 'f020fc78-5c69-481a-adf9-0c6a82f928ca', 'f025468a-fdad-4dfc-9eb1-2b8e4016ec66', 'f0a8e914-c8cd-46a6-988c-757c18753805', 'f0bdb890-14e1-4d0d-9244-8cb788c2d187', 'f131019a-f0f4-41b4-a9ee-64c20a95d83e', 'f15dcf69-9485-4d54-b3ac-dbf18fd2fad3', 'f18ef5d2-0aef-438c-9c1e-25913c5147b2', 'f1ae83c7-a426-47b2-bb8c-3f98d9ba1f10', 'f2060de0-83e1-458e-a41c-4672ea2b31b3', 'f22e5b48-6dbf-433a-855e-ca11878d4b0a', 'f2e65a2c-f37b-4810-83c9-764a5a84f160', 'f35ace9b-0dc5-4236-b54b-ee9a2c404971', 'f365f3b1-37f0-4ed5-be06-e88d7750449b', 'f36edf5e-1fe7-4df4-98c2-caa26c36b9d4', 'f390e159-8582-475e-9702-d7608f976ad4', 'f397f921-bddc-4ed2-9a4c-d285a6eeae38', 'f3c5f37f-2fd2-4794-84ad-71a3c41b2b84', 'f417a8f3-2660-4d66-bba7-7b0dcc002f6c', 'f43fe84f-842c-4bea-8d6f-08bb1d3b2049', 'f47e3e08-0912-4abf-a8a7-9010923dd5f6', 'f4936385-05ea-43ca-b771-9b4e3a1b598b', 'f49da828-7807-4a4c-af13-50163d70cf42', 'f5242b81-56a7-4fa7-9c5a-0760a6635dd8', 'f52f7e17-0bef-4249-80d2-41dd81bcbf58', 'f54b7013-98f7-4ca5-b90c-9dbfbb4480df', 'f554697e-f66a-4e47-bdf1-295d92ff45f3', 'f5f0100d-7a69-4690-9641-fb48356d93cb', 'f6088918-2688-4bf4-a28c-a4f2ef883f52', 'f64ec6e4-6f9a-407c-9982-b849e0375bf1', 'f760cd79-ab9e-4d98-933b-a6c369af606c', 'f79a8700-0a77-44bc-aa82-c88801adb344', 'f7eb9d4a-191c-420c-8c96-f02460721ce3', 'f80cbbe8-64b2-4253-a8a0-c6e91074ffcd', 'f86d8da2-f7af-44dc-a7b3-197336041d94', 'f876438c-d77e-462f-9930-06cac21a7637', 'f87d9a9c-cf20-41c7-823e-2458a53ad386', 'f8812937-55c8-4d0d-9988-29c3d69915c2', 'f881668f-7d0c-44d9-a44c-75ae8393194d', 'f8fa62c4-9d06-4dff-b7a7-a4e2ca30837e', 'f8fd95dd-31ea-445f-84d0-9db431bff817', 'f94a7467-ec75-4df4-ba6c-440ea2572a29', 'faadb032-71f8-4742-80e0-8e3c58a8c9b4', 'facad216-daaa-4df7-a322-f15c2f0eaf87', 'facbb7f5-94df-4dfb-af28-27f37bf74b03', 'fc24119c-eeb5-41e7-9b4a-c6c826dd1445', 'fc7a28f9-7f79-4b87-804f-d7e1e153b9ef', 'fca25ab6-ad07-470a-9462-236ceb03eb13', 'fcfa884e-a5df-47b2-a708-6c34f03ceb8c', 'fd44f0cb-c3b4-4a3e-a192-8d6e03ebe0f5', 'fd8e036e-9d3d-40e6-893b-2ec5095a9854', 'fdf30908-9968-42a6-855c-999745f8cf92', 'fe33cf1c-f323-4f96-9686-b66bd8044111', 'fe7ad30c-0218-42fe-8214-e4f23d48dbbb', 'ffb9a363-1d4a-4479-b148-3809161ce86c'], 'embedded': {'last_modified': {'modified_by': {'uuid': '986b362f-4eb6-4a9c-8173-3ab267227777', 'link_id': '~users~986b362f-4eb6-4a9c-8173-3ab267227777~', '@id': '/users/986b362f-4eb6-4a9c-8173-3ab267227777/', 'display_title': 'Andy Schroeder', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'userid.986b362f-4eb6-4a9c-8173-3ab267227777'], 'view': ['group.admin', 'group.read-only-admin', 'remoteuser.EMBED', 'remoteuser.INDEXER', 'userid.986b362f-4eb6-4a9c-8173-3ab267227777']}}, 'date_modified': '2018-07-31T19:19:50.851044+00:00'}, 'documents': [], 'submitted_by': {'uuid': '95deb527-6d19-45fa-ace9-4e5d4ed1e023', 'link_id': '~users~95deb527-6d19-45fa-ace9-4e5d4ed1e023~', '@id': '/users/95deb527-6d19-45fa-ace9-4e5d4ed1e023/', 'display_title': 'Daan Peric-Hupkes', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'userid.95deb527-6d19-45fa-ace9-4e5d4ed1e023'], 'view': ['group.admin', 'group.read-only-admin', 'remoteuser.EMBED', 'remoteuser.INDEXER', 'userid.95deb527-6d19-45fa-ace9-4e5d4ed1e023']}}, 'aliases': ['vanSteensel-lab:H1_LmnB1'], 'other_processed_files': [{'files': [{'uuid': 'd508a1c2-f4d4-4761-8257-aec6f5548a10', 'file_format': 'txt', '@id': '/files-processed/4DNFIUNJUNBC/', 'display_title': '4DNFIUNJUNBC.txt.gz', 'link_id': '~files-processed~4DNFIUNJUNBC~', 'accession': '4DNFIUNJUNBC', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'c70c6200-a51e-4747-b0c9-ae683eba9c6b', 'file_format': 'bw', '@id': '/files-processed/4DNFI6BH48Y3/', 'display_title': '4DNFI6BH48Y3.bw', 'link_id': '~files-processed~4DNFI6BH48Y3~', 'accession': '4DNFI6BH48Y3', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'GATC binned files', 'description': 'The files associated with the GATC bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': '24043c6b-25a5-4be3-a2aa-b467342e6e83', 'file_format': 'txt', '@id': '/files-processed/4DNFIOFN8R1O/', 'display_title': '4DNFIOFN8R1O.txt.gz', 'link_id': '~files-processed~4DNFIOFN8R1O~', 'accession': '4DNFIOFN8R1O', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '28b12db0-1f11-4a6d-ac89-1a4a44218c16', 'file_format': 'txt', '@id': '/files-processed/4DNFIO2NKJRZ/', 'display_title': '4DNFIO2NKJRZ.txt.gz', 'link_id': '~files-processed~4DNFIO2NKJRZ~', 'accession': '4DNFIO2NKJRZ', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Additional 1 kb binned files', 'description': 'The additional supplementary files associated with the 1 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': '9bd01d51-3aee-4535-9071-752499ca6a70', 'file_format': 'txt', '@id': '/files-processed/4DNFILWLO74H/', 'display_title': '4DNFILWLO74H.txt.gz', 'link_id': '~files-processed~4DNFILWLO74H~', 'accession': '4DNFILWLO74H', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2c0c7103-5a56-410f-b6c4-d91dd0e16bcf', 'file_format': 'bw', '@id': '/files-processed/4DNFIW6T3CC3/', 'display_title': '4DNFIW6T3CC3.bw', 'link_id': '~files-processed~4DNFIW6T3CC3~', 'accession': '4DNFIW6T3CC3', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2a4979c5-3f44-4d5d-8ffe-5f17ba15b793', 'file_format': 'txt', '@id': '/files-processed/4DNFIQOYO2PY/', 'display_title': '4DNFIQOYO2PY.txt.gz', 'link_id': '~files-processed~4DNFIQOYO2PY~', 'accession': '4DNFIQOYO2PY', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '01fab0f2-e76a-4a14-a77e-e5239dd391dc', 'file_format': 'bed', '@id': '/files-processed/4DNFITEE8XZK/', 'display_title': '4DNFITEE8XZK.bed.gz', 'link_id': '~files-processed~4DNFITEE8XZK~', 'accession': '4DNFITEE8XZK', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '2 kb binned files', 'description': 'The files associated with the 2 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': 'e6654895-fcff-4651-a7df-e20ddc455e40', 'file_format': 'txt', '@id': '/files-processed/4DNFITUR6YTN/', 'display_title': '4DNFITUR6YTN.txt.gz', 'link_id': '~files-processed~4DNFITUR6YTN~', 'accession': '4DNFITUR6YTN', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '063961e2-891e-4b2f-be49-97a400f9a1e9', 'file_format': 'bw', '@id': '/files-processed/4DNFIXNBG8L1/', 'display_title': '4DNFIXNBG8L1.bw', 'link_id': '~files-processed~4DNFIXNBG8L1~', 'accession': '4DNFIXNBG8L1', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '9cd05f6f-d339-473d-a566-8eec65ce51e6', 'file_format': 'txt', '@id': '/files-processed/4DNFICZGAM4H/', 'display_title': '4DNFICZGAM4H.txt.gz', 'link_id': '~files-processed~4DNFICZGAM4H~', 'accession': '4DNFICZGAM4H', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'bb489025-b026-4384-9e0f-9657ccd1e273', 'file_format': 'bed', '@id': '/files-processed/4DNFIP6N54B3/', 'display_title': '4DNFIP6N54B3.bed.gz', 'link_id': '~files-processed~4DNFIP6N54B3~', 'accession': '4DNFIP6N54B3', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '5 kb binned files', 'description': 'The files associated with the 5 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': 'eb470753-670d-4bcf-a99c-1b2bb8f73111', 'file_format': 'txt', '@id': '/files-processed/4DNFI1H5LWWB/', 'display_title': '4DNFI1H5LWWB.txt.gz', 'link_id': '~files-processed~4DNFI1H5LWWB~', 'accession': '4DNFI1H5LWWB', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1f4012c8-8b5e-4c5b-9604-c35d8a5e3c21', 'file_format': 'bw', '@id': '/files-processed/4DNFIAKUT6VX/', 'display_title': '4DNFIAKUT6VX.bw', 'link_id': '~files-processed~4DNFIAKUT6VX~', 'accession': '4DNFIAKUT6VX', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '3d43a37c-af1b-42ee-9d2c-d3843f63214c', 'file_format': 'txt', '@id': '/files-processed/4DNFITUID4VI/', 'display_title': '4DNFITUID4VI.txt.gz', 'link_id': '~files-processed~4DNFITUID4VI~', 'accession': '4DNFITUID4VI', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '06ea837b-093e-4cc5-bc1d-87ac9f19f6c2', 'file_format': 'bed', '@id': '/files-processed/4DNFIRP8P9MC/', 'display_title': '4DNFIRP8P9MC.bed.gz', 'link_id': '~files-processed~4DNFIRP8P9MC~', 'accession': '4DNFIRP8P9MC', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '10 kb binned files', 'description': 'The files associated with the 10 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': '5402ee85-d568-4d40-b86d-fec615f5b419', 'file_format': 'txt', '@id': '/files-processed/4DNFI7NP9IVR/', 'display_title': '4DNFI7NP9IVR.txt.gz', 'link_id': '~files-processed~4DNFI7NP9IVR~', 'accession': '4DNFI7NP9IVR', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e6b9e27b-7205-490b-a82c-b3e3b27113ae', 'file_format': 'bw', '@id': '/files-processed/4DNFI2J1OTL1/', 'display_title': '4DNFI2J1OTL1.bw', 'link_id': '~files-processed~4DNFI2J1OTL1~', 'accession': '4DNFI2J1OTL1', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'ddec78d5-2cdb-47d2-b0f5-c88f84dab443', 'file_format': 'txt', '@id': '/files-processed/4DNFIXRI6UIH/', 'display_title': '4DNFIXRI6UIH.txt.gz', 'link_id': '~files-processed~4DNFIXRI6UIH~', 'accession': '4DNFIXRI6UIH', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'cc0b81df-b04d-4e52-b51b-90fe6c82f384', 'file_format': 'bed', '@id': '/files-processed/4DNFIJFD4TWE/', 'display_title': '4DNFIJFD4TWE.bed.gz', 'link_id': '~files-processed~4DNFIJFD4TWE~', 'accession': '4DNFIJFD4TWE', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '20 kb binned files', 'description': 'The files associated with the 20 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': '2bcb9dcc-7dbe-4c1e-b063-7c691db4ce22', 'file_format': 'txt', '@id': '/files-processed/4DNFIGPBGODE/', 'display_title': '4DNFIGPBGODE.txt.gz', 'link_id': '~files-processed~4DNFIGPBGODE~', 'accession': '4DNFIGPBGODE', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0e4bf888-08e9-40f2-a7da-a33efd00aeee', 'file_format': 'bw', '@id': '/files-processed/4DNFIBTQYNRA/', 'display_title': '4DNFIBTQYNRA.bw', 'link_id': '~files-processed~4DNFIBTQYNRA~', 'accession': '4DNFIBTQYNRA', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e1a326ae-70d8-4795-8ee0-b5f62e80c030', 'file_format': 'txt', '@id': '/files-processed/4DNFIMOQJ97R/', 'display_title': '4DNFIMOQJ97R.txt.gz', 'link_id': '~files-processed~4DNFIMOQJ97R~', 'accession': '4DNFIMOQJ97R', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '55cb0f75-30c0-4493-9b5f-e561da2a5642', 'file_format': 'bed', '@id': '/files-processed/4DNFIIJJZHF2/', 'display_title': '4DNFIIJJZHF2.bed.gz', 'link_id': '~files-processed~4DNFIIJJZHF2~', 'accession': '4DNFIIJJZHF2', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '25 kb binned files', 'description': 'The files associated with the 25 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': '03f64d2f-9be4-47ea-aa53-25cece95b8a8', 'file_format': 'txt', '@id': '/files-processed/4DNFIOFREOYI/', 'display_title': '4DNFIOFREOYI.txt.gz', 'link_id': '~files-processed~4DNFIOFREOYI~', 'accession': '4DNFIOFREOYI', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'c02220c9-1f09-4e61-be2e-06364412d396', 'file_format': 'bw', '@id': '/files-processed/4DNFIVTMNBV5/', 'display_title': '4DNFIVTMNBV5.bw', 'link_id': '~files-processed~4DNFIVTMNBV5~', 'accession': '4DNFIVTMNBV5', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'fe33cf1c-f323-4f96-9686-b66bd8044111', 'file_format': 'txt', '@id': '/files-processed/4DNFIEOK4OOX/', 'display_title': '4DNFIEOK4OOX.txt.gz', 'link_id': '~files-processed~4DNFIEOK4OOX~', 'accession': '4DNFIEOK4OOX', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8310a3b2-e421-4656-84dd-f7b0abfea638', 'file_format': 'bed', '@id': '/files-processed/4DNFIJXADI29/', 'display_title': '4DNFIJXADI29.bed.gz', 'link_id': '~files-processed~4DNFIJXADI29~', 'accession': '4DNFIJXADI29', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '50 kb binned files', 'description': 'The files associated with the 50 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': '6e378c15-bf09-4304-bd45-e8a444c2a5fa', 'file_format': 'txt', '@id': '/files-processed/4DNFIPYC31MH/', 'display_title': '4DNFIPYC31MH.txt.gz', 'link_id': '~files-processed~4DNFIPYC31MH~', 'accession': '4DNFIPYC31MH', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a20ead51-2d13-4fb9-b781-c2b172beb4b0', 'file_format': 'bw', '@id': '/files-processed/4DNFIJVMELGG/', 'display_title': '4DNFIJVMELGG.bw', 'link_id': '~files-processed~4DNFIJVMELGG~', 'accession': '4DNFIJVMELGG', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'bd8df9e5-4b82-4a8b-b3df-2eaa821b575c', 'file_format': 'txt', '@id': '/files-processed/4DNFI2H9QMZU/', 'display_title': '4DNFI2H9QMZU.txt.gz', 'link_id': '~files-processed~4DNFI2H9QMZU~', 'accession': '4DNFI2H9QMZU', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '401e101d-55c5-45d1-942b-d036f7d3e70f', 'file_format': 'bed', '@id': '/files-processed/4DNFI7QVVZWA/', 'display_title': '4DNFI7QVVZWA.bed.gz', 'link_id': '~files-processed~4DNFI7QVVZWA~', 'accession': '4DNFI7QVVZWA', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '80 kb binned files', 'description': 'The files associated with the 80 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': 'f00977e7-adba-4f58-87d7-ea705ad2108f', 'file_format': 'txt', '@id': '/files-processed/4DNFIQ32VX9D/', 'display_title': '4DNFIQ32VX9D.txt.gz', 'link_id': '~files-processed~4DNFIQ32VX9D~', 'accession': '4DNFIQ32VX9D', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '91c4ce9e-1620-4cf6-9127-23107829bc60', 'file_format': 'bw', '@id': '/files-processed/4DNFICT88Y52/', 'display_title': '4DNFICT88Y52.bw', 'link_id': '~files-processed~4DNFICT88Y52~', 'accession': '4DNFICT88Y52', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '40b70f90-9f0e-464f-bfba-7616603a64ad', 'file_format': 'txt', '@id': '/files-processed/4DNFI8KB9H8R/', 'display_title': '4DNFI8KB9H8R.txt.gz', 'link_id': '~files-processed~4DNFI8KB9H8R~', 'accession': '4DNFI8KB9H8R', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e395550a-c186-48a8-94ce-3519d8b6a7ea', 'file_format': 'bed', '@id': '/files-processed/4DNFIKSEZOJR/', 'display_title': '4DNFIKSEZOJR.bed.gz', 'link_id': '~files-processed~4DNFIKSEZOJR~', 'accession': '4DNFIKSEZOJR', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '100 kb binned files', 'description': 'The files associated with the 100 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': 'e23b7427-a73e-428a-9ba9-758ef4692584', 'file_format': 'txt', '@id': '/files-processed/4DNFIQP16L8Y/', 'display_title': '4DNFIQP16L8Y.txt.gz', 'link_id': '~files-processed~4DNFIQP16L8Y~', 'accession': '4DNFIQP16L8Y', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8fe88d17-a0fd-4366-8f4a-ec931aea4a61', 'file_format': 'bw', '@id': '/files-processed/4DNFIIP7GJ1N/', 'display_title': '4DNFIIP7GJ1N.bw', 'link_id': '~files-processed~4DNFIIP7GJ1N~', 'accession': '4DNFIIP7GJ1N', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'fca25ab6-ad07-470a-9462-236ceb03eb13', 'file_format': 'txt', '@id': '/files-processed/4DNFIU5LSP7P/', 'display_title': '4DNFIU5LSP7P.txt.gz', 'link_id': '~files-processed~4DNFIU5LSP7P~', 'accession': '4DNFIU5LSP7P', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7e45306a-ed29-4065-a25b-acc3aaf112d5', 'file_format': 'bed', '@id': '/files-processed/4DNFIQQN9DWQ/', 'display_title': '4DNFIQQN9DWQ.bed.gz', 'link_id': '~files-processed~4DNFIQQN9DWQ~', 'accession': '4DNFIQQN9DWQ', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '250 kb binned files', 'description': 'The files associated with the 250 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': [{'uuid': 'fcfa884e-a5df-47b2-a708-6c34f03ceb8c', 'file_format': 'other', '@id': '/files-processed/4DNFIIFBEEEU/', 'display_title': '4DNFIIFBEEEU', 'link_id': '~files-processed~4DNFIIFBEEEU~', 'accession': '4DNFIIFBEEEU', 'file_type_detailed': 'other (other)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8896846b-9c19-493a-ad78-b29162eea666', 'file_format': 'other', '@id': '/files-processed/4DNFI5RCIF4F/', 'display_title': '4DNFI5RCIF4F', 'link_id': '~files-processed~4DNFI5RCIF4F~', 'accession': '4DNFI5RCIF4F', 'file_type_detailed': 'other (other)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Supplementary files for H1 Dataset', 'description': 'Non-bin specific supplementary files for the H1 LMNB1 Dataset.'}], 'schema_version': '2', 'date_created': '2017-12-14T16:19:18.119382+00:00', 'alternate_accessions': [], 'description': 'LaminB1 DamID of H1-hESC Tier 1 cells – cells were transduced with virus expressing Dam-LaminB1, gDNA was harvested after 4 days and processed for DamID-seq', 'experiments_in_set': [{'uuid': '22351015-90bb-4121-bc1f-5b5d99c65769', 'other_processed_files': [{'files': [{'uuid': 'c18f400e-9838-4b04-97b3-a42ef9e66e52', 'file_format': 'txt', '@id': '/files-processed/4DNFIMUPUULG/', 'display_title': '4DNFIMUPUULG.txt.gz', 'link_id': '~files-processed~4DNFIMUPUULG~', 'accession': '4DNFIMUPUULG', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '78775159-a731-44d0-bb9e-281787541407', 'file_format': 'bw', '@id': '/files-processed/4DNFI2RWYK3Y/', 'display_title': '4DNFI2RWYK3Y.bw', 'link_id': '~files-processed~4DNFI2RWYK3Y~', 'accession': '4DNFI2RWYK3Y', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '149b37bd-eff8-448a-832e-ae592c52fd2e', 'file_format': 'txt', '@id': '/files-processed/4DNFI3ZY7SPI/', 'display_title': '4DNFI3ZY7SPI.txt.gz', 'link_id': '~files-processed~4DNFI3ZY7SPI~', 'accession': '4DNFI3ZY7SPI', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'f365f3b1-37f0-4ed5-be06-e88d7750449b', 'file_format': 'bw', '@id': '/files-processed/4DNFIIU9CAU1/', 'display_title': '4DNFIIU9CAU1.bw', 'link_id': '~files-processed~4DNFIIU9CAU1~', 'accession': '4DNFIIU9CAU1', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'GATC binned files', 'description': 'The files associated with the GATC bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '74e09d91-0329-45d1-899d-7b9c450926b4', 'file_format': 'txt', '@id': '/files-processed/4DNFIZUDVFLD/', 'display_title': '4DNFIZUDVFLD.txt.gz', 'link_id': '~files-processed~4DNFIZUDVFLD~', 'accession': '4DNFIZUDVFLD', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8bdc8002-c4cf-43ca-b0f0-0a35eb688101', 'file_format': 'bw', '@id': '/files-processed/4DNFIIRTYHJW/', 'display_title': '4DNFIIRTYHJW.bw', 'link_id': '~files-processed~4DNFIIRTYHJW~', 'accession': '4DNFIIRTYHJW', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8300c32f-9a81-4af1-855d-8ba3d9e09edf', 'file_format': 'txt', '@id': '/files-processed/4DNFICWH4B7E/', 'display_title': '4DNFICWH4B7E.txt.gz', 'link_id': '~files-processed~4DNFICWH4B7E~', 'accession': '4DNFICWH4B7E', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a65740ee-db45-46ff-ad72-122bbb0d108a', 'file_format': 'txt', '@id': '/files-processed/4DNFIF6OCLYH/', 'display_title': '4DNFIF6OCLYH.txt.gz', 'link_id': '~files-processed~4DNFIF6OCLYH~', 'accession': '4DNFIF6OCLYH', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Additional 1 kb binned files', 'description': 'The additional supplementary files associated with the 1 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '439e1ffb-35f5-445e-a968-e5414d0a6e4c', 'file_format': 'txt', '@id': '/files-processed/4DNFIUOEZ8JD/', 'display_title': '4DNFIUOEZ8JD.txt.gz', 'link_id': '~files-processed~4DNFIUOEZ8JD~', 'accession': '4DNFIUOEZ8JD', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '95e5a239-3010-4d3b-ba53-c1439ea3f89b', 'file_format': 'bw', '@id': '/files-processed/4DNFIPOWO4OO/', 'display_title': '4DNFIPOWO4OO.bw', 'link_id': '~files-processed~4DNFIPOWO4OO~', 'accession': '4DNFIPOWO4OO', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2437af1f-f6ec-487d-be44-c4a0700ac11c', 'file_format': 'txt', '@id': '/files-processed/4DNFIVIY962O/', 'display_title': '4DNFIVIY962O.txt.gz', 'link_id': '~files-processed~4DNFIVIY962O~', 'accession': '4DNFIVIY962O', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8d3f8614-18da-4717-ab5a-90e369d508b1', 'file_format': 'bw', '@id': '/files-processed/4DNFITCGZKST/', 'display_title': '4DNFITCGZKST.bw', 'link_id': '~files-processed~4DNFITCGZKST~', 'accession': '4DNFITCGZKST', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'de58d8c0-3645-4269-9dbc-071442a1a18f', 'file_format': 'txt', '@id': '/files-processed/4DNFIOBQZ4A8/', 'display_title': '4DNFIOBQZ4A8.txt.gz', 'link_id': '~files-processed~4DNFIOBQZ4A8~', 'accession': '4DNFIOBQZ4A8', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7321fb29-e465-4baf-ae3a-cf88defcd9fc', 'file_format': 'bed', '@id': '/files-processed/4DNFII3DA8XO/', 'display_title': '4DNFII3DA8XO.bed.gz', 'link_id': '~files-processed~4DNFII3DA8XO~', 'accession': '4DNFII3DA8XO', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '2 kb binned files', 'description': 'The files associated with the 2 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': 'acf3e2bd-59b7-43cb-b9fc-0aaeaad35263', 'file_format': 'txt', '@id': '/files-processed/4DNFICZNI5GI/', 'display_title': '4DNFICZNI5GI.txt.gz', 'link_id': '~files-processed~4DNFICZNI5GI~', 'accession': '4DNFICZNI5GI', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '07b05436-e2da-46f2-90ae-f460dd7abddf', 'file_format': 'bw', '@id': '/files-processed/4DNFIHMBA6OE/', 'display_title': '4DNFIHMBA6OE.bw', 'link_id': '~files-processed~4DNFIHMBA6OE~', 'accession': '4DNFIHMBA6OE', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '3a2f7cea-9e9a-4c8d-b501-f2f27776b425', 'file_format': 'txt', '@id': '/files-processed/4DNFIZH98GRF/', 'display_title': '4DNFIZH98GRF.txt.gz', 'link_id': '~files-processed~4DNFIZH98GRF~', 'accession': '4DNFIZH98GRF', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '41d1f0be-32e1-4e23-a9fc-6210ee7665ff', 'file_format': 'bw', '@id': '/files-processed/4DNFI3ZYP14W/', 'display_title': '4DNFI3ZYP14W.bw', 'link_id': '~files-processed~4DNFI3ZYP14W~', 'accession': '4DNFI3ZYP14W', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'ffb9a363-1d4a-4479-b148-3809161ce86c', 'file_format': 'txt', '@id': '/files-processed/4DNFIOF2UDX2/', 'display_title': '4DNFIOF2UDX2.txt.gz', 'link_id': '~files-processed~4DNFIOF2UDX2~', 'accession': '4DNFIOF2UDX2', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '32a077d8-584d-4223-bf11-9f5df1d2797b', 'file_format': 'bed', '@id': '/files-processed/4DNFIQY7JNXE/', 'display_title': '4DNFIQY7JNXE.bed.gz', 'link_id': '~files-processed~4DNFIQY7JNXE~', 'accession': '4DNFIQY7JNXE', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '5 kb binned files', 'description': 'The files associated with the 5 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': 'b5e13c68-4f56-4353-a769-4c09e530c483', 'file_format': 'txt', '@id': '/files-processed/4DNFILPK3TYU/', 'display_title': '4DNFILPK3TYU.txt.gz', 'link_id': '~files-processed~4DNFILPK3TYU~', 'accession': '4DNFILPK3TYU', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7153c1b9-4d08-46dd-866e-ef8333dba917', 'file_format': 'bw', '@id': '/files-processed/4DNFIA397ZZA/', 'display_title': '4DNFIA397ZZA.bw', 'link_id': '~files-processed~4DNFIA397ZZA~', 'accession': '4DNFIA397ZZA', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'c9e94080-4a8d-4902-9fdb-a94eb817ece8', 'file_format': 'txt', '@id': '/files-processed/4DNFICOWGBZG/', 'display_title': '4DNFICOWGBZG.txt.gz', 'link_id': '~files-processed~4DNFICOWGBZG~', 'accession': '4DNFICOWGBZG', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '90bc7ff3-bf6c-430f-9a07-1026e28c2d60', 'file_format': 'bw', '@id': '/files-processed/4DNFIX6K18EB/', 'display_title': '4DNFIX6K18EB.bw', 'link_id': '~files-processed~4DNFIX6K18EB~', 'accession': '4DNFIX6K18EB', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'fc24119c-eeb5-41e7-9b4a-c6c826dd1445', 'file_format': 'txt', '@id': '/files-processed/4DNFIUS6Z416/', 'display_title': '4DNFIUS6Z416.txt.gz', 'link_id': '~files-processed~4DNFIUS6Z416~', 'accession': '4DNFIUS6Z416', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a5cde8c5-5172-457f-9fe5-9e2073fb800c', 'file_format': 'bed', '@id': '/files-processed/4DNFI72LPVYW/', 'display_title': '4DNFI72LPVYW.bed.gz', 'link_id': '~files-processed~4DNFI72LPVYW~', 'accession': '4DNFI72LPVYW', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '10 kb binned files', 'description': 'The files associated with the 10 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': 'e162c270-6e09-4e25-883b-8c27d832ab93', 'file_format': 'txt', '@id': '/files-processed/4DNFIXKWGN5F/', 'display_title': '4DNFIXKWGN5F.txt.gz', 'link_id': '~files-processed~4DNFIXKWGN5F~', 'accession': '4DNFIXKWGN5F', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '417c774f-0ca4-464e-8f29-44cbd7f299fe', 'file_format': 'bw', '@id': '/files-processed/4DNFIAEFP2BT/', 'display_title': '4DNFIAEFP2BT.bw', 'link_id': '~files-processed~4DNFIAEFP2BT~', 'accession': '4DNFIAEFP2BT', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '6851452d-1227-4e68-bf9e-b5a00fffd22d', 'file_format': 'txt', '@id': '/files-processed/4DNFIUD7PNN2/', 'display_title': '4DNFIUD7PNN2.txt.gz', 'link_id': '~files-processed~4DNFIUD7PNN2~', 'accession': '4DNFIUD7PNN2', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'f87d9a9c-cf20-41c7-823e-2458a53ad386', 'file_format': 'bw', '@id': '/files-processed/4DNFIIB6D134/', 'display_title': '4DNFIIB6D134.bw', 'link_id': '~files-processed~4DNFIIB6D134~', 'accession': '4DNFIIB6D134', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '868895c8-2497-4fc6-9fae-854345da5d8d', 'file_format': 'txt', '@id': '/files-processed/4DNFIKK762LE/', 'display_title': '4DNFIKK762LE.txt.gz', 'link_id': '~files-processed~4DNFIKK762LE~', 'accession': '4DNFIKK762LE', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a13b6328-4aad-42d4-81dd-6d07c39600b4', 'file_format': 'bed', '@id': '/files-processed/4DNFIVX7SW5D/', 'display_title': '4DNFIVX7SW5D.bed.gz', 'link_id': '~files-processed~4DNFIVX7SW5D~', 'accession': '4DNFIVX7SW5D', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '20 kb binned files', 'description': 'The files associated with the 20 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '748db193-9256-4976-ac6f-c4e9e823ffbc', 'file_format': 'txt', '@id': '/files-processed/4DNFIXXYHI65/', 'display_title': '4DNFIXXYHI65.txt.gz', 'link_id': '~files-processed~4DNFIXXYHI65~', 'accession': '4DNFIXXYHI65', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '92b65796-33cb-4b1a-98a8-b25f27e92cfb', 'file_format': 'bw', '@id': '/files-processed/4DNFID2W2A9P/', 'display_title': '4DNFID2W2A9P.bw', 'link_id': '~files-processed~4DNFID2W2A9P~', 'accession': '4DNFID2W2A9P', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '12c9e3f3-d382-4fb2-b024-e7c3c1dcd02d', 'file_format': 'txt', '@id': '/files-processed/4DNFI3K9IDQ5/', 'display_title': '4DNFI3K9IDQ5.txt.gz', 'link_id': '~files-processed~4DNFI3K9IDQ5~', 'accession': '4DNFI3K9IDQ5', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '97acc49e-fd3b-46e5-adf2-cf9112b52cb0', 'file_format': 'bw', '@id': '/files-processed/4DNFIUU9WBLG/', 'display_title': '4DNFIUU9WBLG.bw', 'link_id': '~files-processed~4DNFIUU9WBLG~', 'accession': '4DNFIUU9WBLG', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '727b9399-92f9-41f2-a61e-8532fef538dd', 'file_format': 'txt', '@id': '/files-processed/4DNFIV4OGTPB/', 'display_title': '4DNFIV4OGTPB.txt.gz', 'link_id': '~files-processed~4DNFIV4OGTPB~', 'accession': '4DNFIV4OGTPB', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2f3a81c3-553c-44b3-9431-4b7e4b5a0e7b', 'file_format': 'bed', '@id': '/files-processed/4DNFIAH1X1JX/', 'display_title': '4DNFIAH1X1JX.bed.gz', 'link_id': '~files-processed~4DNFIAH1X1JX~', 'accession': '4DNFIAH1X1JX', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '25 kb binned files', 'description': 'The files associated with the 25 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '4e19b7ac-86fc-45dd-9a64-ccc1ba61ec41', 'file_format': 'txt', '@id': '/files-processed/4DNFIUXTPDJQ/', 'display_title': '4DNFIUXTPDJQ.txt.gz', 'link_id': '~files-processed~4DNFIUXTPDJQ~', 'accession': '4DNFIUXTPDJQ', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e12ae3e3-3eca-4e7d-ad7c-9ff77806e1ac', 'file_format': 'bw', '@id': '/files-processed/4DNFIDPJ76GV/', 'display_title': '4DNFIDPJ76GV.bw', 'link_id': '~files-processed~4DNFIDPJ76GV~', 'accession': '4DNFIDPJ76GV', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'd9021648-5f10-4fbf-9c2b-d2aaf7c60943', 'file_format': 'txt', '@id': '/files-processed/4DNFIG9VJMGV/', 'display_title': '4DNFIG9VJMGV.txt.gz', 'link_id': '~files-processed~4DNFIG9VJMGV~', 'accession': '4DNFIG9VJMGV', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'd7cd1a94-6e7b-49af-a7e6-b964a0bbf79b', 'file_format': 'bw', '@id': '/files-processed/4DNFIJOC9T9W/', 'display_title': '4DNFIJOC9T9W.bw', 'link_id': '~files-processed~4DNFIJOC9T9W~', 'accession': '4DNFIJOC9T9W', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '548acb46-ca54-4b6c-9597-02a8e4f5e4d9', 'file_format': 'txt', '@id': '/files-processed/4DNFIRAQ65E4/', 'display_title': '4DNFIRAQ65E4.txt.gz', 'link_id': '~files-processed~4DNFIRAQ65E4~', 'accession': '4DNFIRAQ65E4', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'd01e903e-60de-4084-b037-2af25342ba90', 'file_format': 'bed', '@id': '/files-processed/4DNFIO48GGXY/', 'display_title': '4DNFIO48GGXY.bed.gz', 'link_id': '~files-processed~4DNFIO48GGXY~', 'accession': '4DNFIO48GGXY', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '50 kb binned files', 'description': 'The files associated with the 50 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': 'f8fd95dd-31ea-445f-84d0-9db431bff817', 'file_format': 'txt', '@id': '/files-processed/4DNFIM252YOD/', 'display_title': '4DNFIM252YOD.txt.gz', 'link_id': '~files-processed~4DNFIM252YOD~', 'accession': '4DNFIM252YOD', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '5396f6de-15f3-49a7-a031-e7e81c122195', 'file_format': 'bw', '@id': '/files-processed/4DNFIOHK4BWC/', 'display_title': '4DNFIOHK4BWC.bw', 'link_id': '~files-processed~4DNFIOHK4BWC~', 'accession': '4DNFIOHK4BWC', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '97b74f37-00f9-4636-b384-d466a32432af', 'file_format': 'txt', '@id': '/files-processed/4DNFIPVS1OVF/', 'display_title': '4DNFIPVS1OVF.txt.gz', 'link_id': '~files-processed~4DNFIPVS1OVF~', 'accession': '4DNFIPVS1OVF', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7e2b2747-2210-4d31-bcad-104d545d15a5', 'file_format': 'bw', '@id': '/files-processed/4DNFID6LQ59Z/', 'display_title': '4DNFID6LQ59Z.bw', 'link_id': '~files-processed~4DNFID6LQ59Z~', 'accession': '4DNFID6LQ59Z', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '36727775-f63d-4bc7-b05a-1115af9af8f9', 'file_format': 'txt', '@id': '/files-processed/4DNFIAW36PIA/', 'display_title': '4DNFIAW36PIA.txt.gz', 'link_id': '~files-processed~4DNFIAW36PIA~', 'accession': '4DNFIAW36PIA', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b3859df0-da57-40cd-9b76-6ef5efc9b537', 'file_format': 'bed', '@id': '/files-processed/4DNFIMIFLZPJ/', 'display_title': '4DNFIMIFLZPJ.bed.gz', 'link_id': '~files-processed~4DNFIMIFLZPJ~', 'accession': '4DNFIMIFLZPJ', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '80 kb binned files', 'description': 'The files associated with the 80 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '580f3d5d-cfbe-4536-8cb3-b226e76d0c45', 'file_format': 'txt', '@id': '/files-processed/4DNFI1524I4Z/', 'display_title': '4DNFI1524I4Z.txt.gz', 'link_id': '~files-processed~4DNFI1524I4Z~', 'accession': '4DNFI1524I4Z', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2ab85c11-14da-49d2-b428-385b6ee341bf', 'file_format': 'bw', '@id': '/files-processed/4DNFIWQDEPQ4/', 'display_title': '4DNFIWQDEPQ4.bw', 'link_id': '~files-processed~4DNFIWQDEPQ4~', 'accession': '4DNFIWQDEPQ4', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '14949ebc-4257-4fe3-a375-fe687034f7e7', 'file_format': 'txt', '@id': '/files-processed/4DNFIGXCDMY2/', 'display_title': '4DNFIGXCDMY2.txt.gz', 'link_id': '~files-processed~4DNFIGXCDMY2~', 'accession': '4DNFIGXCDMY2', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e18a8414-7e8e-4521-87b7-f8838d7a66ef', 'file_format': 'bw', '@id': '/files-processed/4DNFIP38F3PZ/', 'display_title': '4DNFIP38F3PZ.bw', 'link_id': '~files-processed~4DNFIP38F3PZ~', 'accession': '4DNFIP38F3PZ', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '344fd27f-2406-4171-9c0f-f63c32df5abf', 'file_format': 'txt', '@id': '/files-processed/4DNFIXG7M4V9/', 'display_title': '4DNFIXG7M4V9.txt.gz', 'link_id': '~files-processed~4DNFIXG7M4V9~', 'accession': '4DNFIXG7M4V9', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '9a667a78-abfe-4983-bcb1-3aba11347ca7', 'file_format': 'bed', '@id': '/files-processed/4DNFI8F6ZE85/', 'display_title': '4DNFI8F6ZE85.bed.gz', 'link_id': '~files-processed~4DNFI8F6ZE85~', 'accession': '4DNFI8F6ZE85', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '100 kb binned files', 'description': 'The files associated with the 100 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '8cb4f8f2-ed9f-4875-9b1f-6dbacb26d0d2', 'file_format': 'txt', '@id': '/files-processed/4DNFIXXKR7XI/', 'display_title': '4DNFIXXKR7XI.txt.gz', 'link_id': '~files-processed~4DNFIXXKR7XI~', 'accession': '4DNFIXXKR7XI', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'bf7e1a00-f2eb-4eea-8497-fefdeedfdc18', 'file_format': 'bw', '@id': '/files-processed/4DNFI3NGH1YS/', 'display_title': '4DNFI3NGH1YS.bw', 'link_id': '~files-processed~4DNFI3NGH1YS~', 'accession': '4DNFI3NGH1YS', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '281bc792-e782-4308-9d18-2d96bfcf6389', 'file_format': 'txt', '@id': '/files-processed/4DNFI1SOVHGB/', 'display_title': '4DNFI1SOVHGB.txt.gz', 'link_id': '~files-processed~4DNFI1SOVHGB~', 'accession': '4DNFI1SOVHGB', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'fe7ad30c-0218-42fe-8214-e4f23d48dbbb', 'file_format': 'bw', '@id': '/files-processed/4DNFIJOD1O7C/', 'display_title': '4DNFIJOD1O7C.bw', 'link_id': '~files-processed~4DNFIJOD1O7C~', 'accession': '4DNFIJOD1O7C', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '30b22cf1-e3d0-44db-8e27-cd79aabb3d6d', 'file_format': 'txt', '@id': '/files-processed/4DNFIHHKSULZ/', 'display_title': '4DNFIHHKSULZ.txt.gz', 'link_id': '~files-processed~4DNFIHHKSULZ~', 'accession': '4DNFIHHKSULZ', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0d9b34fc-1399-42d4-ab91-999b166b0701', 'file_format': 'bed', '@id': '/files-processed/4DNFIR1UTBX5/', 'display_title': '4DNFIR1UTBX5.bed.gz', 'link_id': '~files-processed~4DNFIR1UTBX5~', 'accession': '4DNFIR1UTBX5', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '250 kb binned files', 'description': 'The files associated with the 250 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 1.'}, {'files': [{'uuid': '0f3b47e4-2365-4b5b-94fe-35b0fcc0b3c4', 'file_format': 'other', '@id': '/files-processed/4DNFIQZJ9IWX/', 'display_title': '4DNFIQZJ9IWX', 'link_id': '~files-processed~4DNFIQZJ9IWX~', 'accession': '4DNFIQZJ9IWX', 'file_type_detailed': 'other (other)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Supplementary files for Replicate 1', 'description': 'Non-bin specific supplementary files for Replicate 1'}], 'experiment_categorizer': {'combined': 'Target: Protein:Lamin-B1 (P20700)', 'field': 'Target', 'value': 'Protein:Lamin-B1 (P20700)'}, 'status': 'released', 'link_id': '~experiments-damid~4DNEXR716AG4~', 'processed_files': [{'uuid': '74a4af26-0e2f-4818-969e-ae772af46cba', 'file_size': 473487261, 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIJDN1FW4/@@download/4DNFIJDN1FW4.bam', 'file_type': 'alignment', 'link_id': '~files-processed~4DNFIJDN1FW4~', 'file_type_detailed': 'alignment (bam)', 'md5sum': '65bd729e03aae3938fd5d2527bb6b52c', 'file_format': 'bam', '@id': '/files-processed/4DNFIJDN1FW4/', 'display_title': '4DNFIJDN1FW4.bam', 'upload_key': '74a4af26-0e2f-4818-969e-ae772af46cba/4DNFIJDN1FW4.bam', 'accession': '4DNFIJDN1FW4', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4577d3f7-9e2d-4a03-97de-61335539534b', 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIJXYARGN/@@download/4DNFIJXYARGN.bw', 'file_type': 'normalized counts', 'link_id': '~files-processed~4DNFIJXYARGN~', 'file_type_detailed': 'normalized counts (bw)', 'md5sum': '9b64beca516a292cbb53e81af1e6cd2f', 'file_format': 'bw', '@id': '/files-processed/4DNFIJXYARGN/', 'display_title': '4DNFIJXYARGN.bw', 'upload_key': '4577d3f7-9e2d-4a03-97de-61335539534b/4DNFIJXYARGN.bw', 'accession': '4DNFIJXYARGN', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b156a751-31bf-410a-86e2-fe4f897061e3', 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIPAJUJYN/@@download/4DNFIPAJUJYN.bed.gz', 'file_type': 'LADs', 'link_id': '~files-processed~4DNFIPAJUJYN~', 'file_type_detailed': 'LADs (bed)', 'md5sum': '728cd20493b62f967d92d98505af0404', 'file_format': 'bed', '@id': '/files-processed/4DNFIPAJUJYN/', 'display_title': '4DNFIPAJUJYN.bed.gz', 'upload_key': 'b156a751-31bf-410a-86e2-fe4f897061e3/4DNFIPAJUJYN.bed.gz', 'accession': '4DNFIPAJUJYN', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'files': [{'uuid': '9f4430d3-c70f-4f68-8b0a-57ab22476dc7', 'file_size': 1430934659, 'file_classification': 'raw file', 'status': 'released', 'href': '/files-fastq/4DNFINTHX4F9/@@download/4DNFINTHX4F9.fastq.gz', 'file_type': 'reads', 'link_id': '~files-fastq~4DNFINTHX4F9~', 'quality_metric': {'uuid': 'bd127d1b-fa3b-4687-8b31-2dd7e669d2f7', 'Sequence length': '65', '@id': '/quality-metrics-fastqc/bd127d1b-fa3b-4687-8b31-2dd7e669d2f7/', 'Total Sequences': 28228164, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}, 'link_id': '~quality-metrics-fastqc~bd127d1b-fa3b-4687-8b31-2dd7e669d2f7~', 'overall_quality_status': 'PASS', 'url': 'https://s3.amazonaws.com/elasticbeanstalk-fourfront-webprod-wfoutput/4DNFINTHX4F9/fastqc_report.html', 'display_title': 'QualityMetricFastqc from 2017-12-21'}, 'file_type_detailed': 'reads (fastq)', 'extra_files': [], 'md5sum': '5c16a31cc359a5725340477f815a7211', 'file_format': 'fastq', '@id': '/files-fastq/4DNFINTHX4F9/', 'display_title': '4DNFINTHX4F9.fastq.gz', 'upload_key': '9f4430d3-c70f-4f68-8b0a-57ab22476dc7/4DNFINTHX4F9.fastq.gz', 'accession': '4DNFINTHX4F9', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'biosample': {'uuid': 'd1da185c-ca4c-41d8-8bf6-1d0003e60455', 'modifications_summary': 'None', 'treatments_summary': 'None', 'biosource': [{'uuid': '68172441-97c4-40cc-b73f-d0f5dbc5cc05', '@id': '/biosources/4DNSRV3SKQ8M/', 'display_title': 'H1-hESC (Tier 1) - 4DNSRV3SKQ8M', 'individual': {'organism': {'uuid': '7745b647-ff15-4ff3-9ced-b897d4e2983c', '@id': '/organisms/human/', 'display_title': 'human', 'link_id': '~organisms~human~', 'name': 'human', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}}, 'biosource_type': 'stem cell derived cell line', 'link_id': '~biosources~4DNSRV3SKQ8M~', 'cell_line': {'uuid': 'c3f6e06f-900e-45f9-9c23-983c5673f58d', '@id': '/ontology-terms/EFO%3A0003042/', 'display_title': 'H1-hESC', 'link_id': '~ontology-terms~EFO%3A0003042~', 'synonyms': ['H1'], 'slim_terms': [{'uuid': '111189bc-8535-4448-903e-854af460a233', 'link_id': '~ontology-terms~EFO%3A0000322~', '@id': '/ontology-terms/EFO%3A0000322/', 'display_title': 'cell line', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'link_id': '~biosamples~4DNBS5PK6X9E~', 'biosource_summary': 'H1-hESC (Tier 1)', 'treatments': [], '@id': '/biosamples/4DNBS5PK6X9E/', 'display_title': '4DNBS5PK6X9E', 'modifications': [], 'accession': '4DNBS5PK6X9E', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, '@id': '/experiments-damid/4DNEXR716AG4/', 'display_title': 'DAM-ID seq on H1-hESC (Tier 1) - 4DNEXR716AG4', 'experiment_type': 'DAM-ID seq', 'accession': '4DNEXR716AG4', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, {'uuid': '3e8087c9-1311-484d-9cea-8dac908b0aee', 'other_processed_files': [{'files': [{'uuid': '5f33da65-0211-418c-bb5f-5510b1b2aab8', 'file_format': 'txt', '@id': '/files-processed/4DNFIVOUCMHL/', 'display_title': '4DNFIVOUCMHL.txt.gz', 'link_id': '~files-processed~4DNFIVOUCMHL~', 'accession': '4DNFIVOUCMHL', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1df0c71e-b8a3-49cf-9bcb-c342c0736270', 'file_format': 'bw', '@id': '/files-processed/4DNFIVG36P33/', 'display_title': '4DNFIVG36P33.bw', 'link_id': '~files-processed~4DNFIVG36P33~', 'accession': '4DNFIVG36P33', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '38d09d2b-5f25-4f05-acdb-555f46bb3bfa', 'file_format': 'txt', '@id': '/files-processed/4DNFI58PBSCQ/', 'display_title': '4DNFI58PBSCQ.txt.gz', 'link_id': '~files-processed~4DNFI58PBSCQ~', 'accession': '4DNFI58PBSCQ', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '69494da1-fa34-4aca-9ff4-639073e3e10b', 'file_format': 'bw', '@id': '/files-processed/4DNFIAYM5VOE/', 'display_title': '4DNFIAYM5VOE.bw', 'link_id': '~files-processed~4DNFIAYM5VOE~', 'accession': '4DNFIAYM5VOE', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'GATC binned files', 'description': 'The files associated with the GATC bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': 'c24ac0c8-e0ea-44c6-bbbd-6068c4501c1c', 'file_format': 'txt', '@id': '/files-processed/4DNFIR9UWPFG/', 'display_title': '4DNFIR9UWPFG.txt.gz', 'link_id': '~files-processed~4DNFIR9UWPFG~', 'accession': '4DNFIR9UWPFG', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '07c8be77-ab7d-4e6c-8fc4-85a346cf0ebb', 'file_format': 'bw', '@id': '/files-processed/4DNFIE6GZRGH/', 'display_title': '4DNFIE6GZRGH.bw', 'link_id': '~files-processed~4DNFIE6GZRGH~', 'accession': '4DNFIE6GZRGH', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b94e0001-19a4-4bd5-a383-9abc486c2734', 'file_format': 'txt', '@id': '/files-processed/4DNFIHNA2ZJQ/', 'display_title': '4DNFIHNA2ZJQ.txt.gz', 'link_id': '~files-processed~4DNFIHNA2ZJQ~', 'accession': '4DNFIHNA2ZJQ', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'bd881a96-d1ba-473a-8afb-ff59ae4dba9d', 'file_format': 'txt', '@id': '/files-processed/4DNFIXBZ7ORR/', 'display_title': '4DNFIXBZ7ORR.txt.gz', 'link_id': '~files-processed~4DNFIXBZ7ORR~', 'accession': '4DNFIXBZ7ORR', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Additional 1 kb binned files', 'description': 'The additional supplementary files associated with the 1 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '48b88e33-bc15-477a-8943-0b5fe65af623', 'file_format': 'txt', '@id': '/files-processed/4DNFIC55R4Y6/', 'display_title': '4DNFIC55R4Y6.txt.gz', 'link_id': '~files-processed~4DNFIC55R4Y6~', 'accession': '4DNFIC55R4Y6', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b1929671-8076-4e24-8db4-144e6fd81397', 'file_format': 'bw', '@id': '/files-processed/4DNFIE9C1N4J/', 'display_title': '4DNFIE9C1N4J.bw', 'link_id': '~files-processed~4DNFIE9C1N4J~', 'accession': '4DNFIE9C1N4J', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0f606c1c-bfcb-4b5d-a9e9-ee774080c5d3', 'file_format': 'txt', '@id': '/files-processed/4DNFIDPBQSUD/', 'display_title': '4DNFIDPBQSUD.txt.gz', 'link_id': '~files-processed~4DNFIDPBQSUD~', 'accession': '4DNFIDPBQSUD', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '950172af-87f1-44d2-aac4-22112322efac', 'file_format': 'bw', '@id': '/files-processed/4DNFIR91Y2OX/', 'display_title': '4DNFIR91Y2OX.bw', 'link_id': '~files-processed~4DNFIR91Y2OX~', 'accession': '4DNFIR91Y2OX', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'bb11d739-b081-4d09-b9cb-650f69edd911', 'file_format': 'txt', '@id': '/files-processed/4DNFI7YB6LJ9/', 'display_title': '4DNFI7YB6LJ9.txt.gz', 'link_id': '~files-processed~4DNFI7YB6LJ9~', 'accession': '4DNFI7YB6LJ9', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '6b08421f-f635-4a01-aac3-f0d2e6f900d8', 'file_format': 'bed', '@id': '/files-processed/4DNFII5HTUBS/', 'display_title': '4DNFII5HTUBS.bed.gz', 'link_id': '~files-processed~4DNFII5HTUBS~', 'accession': '4DNFII5HTUBS', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '2 kb binned files', 'description': 'The files associated with the 2 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '6301c7cc-5ae7-43d1-85db-610f7973e9bb', 'file_format': 'txt', '@id': '/files-processed/4DNFIQBTKCZ6/', 'display_title': '4DNFIQBTKCZ6.txt.gz', 'link_id': '~files-processed~4DNFIQBTKCZ6~', 'accession': '4DNFIQBTKCZ6', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a401eee2-3dcd-4512-a51d-b29069ef3ee3', 'file_format': 'bw', '@id': '/files-processed/4DNFIAYFRIL7/', 'display_title': '4DNFIAYFRIL7.bw', 'link_id': '~files-processed~4DNFIAYFRIL7~', 'accession': '4DNFIAYFRIL7', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2bc83039-aa06-4c96-9876-14ec2ad4e0c1', 'file_format': 'txt', '@id': '/files-processed/4DNFIOVU95S4/', 'display_title': '4DNFIOVU95S4.txt.gz', 'link_id': '~files-processed~4DNFIOVU95S4~', 'accession': '4DNFIOVU95S4', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4e331085-75b8-495b-ab08-23110ace1d30', 'file_format': 'bw', '@id': '/files-processed/4DNFIC4VHUGV/', 'display_title': '4DNFIC4VHUGV.bw', 'link_id': '~files-processed~4DNFIC4VHUGV~', 'accession': '4DNFIC4VHUGV', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '5efd514a-f072-4537-bbfc-0707ad127577', 'file_format': 'txt', '@id': '/files-processed/4DNFIONMDS8R/', 'display_title': '4DNFIONMDS8R.txt.gz', 'link_id': '~files-processed~4DNFIONMDS8R~', 'accession': '4DNFIONMDS8R', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'ce814cab-939a-4bdc-8d15-9019f50f23ef', 'file_format': 'bed', '@id': '/files-processed/4DNFIDDJTAX3/', 'display_title': '4DNFIDDJTAX3.bed.gz', 'link_id': '~files-processed~4DNFIDDJTAX3~', 'accession': '4DNFIDDJTAX3', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '5 kb binned files', 'description': 'The files associated with the 5 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': 'cc081549-cc1d-40df-80f2-665a0393fa2a', 'file_format': 'txt', '@id': '/files-processed/4DNFI7H8BTBM/', 'display_title': '4DNFI7H8BTBM.txt.gz', 'link_id': '~files-processed~4DNFI7H8BTBM~', 'accession': '4DNFI7H8BTBM', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'c12cd770-0871-4f15-88c4-69f571e2a829', 'file_format': 'bw', '@id': '/files-processed/4DNFIS1MVSAH/', 'display_title': '4DNFIS1MVSAH.bw', 'link_id': '~files-processed~4DNFIS1MVSAH~', 'accession': '4DNFIS1MVSAH', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2ad3484f-d2cc-4fd5-95a4-c1323fbb511f', 'file_format': 'txt', '@id': '/files-processed/4DNFIXO2ZRCS/', 'display_title': '4DNFIXO2ZRCS.txt.gz', 'link_id': '~files-processed~4DNFIXO2ZRCS~', 'accession': '4DNFIXO2ZRCS', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '9510e997-27a2-4260-861c-efca9efa542a', 'file_format': 'bw', '@id': '/files-processed/4DNFICGGIUSR/', 'display_title': '4DNFICGGIUSR.bw', 'link_id': '~files-processed~4DNFICGGIUSR~', 'accession': '4DNFICGGIUSR', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '9604c8eb-96f1-4008-b5cc-7055c1016bdc', 'file_format': 'txt', '@id': '/files-processed/4DNFIR5YDYSL/', 'display_title': '4DNFIR5YDYSL.txt.gz', 'link_id': '~files-processed~4DNFIR5YDYSL~', 'accession': '4DNFIR5YDYSL', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b159360b-8681-4baa-91c8-d495714424dd', 'file_format': 'bed', '@id': '/files-processed/4DNFI8EITI5T/', 'display_title': '4DNFI8EITI5T.bed.gz', 'link_id': '~files-processed~4DNFI8EITI5T~', 'accession': '4DNFI8EITI5T', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '10 kb binned files', 'description': 'The files associated with the 10 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '9fbad3fd-3c10-4c27-bbe6-0ec536f9e963', 'file_format': 'txt', '@id': '/files-processed/4DNFIA5AJGUG/', 'display_title': '4DNFIA5AJGUG.txt.gz', 'link_id': '~files-processed~4DNFIA5AJGUG~', 'accession': '4DNFIA5AJGUG', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '79112ab5-1aa5-4bca-a89d-3949fe4fb661', 'file_format': 'bw', '@id': '/files-processed/4DNFI1TVDL7B/', 'display_title': '4DNFI1TVDL7B.bw', 'link_id': '~files-processed~4DNFI1TVDL7B~', 'accession': '4DNFI1TVDL7B', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a4af5721-6432-4cce-9a50-c6f499a75202', 'file_format': 'txt', '@id': '/files-processed/4DNFI3387AHV/', 'display_title': '4DNFI3387AHV.txt.gz', 'link_id': '~files-processed~4DNFI3387AHV~', 'accession': '4DNFI3387AHV', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2c038815-bd38-47d4-bc00-333bd0696316', 'file_format': 'bw', '@id': '/files-processed/4DNFIACYDDSY/', 'display_title': '4DNFIACYDDSY.bw', 'link_id': '~files-processed~4DNFIACYDDSY~', 'accession': '4DNFIACYDDSY', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '5f53f149-f653-42b0-a903-d12647fd4af6', 'file_format': 'txt', '@id': '/files-processed/4DNFIQ5X4YOV/', 'display_title': '4DNFIQ5X4YOV.txt.gz', 'link_id': '~files-processed~4DNFIQ5X4YOV~', 'accession': '4DNFIQ5X4YOV', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '11fa57ea-95a2-4a59-ad69-5f4f70e6093c', 'file_format': 'bed', '@id': '/files-processed/4DNFIXBTEZVI/', 'display_title': '4DNFIXBTEZVI.bed.gz', 'link_id': '~files-processed~4DNFIXBTEZVI~', 'accession': '4DNFIXBTEZVI', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '20 kb binned files', 'description': 'The files associated with the 20 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '3e7d8342-c051-45d5-b04d-8b1bd62bdbf3', 'file_format': 'txt', '@id': '/files-processed/4DNFI213NR6I/', 'display_title': '4DNFI213NR6I.txt.gz', 'link_id': '~files-processed~4DNFI213NR6I~', 'accession': '4DNFI213NR6I', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4ce53fb2-987e-4164-9c33-029a55d08cbf', 'file_format': 'bw', '@id': '/files-processed/4DNFIJ4RB2IJ/', 'display_title': '4DNFIJ4RB2IJ.bw', 'link_id': '~files-processed~4DNFIJ4RB2IJ~', 'accession': '4DNFIJ4RB2IJ', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'be64985d-012d-433f-87b0-0efefec06a94', 'file_format': 'txt', '@id': '/files-processed/4DNFICUME31M/', 'display_title': '4DNFICUME31M.txt.gz', 'link_id': '~files-processed~4DNFICUME31M~', 'accession': '4DNFICUME31M', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '6d8aac76-93bf-4423-ab15-aae9c66d7ecc', 'file_format': 'bw', '@id': '/files-processed/4DNFIRUUWDHI/', 'display_title': '4DNFIRUUWDHI.bw', 'link_id': '~files-processed~4DNFIRUUWDHI~', 'accession': '4DNFIRUUWDHI', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'fd8e036e-9d3d-40e6-893b-2ec5095a9854', 'file_format': 'txt', '@id': '/files-processed/4DNFICJMUAIG/', 'display_title': '4DNFICJMUAIG.txt.gz', 'link_id': '~files-processed~4DNFICJMUAIG~', 'accession': '4DNFICJMUAIG', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7601d9a0-34de-477c-b7e5-6ac26f1a855f', 'file_format': 'bed', '@id': '/files-processed/4DNFIAWP1935/', 'display_title': '4DNFIAWP1935.bed.gz', 'link_id': '~files-processed~4DNFIAWP1935~', 'accession': '4DNFIAWP1935', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '25 kb binned files', 'description': 'The files associated with the 25 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': 'ce512508-ec09-4699-9384-55eac2c090e6', 'file_format': 'txt', '@id': '/files-processed/4DNFIYQQZTJP/', 'display_title': '4DNFIYQQZTJP.txt.gz', 'link_id': '~files-processed~4DNFIYQQZTJP~', 'accession': '4DNFIYQQZTJP', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '07f60950-c5e8-411e-b6b4-dfdf9526b53f', 'file_format': 'bw', '@id': '/files-processed/4DNFINE79TZ1/', 'display_title': '4DNFINE79TZ1.bw', 'link_id': '~files-processed~4DNFINE79TZ1~', 'accession': '4DNFINE79TZ1', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4b43218f-af8b-4395-a130-027be3cd82e0', 'file_format': 'txt', '@id': '/files-processed/4DNFI5S6G939/', 'display_title': '4DNFI5S6G939.txt.gz', 'link_id': '~files-processed~4DNFI5S6G939~', 'accession': '4DNFI5S6G939', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b09e0dc2-00f0-4546-8893-0007877cf741', 'file_format': 'bw', '@id': '/files-processed/4DNFI3QI958D/', 'display_title': '4DNFI3QI958D.bw', 'link_id': '~files-processed~4DNFI3QI958D~', 'accession': '4DNFI3QI958D', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4fe066da-7f61-4754-878d-5628f6585e03', 'file_format': 'txt', '@id': '/files-processed/4DNFITR9R8CE/', 'display_title': '4DNFITR9R8CE.txt.gz', 'link_id': '~files-processed~4DNFITR9R8CE~', 'accession': '4DNFITR9R8CE', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '66b5bd83-6b3b-4a19-b372-a4f88ff3b9ef', 'file_format': 'bed', '@id': '/files-processed/4DNFIQLOKMQK/', 'display_title': '4DNFIQLOKMQK.bed.gz', 'link_id': '~files-processed~4DNFIQLOKMQK~', 'accession': '4DNFIQLOKMQK', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '50 kb binned files', 'description': 'The files associated with the 50 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '65bcc2f7-ab2d-4439-a28e-0a753a609835', 'file_format': 'txt', '@id': '/files-processed/4DNFI69GYDMT/', 'display_title': '4DNFI69GYDMT.txt.gz', 'link_id': '~files-processed~4DNFI69GYDMT~', 'accession': '4DNFI69GYDMT', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7be9606e-7a12-4a0b-b204-8478b51873f9', 'file_format': 'bw', '@id': '/files-processed/4DNFIIQ7FJS1/', 'display_title': '4DNFIIQ7FJS1.bw', 'link_id': '~files-processed~4DNFIIQ7FJS1~', 'accession': '4DNFIIQ7FJS1', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '29b9f6a5-e31c-4c05-b1bb-31437866a679', 'file_format': 'txt', '@id': '/files-processed/4DNFI2BBO95U/', 'display_title': '4DNFI2BBO95U.txt.gz', 'link_id': '~files-processed~4DNFI2BBO95U~', 'accession': '4DNFI2BBO95U', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0b5cbce7-71dc-4e27-8246-f2b636102b85', 'file_format': 'bw', '@id': '/files-processed/4DNFIYRXZYI5/', 'display_title': '4DNFIYRXZYI5.bw', 'link_id': '~files-processed~4DNFIYRXZYI5~', 'accession': '4DNFIYRXZYI5', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '78a67c3e-6274-4f13-b998-7dd0d8ab9f5f', 'file_format': 'txt', '@id': '/files-processed/4DNFIJPBVWD3/', 'display_title': '4DNFIJPBVWD3.txt.gz', 'link_id': '~files-processed~4DNFIJPBVWD3~', 'accession': '4DNFIJPBVWD3', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1a95f073-c8ac-4b56-b11d-2c6a02cc7f30', 'file_format': 'bed', '@id': '/files-processed/4DNFII6JUGN4/', 'display_title': '4DNFII6JUGN4.bed.gz', 'link_id': '~files-processed~4DNFII6JUGN4~', 'accession': '4DNFII6JUGN4', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '80 kb binned files', 'description': 'The files associated with the 80 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '1f860ec8-24e0-484e-bb19-67e22360b632', 'file_format': 'txt', '@id': '/files-processed/4DNFIKZPVN2P/', 'display_title': '4DNFIKZPVN2P.txt.gz', 'link_id': '~files-processed~4DNFIKZPVN2P~', 'accession': '4DNFIKZPVN2P', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a81e847c-7016-41c2-abde-5d016c4afccf', 'file_format': 'bw', '@id': '/files-processed/4DNFIFZBCN5T/', 'display_title': '4DNFIFZBCN5T.bw', 'link_id': '~files-processed~4DNFIFZBCN5T~', 'accession': '4DNFIFZBCN5T', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a582c75c-ec7e-40ff-95c7-a0d510caf95d', 'file_format': 'txt', '@id': '/files-processed/4DNFI9TTH2Y9/', 'display_title': '4DNFI9TTH2Y9.txt.gz', 'link_id': '~files-processed~4DNFI9TTH2Y9~', 'accession': '4DNFI9TTH2Y9', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4efa4b0a-5587-4876-9d33-813eb1e7f491', 'file_format': 'bw', '@id': '/files-processed/4DNFII579RD8/', 'display_title': '4DNFII579RD8.bw', 'link_id': '~files-processed~4DNFII579RD8~', 'accession': '4DNFII579RD8', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '66526e51-0ce0-4417-acc7-1564dd20324b', 'file_format': 'txt', '@id': '/files-processed/4DNFI5INDI65/', 'display_title': '4DNFI5INDI65.txt.gz', 'link_id': '~files-processed~4DNFI5INDI65~', 'accession': '4DNFI5INDI65', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1fee3029-42fb-433e-bc35-c62a0b8d73b0', 'file_format': 'bed', '@id': '/files-processed/4DNFISWSSY2Q/', 'display_title': '4DNFISWSSY2Q.bed.gz', 'link_id': '~files-processed~4DNFISWSSY2Q~', 'accession': '4DNFISWSSY2Q', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '100 kb binned files', 'description': 'The files associated with the 100 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': 'b5457a2b-5e33-41ec-89f1-c37fd1686532', 'file_format': 'txt', '@id': '/files-processed/4DNFIB3282ZX/', 'display_title': '4DNFIB3282ZX.txt.gz', 'link_id': '~files-processed~4DNFIB3282ZX~', 'accession': '4DNFIB3282ZX', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '56bcd0d9-0f7a-42fc-b949-f6eba2a561a2', 'file_format': 'bw', '@id': '/files-processed/4DNFIEBKUA79/', 'display_title': '4DNFIEBKUA79.bw', 'link_id': '~files-processed~4DNFIEBKUA79~', 'accession': '4DNFIEBKUA79', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '00ff4c9e-31ef-478a-82e6-c43503bc4cff', 'file_format': 'txt', '@id': '/files-processed/4DNFI7QWRTIS/', 'display_title': '4DNFI7QWRTIS.txt.gz', 'link_id': '~files-processed~4DNFI7QWRTIS~', 'accession': '4DNFI7QWRTIS', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'd7fb317a-3e26-4607-a73c-cc23c511eb2d', 'file_format': 'bw', '@id': '/files-processed/4DNFIQSFCHML/', 'display_title': '4DNFIQSFCHML.bw', 'link_id': '~files-processed~4DNFIQSFCHML~', 'accession': '4DNFIQSFCHML', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'bf3d8c8b-fe3d-4c40-8d83-1897a4da5551', 'file_format': 'txt', '@id': '/files-processed/4DNFINYMJG8X/', 'display_title': '4DNFINYMJG8X.txt.gz', 'link_id': '~files-processed~4DNFINYMJG8X~', 'accession': '4DNFINYMJG8X', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a9802b54-3836-4cc1-a343-174b799bfe11', 'file_format': 'bed', '@id': '/files-processed/4DNFIA5CHLHD/', 'display_title': '4DNFIA5CHLHD.bed.gz', 'link_id': '~files-processed~4DNFIA5CHLHD~', 'accession': '4DNFIA5CHLHD', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '250 kb binned files', 'description': 'The files associated with the 250 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 2.'}, {'files': [{'uuid': '9cc0442d-9a51-4e7a-985a-aa345d0e0322', 'file_format': 'other', '@id': '/files-processed/4DNFI2SZX561/', 'display_title': '4DNFI2SZX561', 'link_id': '~files-processed~4DNFI2SZX561~', 'accession': '4DNFI2SZX561', 'file_type_detailed': 'other (other)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Supplementary files for Replicate 2', 'description': 'Non-bin specific supplementary files for Replicate 2'}], 'experiment_categorizer': {'combined': 'Target: Protein:Lamin-B1 (P20700)', 'field': 'Target', 'value': 'Protein:Lamin-B1 (P20700)'}, 'status': 'released', 'link_id': '~experiments-damid~4DNEXMST3CDE~', 'processed_files': [{'uuid': '6f9714dc-2138-41fe-915a-9d40902efa67', 'file_size': 974731506, 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFI7S5EK5P/@@download/4DNFI7S5EK5P.bam', 'file_type': 'alignment', 'link_id': '~files-processed~4DNFI7S5EK5P~', 'file_type_detailed': 'alignment (bam)', 'md5sum': '3af04baf1328c3eb67d25cb9a68063ee', 'file_format': 'bam', '@id': '/files-processed/4DNFI7S5EK5P/', 'display_title': '4DNFI7S5EK5P.bam', 'upload_key': '6f9714dc-2138-41fe-915a-9d40902efa67/4DNFI7S5EK5P.bam', 'accession': '4DNFI7S5EK5P', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0683b6d6-5fcb-4135-ae23-ea1cb9dc2d92', 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIJJRP4BA/@@download/4DNFIJJRP4BA.bw', 'file_type': 'normalized counts', 'link_id': '~files-processed~4DNFIJJRP4BA~', 'file_type_detailed': 'normalized counts (bw)', 'md5sum': '5b8d4be8d02c357b4c8c7252032906d8', 'file_format': 'bw', '@id': '/files-processed/4DNFIJJRP4BA/', 'display_title': '4DNFIJJRP4BA.bw', 'upload_key': '0683b6d6-5fcb-4135-ae23-ea1cb9dc2d92/4DNFIJJRP4BA.bw', 'accession': '4DNFIJJRP4BA', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '10e47432-8191-4908-b402-6e9e4837ace7', 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIZAKGGYF/@@download/4DNFIZAKGGYF.bed.gz', 'file_type': 'LADs', 'link_id': '~files-processed~4DNFIZAKGGYF~', 'file_type_detailed': 'LADs (bed)', 'md5sum': '8d0edf3627157b3dfec44f2977b7bb71', 'file_format': 'bed', '@id': '/files-processed/4DNFIZAKGGYF/', 'display_title': '4DNFIZAKGGYF.bed.gz', 'upload_key': '10e47432-8191-4908-b402-6e9e4837ace7/4DNFIZAKGGYF.bed.gz', 'accession': '4DNFIZAKGGYF', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'files': [{'uuid': '801429b7-a742-4c62-ad8f-5ea755656669', 'file_size': 1961555683, 'file_classification': 'raw file', 'status': 'released', 'href': '/files-fastq/4DNFIPD9M4HG/@@download/4DNFIPD9M4HG.fastq.gz', 'file_type': 'reads', 'link_id': '~files-fastq~4DNFIPD9M4HG~', 'quality_metric': {'uuid': '48d9dc72-4f60-4dba-b20f-6551f1624aaa', 'Sequence length': '65', '@id': '/quality-metrics-fastqc/48d9dc72-4f60-4dba-b20f-6551f1624aaa/', 'Total Sequences': 41171503, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}, 'link_id': '~quality-metrics-fastqc~48d9dc72-4f60-4dba-b20f-6551f1624aaa~', 'overall_quality_status': 'PASS', 'url': 'https://s3.amazonaws.com/elasticbeanstalk-fourfront-webprod-wfoutput/4DNFIPD9M4HG/fastqc_report.html', 'display_title': 'QualityMetricFastqc from 2017-12-21'}, 'file_type_detailed': 'reads (fastq)', 'extra_files': [], 'md5sum': '053db2e26062cbc6ba74a4bdc1a7dbb4', 'file_format': 'fastq', '@id': '/files-fastq/4DNFIPD9M4HG/', 'display_title': '4DNFIPD9M4HG.fastq.gz', 'upload_key': '801429b7-a742-4c62-ad8f-5ea755656669/4DNFIPD9M4HG.fastq.gz', 'accession': '4DNFIPD9M4HG', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'biosample': {'uuid': '15918da6-2476-4a01-a736-621e01e636b4', 'modifications_summary': 'None', 'treatments_summary': 'None', 'biosource': [{'uuid': '68172441-97c4-40cc-b73f-d0f5dbc5cc05', '@id': '/biosources/4DNSRV3SKQ8M/', 'display_title': 'H1-hESC (Tier 1) - 4DNSRV3SKQ8M', 'individual': {'organism': {'uuid': '7745b647-ff15-4ff3-9ced-b897d4e2983c', '@id': '/organisms/human/', 'display_title': 'human', 'link_id': '~organisms~human~', 'name': 'human', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}}, 'biosource_type': 'stem cell derived cell line', 'link_id': '~biosources~4DNSRV3SKQ8M~', 'cell_line': {'uuid': 'c3f6e06f-900e-45f9-9c23-983c5673f58d', '@id': '/ontology-terms/EFO%3A0003042/', 'display_title': 'H1-hESC', 'link_id': '~ontology-terms~EFO%3A0003042~', 'synonyms': ['H1'], 'slim_terms': [{'uuid': '111189bc-8535-4448-903e-854af460a233', 'link_id': '~ontology-terms~EFO%3A0000322~', '@id': '/ontology-terms/EFO%3A0000322/', 'display_title': 'cell line', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'link_id': '~biosamples~4DNBST7ZM51V~', 'biosource_summary': 'H1-hESC (Tier 1)', 'treatments': [], '@id': '/biosamples/4DNBST7ZM51V/', 'display_title': '4DNBST7ZM51V', 'modifications': [], 'accession': '4DNBST7ZM51V', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, '@id': '/experiments-damid/4DNEXMST3CDE/', 'display_title': 'DAM-ID seq on H1-hESC (Tier 1) - 4DNEXMST3CDE', 'experiment_type': 'DAM-ID seq', 'accession': '4DNEXMST3CDE', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, {'uuid': '804d8f54-07b7-4f82-8c2b-71539b508aca', 'other_processed_files': [{'files': [{'uuid': '1ddf2fb7-dd07-4364-9154-636ae293c68a', 'file_format': 'txt', '@id': '/files-processed/4DNFISPWYQW5/', 'display_title': '4DNFISPWYQW5.txt.gz', 'file_size': 50193966, 'link_id': '~files-processed~4DNFISPWYQW5~', 'accession': '4DNFISPWYQW5', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2b217c78-edeb-4e2a-98b0-8b9154f3e70f', 'file_format': 'bw', '@id': '/files-processed/4DNFINDQ2Y23/', 'display_title': '4DNFINDQ2Y23.bw', 'file_size': 39358957, 'link_id': '~files-processed~4DNFINDQ2Y23~', 'accession': '4DNFINDQ2Y23', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'f390e159-8582-475e-9702-d7608f976ad4', 'file_format': 'txt', '@id': '/files-processed/4DNFIX8PMO4T/', 'display_title': '4DNFIX8PMO4T.txt.gz', 'file_size': 58155094, 'link_id': '~files-processed~4DNFIX8PMO4T~', 'accession': '4DNFIX8PMO4T', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a85ceaa5-6348-4911-b88b-d40f64f48ecc', 'file_format': 'bw', '@id': '/files-processed/4DNFIHKLCGO5/', 'display_title': '4DNFIHKLCGO5.bw', 'file_size': 23568368, 'link_id': '~files-processed~4DNFIHKLCGO5~', 'accession': '4DNFIHKLCGO5', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'GATC binned files', 'description': 'The files associated with the GATC bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '4c7ec1c1-1693-49dd-b1c0-48a4b75a4e15', 'file_format': 'txt', '@id': '/files-processed/4DNFICYMXZKK/', 'display_title': '4DNFICYMXZKK.txt.gz', 'file_size': 16177664, 'link_id': '~files-processed~4DNFICYMXZKK~', 'accession': '4DNFICYMXZKK', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e6b6aa3b-c776-49a3-a4ec-3ef68a321203', 'file_format': 'bw', '@id': '/files-processed/4DNFIAJAIO4U/', 'display_title': '4DNFIAJAIO4U.bw', 'file_size': 18255575, 'link_id': '~files-processed~4DNFIAJAIO4U~', 'accession': '4DNFIAJAIO4U', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '54a69337-7ba4-4c78-8a80-c9d0675b4850', 'file_format': 'txt', '@id': '/files-processed/4DNFI7FQ2WFM/', 'display_title': '4DNFI7FQ2WFM.txt.gz', 'file_size': 23842739, 'link_id': '~files-processed~4DNFI7FQ2WFM~', 'accession': '4DNFI7FQ2WFM', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'c6bd7f1f-a03b-4394-84e3-a740c43b188a', 'file_format': 'txt', '@id': '/files-processed/4DNFITVPG2CT/', 'display_title': '4DNFITVPG2CT.txt.gz', 'file_size': 14870667, 'link_id': '~files-processed~4DNFITVPG2CT~', 'accession': '4DNFITVPG2CT', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Additional 1 kb binned files', 'description': 'The additional supplementary files associated with the 1 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '2ae11d64-4e02-40f8-b448-a396dcabb856', 'file_format': 'txt', '@id': '/files-processed/4DNFIZYO2F3S/', 'display_title': '4DNFIZYO2F3S.txt.gz', 'file_size': 8241599, 'link_id': '~files-processed~4DNFIZYO2F3S~', 'accession': '4DNFIZYO2F3S', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a7a90458-4243-4cf8-bef0-82c3f7d6ab41', 'file_format': 'bw', '@id': '/files-processed/4DNFIC45DH5V/', 'display_title': '4DNFIC45DH5V.bw', 'file_size': 9705438, 'link_id': '~files-processed~4DNFIC45DH5V~', 'accession': '4DNFIC45DH5V', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '44ea12d1-7346-4244-a6b8-06549f1fa181', 'file_format': 'txt', '@id': '/files-processed/4DNFIT5W8PB6/', 'display_title': '4DNFIT5W8PB6.txt.gz', 'file_size': 14605674, 'link_id': '~files-processed~4DNFIT5W8PB6~', 'accession': '4DNFIT5W8PB6', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0e329024-c611-4537-b26d-d26519f94fe7', 'file_format': 'bw', '@id': '/files-processed/4DNFIZ6BDHOY/', 'display_title': '4DNFIZ6BDHOY.bw', 'file_size': 10460561, 'link_id': '~files-processed~4DNFIZ6BDHOY~', 'accession': '4DNFIZ6BDHOY', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '7013a367-924f-4002-8682-bfc9a1ab0904', 'file_format': 'txt', '@id': '/files-processed/4DNFIFXQEOL3/', 'display_title': '4DNFIFXQEOL3.txt.gz', 'file_size': 7191239, 'link_id': '~files-processed~4DNFIFXQEOL3~', 'accession': '4DNFIFXQEOL3', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '90beeed8-e401-4758-a543-f8dc2433a203', 'file_format': 'bed', '@id': '/files-processed/4DNFIINBOET5/', 'display_title': '4DNFIINBOET5.bed.gz', 'file_size': 14217, 'link_id': '~files-processed~4DNFIINBOET5~', 'accession': '4DNFIINBOET5', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '2 kb binned files', 'description': 'The files associated with the 2 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '0429b923-dc29-4514-a1a2-2d6d1d76d973', 'file_format': 'txt', '@id': '/files-processed/4DNFIB69P9AH/', 'display_title': '4DNFIB69P9AH.txt.gz', 'file_size': 3407842, 'link_id': '~files-processed~4DNFIB69P9AH~', 'accession': '4DNFIB69P9AH', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '5704b3a9-f889-48cc-b19d-64ab5b0564b1', 'file_format': 'bw', '@id': '/files-processed/4DNFIKO9C3LW/', 'display_title': '4DNFIKO9C3LW.bw', 'file_size': 4348236, 'link_id': '~files-processed~4DNFIKO9C3LW~', 'accession': '4DNFIKO9C3LW', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b352a9e0-30b2-4837-a2e8-5a404ddb7d20', 'file_format': 'txt', '@id': '/files-processed/4DNFI64P1ELO/', 'display_title': '4DNFI64P1ELO.txt.gz', 'file_size': 7168885, 'link_id': '~files-processed~4DNFI64P1ELO~', 'accession': '4DNFI64P1ELO', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '81324f54-3127-4d68-8c7a-997c1727e0a9', 'file_format': 'bw', '@id': '/files-processed/4DNFIDZD43NK/', 'display_title': '4DNFIDZD43NK.bw', 'file_size': 5286189, 'link_id': '~files-processed~4DNFIDZD43NK~', 'accession': '4DNFIDZD43NK', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '5343effb-62ca-4f49-98ba-28747670cb54', 'file_format': 'txt', '@id': '/files-processed/4DNFI9OCV3Y7/', 'display_title': '4DNFI9OCV3Y7.txt.gz', 'file_size': 2737056, 'link_id': '~files-processed~4DNFI9OCV3Y7~', 'accession': '4DNFI9OCV3Y7', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '284d5402-0107-44e4-bc2b-1aeb578eff84', 'file_format': 'bed', '@id': '/files-processed/4DNFIM2X94N9/', 'display_title': '4DNFIM2X94N9.bed.gz', 'file_size': 12811, 'link_id': '~files-processed~4DNFIM2X94N9~', 'accession': '4DNFIM2X94N9', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '5 kb binned files', 'description': 'The files associated with the 5 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '839a4264-6499-4f11-b349-bdc93236cb28', 'file_format': 'txt', '@id': '/files-processed/4DNFIVD3TAQ4/', 'display_title': '4DNFIVD3TAQ4.txt.gz', 'file_size': 1763524, 'link_id': '~files-processed~4DNFIVD3TAQ4~', 'accession': '4DNFIVD3TAQ4', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '68b0ff45-92c3-4aad-8887-3606d9c9d21d', 'file_format': 'bw', '@id': '/files-processed/4DNFIYYYA82V/', 'display_title': '4DNFIYYYA82V.bw', 'file_size': 2346229, 'link_id': '~files-processed~4DNFIYYYA82V~', 'accession': '4DNFIYYYA82V', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b69dfc34-20ec-4a08-9975-2afaa59988b1', 'file_format': 'txt', '@id': '/files-processed/4DNFIVL5G1EK/', 'display_title': '4DNFIVL5G1EK.txt.gz', 'file_size': 3929106, 'link_id': '~files-processed~4DNFIVL5G1EK~', 'accession': '4DNFIVL5G1EK', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1e06fd23-07e1-4200-94c2-952bbcb57a91', 'file_format': 'bw', '@id': '/files-processed/4DNFI2RAXA9M/', 'display_title': '4DNFI2RAXA9M.bw', 'file_size': 2826754, 'link_id': '~files-processed~4DNFI2RAXA9M~', 'accession': '4DNFI2RAXA9M', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '9bb81b4e-1f0c-45d8-bc06-15f3b8fd6de8', 'file_format': 'txt', '@id': '/files-processed/4DNFIL24S5JE/', 'display_title': '4DNFIL24S5JE.txt.gz', 'file_size': 1338856, 'link_id': '~files-processed~4DNFIL24S5JE~', 'accession': '4DNFIL24S5JE', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '24482a30-3f46-4e0d-911e-c76beaa1e19f', 'file_format': 'bed', '@id': '/files-processed/4DNFICM8AM4P/', 'display_title': '4DNFICM8AM4P.bed.gz', 'file_size': 7191, 'link_id': '~files-processed~4DNFICM8AM4P~', 'accession': '4DNFICM8AM4P', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '10 kb binned files', 'description': 'The files associated with the 10 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '01bc8c8c-083b-4119-bfde-e3272dfe9e5b', 'file_format': 'txt', '@id': '/files-processed/4DNFI9DTN8M3/', 'display_title': '4DNFI9DTN8M3.txt.gz', 'file_size': 888522, 'link_id': '~files-processed~4DNFI9DTN8M3~', 'accession': '4DNFI9DTN8M3', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '972c0f31-244f-40ac-9961-308a22babe35', 'file_format': 'bw', '@id': '/files-processed/4DNFI5TKZM2L/', 'display_title': '4DNFI5TKZM2L.bw', 'file_size': 1248438, 'link_id': '~files-processed~4DNFI5TKZM2L~', 'accession': '4DNFI5TKZM2L', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '6e06c80b-23ef-4957-85f7-b94ba0420984', 'file_format': 'txt', '@id': '/files-processed/4DNFIYEZEWNW/', 'display_title': '4DNFIYEZEWNW.txt.gz', 'file_size': 2012434, 'link_id': '~files-processed~4DNFIYEZEWNW~', 'accession': '4DNFIYEZEWNW', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '0ddedcbe-5167-4b3b-a223-42d6bce3127b', 'file_format': 'bw', '@id': '/files-processed/4DNFIYX7OVMK/', 'display_title': '4DNFIYX7OVMK.bw', 'file_size': 2225343, 'link_id': '~files-processed~4DNFIYX7OVMK~', 'accession': '4DNFIYX7OVMK', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'facad216-daaa-4df7-a322-f15c2f0eaf87', 'file_format': 'txt', '@id': '/files-processed/4DNFI4LK3AJJ/', 'display_title': '4DNFI4LK3AJJ.txt.gz', 'file_size': 649640, 'link_id': '~files-processed~4DNFI4LK3AJJ~', 'accession': '4DNFI4LK3AJJ', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '2eab3194-c7ba-40ff-8c5e-206c365045d2', 'file_format': 'bed', '@id': '/files-processed/4DNFIB4GJ5WT/', 'display_title': '4DNFIB4GJ5WT.bed.gz', 'file_size': 4900, 'link_id': '~files-processed~4DNFIB4GJ5WT~', 'accession': '4DNFIB4GJ5WT', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '20 kb binned files', 'description': 'The files associated with the 20 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': 'c76580e8-031c-4864-8d7a-7fd65d4ca7a9', 'file_format': 'txt', '@id': '/files-processed/4DNFICRWGW37/', 'display_title': '4DNFICRWGW37.txt.gz', 'file_size': 714520, 'link_id': '~files-processed~4DNFICRWGW37~', 'accession': '4DNFICRWGW37', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b18568b9-8d84-45d3-9fcd-e5f5c59b8c76', 'file_format': 'bw', '@id': '/files-processed/4DNFIEGCQUE3/', 'display_title': '4DNFIEGCQUE3.bw', 'file_size': 1044975, 'link_id': '~files-processed~4DNFIEGCQUE3~', 'accession': '4DNFIEGCQUE3', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'f0a8e914-c8cd-46a6-988c-757c18753805', 'file_format': 'txt', '@id': '/files-processed/4DNFIGIUBW7Y/', 'display_title': '4DNFIGIUBW7Y.txt.gz', 'file_size': 1615220, 'link_id': '~files-processed~4DNFIGIUBW7Y~', 'accession': '4DNFIGIUBW7Y', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'b9280ca8-5a9a-458d-b4fd-157b7a099f99', 'file_format': 'bw', '@id': '/files-processed/4DNFI2JABSZA/', 'display_title': '4DNFI2JABSZA.bw', 'file_size': 1811083, 'link_id': '~files-processed~4DNFI2JABSZA~', 'accession': '4DNFI2JABSZA', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '5f1719ce-8e36-4d79-81e6-cb370739d0d5', 'file_format': 'txt', '@id': '/files-processed/4DNFIL7ZA137/', 'display_title': '4DNFIL7ZA137.txt.gz', 'file_size': 516439, 'link_id': '~files-processed~4DNFIL7ZA137~', 'accession': '4DNFIL7ZA137', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'e7e200c6-dcae-4aa4-b0d6-08888eb8697e', 'file_format': 'bed', '@id': '/files-processed/4DNFIKWZ1RZN/', 'display_title': '4DNFIKWZ1RZN.bed.gz', 'file_size': 4376, 'link_id': '~files-processed~4DNFIKWZ1RZN~', 'accession': '4DNFIKWZ1RZN', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '25 kb binned files', 'description': 'The files associated with the 25 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '0ee90d06-3a98-45e6-ba37-0ba6214a878b', 'file_format': 'txt', '@id': '/files-processed/4DNFIY51UCAH/', 'display_title': '4DNFIY51UCAH.txt.gz', 'file_size': 357773, 'link_id': '~files-processed~4DNFIY51UCAH~', 'accession': '4DNFIY51UCAH', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '3c35fbcd-2c00-4c12-a75a-d1f340177739', 'file_format': 'bw', '@id': '/files-processed/4DNFIZPPT9JO/', 'display_title': '4DNFIZPPT9JO.bw', 'file_size': 568712, 'link_id': '~files-processed~4DNFIZPPT9JO~', 'accession': '4DNFIZPPT9JO', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1f930c17-3b84-4c7b-ac44-b484ffdc19d2', 'file_format': 'txt', '@id': '/files-processed/4DNFIGIUVEV5/', 'display_title': '4DNFIGIUVEV5.txt.gz', 'file_size': 809505, 'link_id': '~files-processed~4DNFIGIUVEV5~', 'accession': '4DNFIGIUVEV5', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '54f877c3-9e06-4d4a-93ed-5e3d5d0ddcfc', 'file_format': 'bw', '@id': '/files-processed/4DNFIIX2JM55/', 'display_title': '4DNFIIX2JM55.bw', 'file_size': 939338, 'link_id': '~files-processed~4DNFIIX2JM55~', 'accession': '4DNFIIX2JM55', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'ba2cbef0-7d30-44b3-84c0-168561062142', 'file_format': 'txt', '@id': '/files-processed/4DNFIH6YDWKT/', 'display_title': '4DNFIH6YDWKT.txt.gz', 'file_size': 251618, 'link_id': '~files-processed~4DNFIH6YDWKT~', 'accession': '4DNFIH6YDWKT', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'f1ae83c7-a426-47b2-bb8c-3f98d9ba1f10', 'file_format': 'bed', '@id': '/files-processed/4DNFIBMOKXD8/', 'display_title': '4DNFIBMOKXD8.bed.gz', 'file_size': 3293, 'link_id': '~files-processed~4DNFIBMOKXD8~', 'accession': '4DNFIBMOKXD8', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '50 kb binned files', 'description': 'The files associated with the 50 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '9fb452a2-b24d-4f5e-914c-80b73e7653ff', 'file_format': 'txt', '@id': '/files-processed/4DNFIVR14KSQ/', 'display_title': '4DNFIVR14KSQ.txt.gz', 'file_size': 228264, 'link_id': '~files-processed~4DNFIVR14KSQ~', 'accession': '4DNFIVR14KSQ', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '97f3f525-30ab-4ec9-b31b-6ef008112ab7', 'file_format': 'bw', '@id': '/files-processed/4DNFIP6SWTYG/', 'display_title': '4DNFIP6SWTYG.bw', 'file_size': 369172, 'link_id': '~files-processed~4DNFIP6SWTYG~', 'accession': '4DNFIP6SWTYG', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'c9f10528-9818-487e-9efc-9685b0998569', 'file_format': 'txt', '@id': '/files-processed/4DNFIPG3IWQH/', 'display_title': '4DNFIPG3IWQH.txt.gz', 'file_size': 511946, 'link_id': '~files-processed~4DNFIPG3IWQH~', 'accession': '4DNFIPG3IWQH', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '6bf42006-2220-4c14-bf70-ac6c3f94b00b', 'file_format': 'bw', '@id': '/files-processed/4DNFINGGAQP3/', 'display_title': '4DNFINGGAQP3.bw', 'file_size': 598763, 'link_id': '~files-processed~4DNFINGGAQP3~', 'accession': '4DNFINGGAQP3', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'eacff5a1-bf5a-4fe8-8ec9-6e175a40a9fd', 'file_format': 'txt', '@id': '/files-processed/4DNFI3UJWOX9/', 'display_title': '4DNFI3UJWOX9.txt.gz', 'file_size': 163847, 'link_id': '~files-processed~4DNFI3UJWOX9~', 'accession': '4DNFI3UJWOX9', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '904455a5-d8cd-42ee-8c61-a91acd503a8e', 'file_format': 'bed', '@id': '/files-processed/4DNFIP2ZNNAB/', 'display_title': '4DNFIP2ZNNAB.bed.gz', 'file_size': 2789, 'link_id': '~files-processed~4DNFIP2ZNNAB~', 'accession': '4DNFIP2ZNNAB', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '80 kb binned files', 'description': 'The files associated with the 80 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '90ab23f5-8db0-453d-b958-e56daf8a173b', 'file_format': 'txt', '@id': '/files-processed/4DNFIZQTKU2T/', 'display_title': '4DNFIZQTKU2T.txt.gz', 'file_size': 180547, 'link_id': '~files-processed~4DNFIZQTKU2T~', 'accession': '4DNFIZQTKU2T', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '483695a1-b525-401a-beb3-2decbe1884a7', 'file_format': 'bw', '@id': '/files-processed/4DNFICAH5QBS/', 'display_title': '4DNFICAH5QBS.bw', 'file_size': 311708, 'link_id': '~files-processed~4DNFICAH5QBS~', 'accession': '4DNFICAH5QBS', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'd949d6bb-b98b-41e4-a3b5-fb1767d37db4', 'file_format': 'txt', '@id': '/files-processed/4DNFIAHUMZUC/', 'display_title': '4DNFIAHUMZUC.txt.gz', 'file_size': 406626, 'link_id': '~files-processed~4DNFIAHUMZUC~', 'accession': '4DNFIAHUMZUC', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4bce48c8-23ce-45ef-8dc0-80225c8e973e', 'file_format': 'bw', '@id': '/files-processed/4DNFIDBC1TSC/', 'display_title': '4DNFIDBC1TSC.bw', 'file_size': 490504, 'link_id': '~files-processed~4DNFIDBC1TSC~', 'accession': '4DNFIDBC1TSC', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '00e22704-f080-4073-91e7-f04c0c2acd7c', 'file_format': 'txt', '@id': '/files-processed/4DNFI8MOLU6H/', 'display_title': '4DNFI8MOLU6H.txt.gz', 'file_size': 127923, 'link_id': '~files-processed~4DNFI8MOLU6H~', 'accession': '4DNFI8MOLU6H', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '1e73764c-81f6-460f-96b5-a2771601fad5', 'file_format': 'bed', '@id': '/files-processed/4DNFIVSR6NQW/', 'display_title': '4DNFIVSR6NQW.bed.gz', 'file_size': 2394, 'link_id': '~files-processed~4DNFIVSR6NQW~', 'accession': '4DNFIVSR6NQW', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '100 kb binned files', 'description': 'The files associated with the 100 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': '643007f9-e7bc-4a2d-ba49-1fd0ba484b3d', 'file_format': 'txt', '@id': '/files-processed/4DNFIDRO7H9W/', 'display_title': '4DNFIDRO7H9W.txt.gz', 'file_size': 69189, 'link_id': '~files-processed~4DNFIDRO7H9W~', 'accession': '4DNFIDRO7H9W', 'file_type_detailed': 'counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'ce462d76-a2ae-4c94-b281-4c8c797a9d26', 'file_format': 'bw', '@id': '/files-processed/4DNFIFKOTUFH/', 'display_title': '4DNFIFKOTUFH.bw', 'file_size': 149651, 'link_id': '~files-processed~4DNFIFKOTUFH~', 'accession': '4DNFIFKOTUFH', 'file_type_detailed': 'counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '4c0dd361-e0e1-46dd-bfa0-e82344bd3959', 'file_format': 'txt', '@id': '/files-processed/4DNFIMFL6F8F/', 'display_title': '4DNFIMFL6F8F.txt.gz', 'file_size': 158618, 'link_id': '~files-processed~4DNFIMFL6F8F~', 'accession': '4DNFIMFL6F8F', 'file_type_detailed': 'normalized counts (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '8dc994e3-2aa8-4caa-be8a-20d413e6a518', 'file_format': 'bw', '@id': '/files-processed/4DNFI9ZMQ1N1/', 'display_title': '4DNFI9ZMQ1N1.bw', 'file_size': 220005, 'link_id': '~files-processed~4DNFI9ZMQ1N1~', 'accession': '4DNFI9ZMQ1N1', 'file_type_detailed': 'normalized counts (bw)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '576cd077-0010-464a-ba15-70546e725531', 'file_format': 'txt', '@id': '/files-processed/4DNFINS9ZNM4/', 'display_title': '4DNFINS9ZNM4.txt.gz', 'file_size': 47099, 'link_id': '~files-processed~4DNFINS9ZNM4~', 'accession': '4DNFINS9ZNM4', 'file_type_detailed': 'LADs (txt)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '04b2c874-95df-48e9-a072-cf383cd4e0a5', 'file_format': 'bed', '@id': '/files-processed/4DNFITFOH1LU/', 'display_title': '4DNFITFOH1LU.bed.gz', 'file_size': 1682, 'link_id': '~files-processed~4DNFITFOH1LU~', 'accession': '4DNFITFOH1LU', 'file_type_detailed': 'LADs (bed)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': '250 kb binned files', 'description': 'The files associated with the 250 kb bin size processing of DAM ID seq data for the H1 LMNB1 Replicate 3.'}, {'files': [{'uuid': 'a01ec5fc-5016-43d1-a3fa-f435146aab02', 'file_format': 'other', '@id': '/files-processed/4DNFIJZXXMHV/', 'display_title': '4DNFIJZXXMHV', 'link_id': '~files-processed~4DNFIJZXXMHV~', 'accession': '4DNFIJZXXMHV', 'file_type_detailed': 'other (other)', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'type': 'preliminary', 'title': 'Supplementary files for Replicate 3', 'description': 'Non-bin specific supplementary files for Replicate 3'}], 'experiment_categorizer': {'combined': 'Target: Protein:Lamin-B1 (P20700)', 'field': 'Target', 'value': 'Protein:Lamin-B1 (P20700)'}, 'status': 'released', 'link_id': '~experiments-damid~4DNEXTTYNWII~', 'processed_files': [{'uuid': '0b7cb7d2-dd17-4d2c-a80a-d2b6228abce8', 'file_size': 936694373, 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIE81WDNP/@@download/4DNFIE81WDNP.bam', 'file_type': 'alignment', 'link_id': '~files-processed~4DNFIE81WDNP~', 'file_type_detailed': 'alignment (bam)', 'md5sum': '6fcefac0e0422fb25f4f94a495c843b1', 'file_format': 'bam', '@id': '/files-processed/4DNFIE81WDNP/', 'display_title': '4DNFIE81WDNP.bam', 'upload_key': '0b7cb7d2-dd17-4d2c-a80a-d2b6228abce8/4DNFIE81WDNP.bam', 'accession': '4DNFIE81WDNP', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'a31f8fe3-01be-4fae-9ca3-9fc8de4b7193', 'file_size': 15647847, 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIL4D7FZY/@@download/4DNFIL4D7FZY.bw', 'file_type': 'normalized counts', 'link_id': '~files-processed~4DNFIL4D7FZY~', 'file_type_detailed': 'normalized counts (bw)', 'md5sum': '8457b61068f40370869fb69df95ed3ff', 'file_format': 'bw', '@id': '/files-processed/4DNFIL4D7FZY/', 'display_title': '4DNFIL4D7FZY.bw', 'upload_key': 'a31f8fe3-01be-4fae-9ca3-9fc8de4b7193/4DNFIL4D7FZY.bw', 'accession': '4DNFIL4D7FZY', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': '76450ea8-d15c-4572-ace8-a7ea947de5a8', 'file_size': 17750, 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIAH8A2RJ/@@download/4DNFIAH8A2RJ.bed.gz', 'file_type': 'LADs', 'link_id': '~files-processed~4DNFIAH8A2RJ~', 'file_type_detailed': 'LADs (bed)', 'md5sum': 'c04ecbfad5a162cf01c90a1ca9c7d421', 'file_format': 'bed', '@id': '/files-processed/4DNFIAH8A2RJ/', 'display_title': '4DNFIAH8A2RJ.bed.gz', 'upload_key': '76450ea8-d15c-4572-ace8-a7ea947de5a8/4DNFIAH8A2RJ.bed.gz', 'accession': '4DNFIAH8A2RJ', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'files': [{'uuid': '798b8174-fcfc-4acd-8cdc-8a40e2b84cab', 'file_size': 1987923040, 'file_classification': 'raw file', 'status': 'released', 'href': '/files-fastq/4DNFIEE9OW9A/@@download/4DNFIEE9OW9A.fastq.gz', 'file_type': 'reads', 'link_id': '~files-fastq~4DNFIEE9OW9A~', 'quality_metric': {'uuid': '40f4120e-7c12-4a25-9b2f-efc28b09e63b', 'Sequence length': '65', '@id': '/quality-metrics-fastqc/40f4120e-7c12-4a25-9b2f-efc28b09e63b/', 'Total Sequences': 41839435, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}, 'link_id': '~quality-metrics-fastqc~40f4120e-7c12-4a25-9b2f-efc28b09e63b~', 'overall_quality_status': 'PASS', 'url': 'https://s3.amazonaws.com/elasticbeanstalk-fourfront-webprod-wfoutput/4DNFIEE9OW9A/fastqc_report.html', 'display_title': 'QualityMetricFastqc from 2017-12-21'}, 'file_type_detailed': 'reads (fastq)', 'extra_files': [], 'md5sum': '509ea184e569e2f46137bc8b56dfa3cf', 'file_format': 'fastq', '@id': '/files-fastq/4DNFIEE9OW9A/', 'display_title': '4DNFIEE9OW9A.fastq.gz', 'upload_key': '798b8174-fcfc-4acd-8cdc-8a40e2b84cab/4DNFIEE9OW9A.fastq.gz', 'accession': '4DNFIEE9OW9A', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'biosample': {'uuid': '3dcc1971-9c0f-42d6-933f-e4770147b827', 'modifications_summary': 'None', 'treatments_summary': 'None', 'biosource': [{'uuid': '68172441-97c4-40cc-b73f-d0f5dbc5cc05', '@id': '/biosources/4DNSRV3SKQ8M/', 'display_title': 'H1-hESC (Tier 1) - 4DNSRV3SKQ8M', 'individual': {'organism': {'uuid': '7745b647-ff15-4ff3-9ced-b897d4e2983c', '@id': '/organisms/human/', 'display_title': 'human', 'link_id': '~organisms~human~', 'name': 'human', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}}, 'biosource_type': 'stem cell derived cell line', 'link_id': '~biosources~4DNSRV3SKQ8M~', 'cell_line': {'uuid': 'c3f6e06f-900e-45f9-9c23-983c5673f58d', '@id': '/ontology-terms/EFO%3A0003042/', 'display_title': 'H1-hESC', 'link_id': '~ontology-terms~EFO%3A0003042~', 'synonyms': ['H1'], 'slim_terms': [{'uuid': '111189bc-8535-4448-903e-854af460a233', 'link_id': '~ontology-terms~EFO%3A0000322~', '@id': '/ontology-terms/EFO%3A0000322/', 'display_title': 'cell line', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'link_id': '~biosamples~4DNBSW44B6J2~', 'biosource_summary': 'H1-hESC (Tier 1)', 'treatments': [], '@id': '/biosamples/4DNBSW44B6J2/', 'display_title': '4DNBSW44B6J2', 'modifications': [], 'accession': '4DNBSW44B6J2', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, '@id': '/experiments-damid/4DNEXTTYNWII/', 'display_title': 'DAM-ID seq on H1-hESC (Tier 1) - 4DNEXTTYNWII', 'experiment_type': 'DAM-ID seq', 'accession': '4DNEXTTYNWII', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}], 'processed_files': [{'uuid': '85490436-b099-4760-9d4e-621e2dea4572', 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFIOCIY5BQ/@@download/4DNFIOCIY5BQ.bed.gz', 'file_type': 'LADs', 'link_id': '~files-processed~4DNFIOCIY5BQ~', 'file_type_detailed': 'LADs (bed)', 'md5sum': '1bf3500efafbc0a37f68c12a3b40dfcf', 'file_format': 'bed', '@id': '/files-processed/4DNFIOCIY5BQ/', 'display_title': '4DNFIOCIY5BQ.bed.gz', 'upload_key': '85490436-b099-4760-9d4e-621e2dea4572/4DNFIOCIY5BQ.bed.gz', 'accession': '4DNFIOCIY5BQ', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}, {'uuid': 'dceba897-c946-40b8-bb64-8776cf0d72d9', 'file_classification': 'processed file', 'status': 'uploaded', 'href': '/files-processed/4DNFI6HQSEU1/@@download/4DNFI6HQSEU1.bw', 'file_type': 'normalized counts', 'link_id': '~files-processed~4DNFI6HQSEU1~', 'file_type_detailed': 'normalized counts (bw)', 'md5sum': '889ea00ba5267f3349326b8ad2b09804', 'file_format': 'bw', '@id': '/files-processed/4DNFI6HQSEU1/', 'display_title': '4DNFI6HQSEU1.bw', 'upload_key': 'dceba897-c946-40b8-bb64-8776cf0d72d9/4DNFI6HQSEU1.bw', 'accession': '4DNFI6HQSEU1', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['award.91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'group.admin', 'group.read-only-admin', 'lab.6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'remoteuser.EMBED', 'remoteuser.INDEXER']}}], 'dbxrefs': [], 'accession': '4DNESXKBPZKQ', 'uuid': 'eae5ee86-b6ee-4112-941b-311d6026acb7', 'tags': ['4DN Joint Analysis 2018', '2018_01_public', '2018_01_internal', '2018_01_internal_JA'], 'replicate_exps': [{'bio_rep_no': 1, 'tec_rep_no': 1, 'replicate_exp': {'uuid': '22351015-90bb-4121-bc1f-5b5d99c65769', '@id': '/experiments-damid/4DNEXR716AG4/', 'display_title': 'DAM-ID seq on H1-hESC (Tier 1) - 4DNEXR716AG4', 'link_id': '~experiments-damid~4DNEXR716AG4~', 'accession': '4DNEXR716AG4', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}}, {'bio_rep_no': 2, 'tec_rep_no': 1, 'replicate_exp': {'uuid': '3e8087c9-1311-484d-9cea-8dac908b0aee', '@id': '/experiments-damid/4DNEXMST3CDE/', 'display_title': 'DAM-ID seq on H1-hESC (Tier 1) - 4DNEXMST3CDE', 'link_id': '~experiments-damid~4DNEXMST3CDE~', 'accession': '4DNEXMST3CDE', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}}, {'bio_rep_no': 3, 'tec_rep_no': 1, 'replicate_exp': {'uuid': '804d8f54-07b7-4f82-8c2b-71539b508aca', '@id': '/experiments-damid/4DNEXTTYNWII/', 'display_title': 'DAM-ID seq on H1-hESC (Tier 1) - 4DNEXTTYNWII', 'link_id': '~experiments-damid~4DNEXTTYNWII~', 'accession': '4DNEXTTYNWII', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}}], 'award': {'uuid': '91b694c3-f4d7-4ddd-8278-16f94e15c1c5', '@id': '/awards/1U54DK107965-01/', 'display_title': 'COMBINED CYTOLOGICAL, GENOMIC, AND FUNCTIONAL MAPPING OF NUCLEAR GENOME ORGANIZATION', 'link_id': '~awards~1U54DK107965-01~', 'center_title': 'NOFIC - Belmont', 'project': '4DN', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}, 'status': 'released', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}, 'publications_of_set': [], 'link_id': '~experiment-set-replicates~4DNESXKBPZKQ~', 'external_references': [], '@id': '/experiment-set-replicates/4DNESXKBPZKQ/', 'number_of_experiments': 3, 'experimentset_type': 'replicate', 'lab': {'uuid': '6343d57a-213e-4ac7-89ec-8b5e74c21bd2', '@id': '/labs/bas-van-steensel-lab/', 'display_title': 'Bas van Steensel, NKI', 'link_id': '~labs~bas-van-steensel-lab~', 'title': 'Bas van Steensel, NKI', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin', 'submits_for.6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'view': ['system.Everyone']}}, 'public_release': '2018-02-14', '@type': ['ExperimentSetReplicate', 'ExperimentSet', 'Item'], 'display_title': '4DNESXKBPZKQ'}, 'object': {'last_modified': {'modified_by': '/users/986b362f-4eb6-4a9c-8173-3ab267227777/', 'date_modified': '2018-07-31T19:19:50.851044+00:00'}, 'documents': [], 'submitted_by': '/users/95deb527-6d19-45fa-ace9-4e5d4ed1e023/', 'aliases': ['vanSteensel-lab:H1_LmnB1'], 'other_processed_files': [{'files': ['/files-processed/4DNFIUNJUNBC/', '/files-processed/4DNFI6BH48Y3/'], 'type': 'preliminary', 'title': 'GATC binned files', 'description': 'The files associated with the GATC bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIOFN8R1O/', '/files-processed/4DNFIO2NKJRZ/'], 'type': 'preliminary', 'title': 'Additional 1 kb binned files', 'description': 'The additional supplementary files associated with the 1 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFILWLO74H/', '/files-processed/4DNFIW6T3CC3/', '/files-processed/4DNFIQOYO2PY/', '/files-processed/4DNFITEE8XZK/'], 'type': 'preliminary', 'title': '2 kb binned files', 'description': 'The files associated with the 2 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFITUR6YTN/', '/files-processed/4DNFIXNBG8L1/', '/files-processed/4DNFICZGAM4H/', '/files-processed/4DNFIP6N54B3/'], 'type': 'preliminary', 'title': '5 kb binned files', 'description': 'The files associated with the 5 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFI1H5LWWB/', '/files-processed/4DNFIAKUT6VX/', '/files-processed/4DNFITUID4VI/', '/files-processed/4DNFIRP8P9MC/'], 'type': 'preliminary', 'title': '10 kb binned files', 'description': 'The files associated with the 10 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFI7NP9IVR/', '/files-processed/4DNFI2J1OTL1/', '/files-processed/4DNFIXRI6UIH/', '/files-processed/4DNFIJFD4TWE/'], 'type': 'preliminary', 'title': '20 kb binned files', 'description': 'The files associated with the 20 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIGPBGODE/', '/files-processed/4DNFIBTQYNRA/', '/files-processed/4DNFIMOQJ97R/', '/files-processed/4DNFIIJJZHF2/'], 'type': 'preliminary', 'title': '25 kb binned files', 'description': 'The files associated with the 25 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIOFREOYI/', '/files-processed/4DNFIVTMNBV5/', '/files-processed/4DNFIEOK4OOX/', '/files-processed/4DNFIJXADI29/'], 'type': 'preliminary', 'title': '50 kb binned files', 'description': 'The files associated with the 50 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIPYC31MH/', '/files-processed/4DNFIJVMELGG/', '/files-processed/4DNFI2H9QMZU/', '/files-processed/4DNFI7QVVZWA/'], 'type': 'preliminary', 'title': '80 kb binned files', 'description': 'The files associated with the 80 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIQ32VX9D/', '/files-processed/4DNFICT88Y52/', '/files-processed/4DNFI8KB9H8R/', '/files-processed/4DNFIKSEZOJR/'], 'type': 'preliminary', 'title': '100 kb binned files', 'description': 'The files associated with the 100 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIQP16L8Y/', '/files-processed/4DNFIIP7GJ1N/', '/files-processed/4DNFIU5LSP7P/', '/files-processed/4DNFIQQN9DWQ/'], 'type': 'preliminary', 'title': '250 kb binned files', 'description': 'The files associated with the 250 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['/files-processed/4DNFIIFBEEEU/', '/files-processed/4DNFI5RCIF4F/'], 'type': 'preliminary', 'title': 'Supplementary files for H1 Dataset', 'description': 'Non-bin specific supplementary files for the H1 LMNB1 Dataset.'}], 'schema_version': '2', 'date_created': '2017-12-14T16:19:18.119382+00:00', 'alternate_accessions': [], 'description': 'LaminB1 DamID of H1-hESC Tier 1 cells – cells were transduced with virus expressing Dam-LaminB1, gDNA was harvested after 4 days and processed for DamID-seq', 'experiments_in_set': ['/experiments-damid/4DNEXR716AG4/', '/experiments-damid/4DNEXMST3CDE/', '/experiments-damid/4DNEXTTYNWII/'], 'processed_files': ['/files-processed/4DNFIOCIY5BQ/', '/files-processed/4DNFI6HQSEU1/'], 'dbxrefs': [], 'accession': '4DNESXKBPZKQ', 'uuid': 'eae5ee86-b6ee-4112-941b-311d6026acb7', 'tags': ['4DN Joint Analysis 2018', '2018_01_public', '2018_01_internal', '2018_01_internal_JA'], 'replicate_exps': [{'bio_rep_no': 1, 'tec_rep_no': 1, 'replicate_exp': '/experiments-damid/4DNEXR716AG4/'}, {'bio_rep_no': 2, 'tec_rep_no': 1, 'replicate_exp': '/experiments-damid/4DNEXMST3CDE/'}, {'bio_rep_no': 3, 'tec_rep_no': 1, 'replicate_exp': '/experiments-damid/4DNEXTTYNWII/'}], 'award': '/awards/1U54DK107965-01/', 'status': 'released', 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}, 'publications_of_set': [], 'link_id': '~experiment-set-replicates~4DNESXKBPZKQ~', 'external_references': [], '@id': '/experiment-set-replicates/4DNESXKBPZKQ/', 'number_of_experiments': 3, 'experimentset_type': 'replicate', 'lab': '/labs/bas-van-steensel-lab/', 'public_release': '2018-02-14', '@type': ['ExperimentSetReplicate', 'ExperimentSet', 'Item'], 'display_title': '4DNESXKBPZKQ'}, 'item_type': 'experiment_set_replicate', 'propsheets': {}, 'audit': {'INTERNAL_ACTION': [{'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIVD3TAQ4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIMFL6F8F/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI8MOLU6H/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIVR14KSQ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIZQTKU2T/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIDBC1TSC/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI9DTN8M3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIINBOET5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIL24S5JE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIVSR6NQW/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIP6SWTYG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIBMOKXD8/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIKWZ1RZN/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIGIUVEV5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI4LK3AJJ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIB69P9AH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIM2X94N9/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIVL5G1EK/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFISPWYQW5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI5TKZM2L/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIC45DH5V/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFICRWGW37/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIYX7OVMK/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIX8PMO4T/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIDRO7H9W/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIY51UCAH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIJZXXMHV/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIAJAIO4U/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIGIUBW7Y/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIP2ZNNAB/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIYYYA82V/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIPG3IWQH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIYEZEWNW/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIIX2JM55/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIAH8A2RJ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIEGCQUE3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI7FQ2WFM/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIAHUMZUC/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIE81WDNP/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIT5W8PB6/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI9ZMQ1N1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI64P1ELO/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFICAH5QBS/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFINGGAQP3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIB4GJ5WT/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFICYMXZKK/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIZPPT9JO/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIH6YDWKT/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIL4D7FZY/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI9OCV3Y7/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIZ6BDHOY/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI2JABSZA/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIL7ZA137/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIFKOTUFH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI2RAXA9M/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIHKLCGO5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFINDQ2Y23/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFITVPG2CT/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIFXQEOL3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIKO9C3LW/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIDZD43NK/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFI3UJWOX9/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFINS9ZNM4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFITFOH1LU/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFIZYO2F3S/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXTTYNWII/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXTTYNWII/ has uploaded subobject /files-processed/4DNFICM8AM4P/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIWQDEPQ4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIZUDVFLD/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIPOWO4OO/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIA397ZZA/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIIB6D134/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIM252YOD/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFICOWGBZG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIXXKR7XI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI1SOVHGB/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFICZNI5GI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIV4OGTPB/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIZH98GRF/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFII3DA8XO/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIAH1X1JX/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIX6K18EB/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIJDN1FW4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIAW36PIA/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIUXTPDJQ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI3K9IDQ5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIO48GGXY/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFID6LQ59Z/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIJOD1O7C/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFID2W2A9P/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIHMBA6OE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIOHK4BWC/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIF6OCLYH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFICWH4B7E/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIMIFLZPJ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIQZJ9IWX/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIMUPUULG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIHHKSULZ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI1524I4Z/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI3ZY7SPI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIVIY962O/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIR1UTBX5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIJXYARGN/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIPVS1OVF/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIUS6Z416/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIXKWGN5F/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIP38F3PZ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI3NGH1YS/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIUU9WBLG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIQY7JNXE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIPAJUJYN/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIRAQ65E4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIOF2UDX2/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIGXCDMY2/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFITCGZKST/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIVX7SW5D/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIKK762LE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIUD7PNN2/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIIRTYHJW/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI2RWYK3Y/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIDPJ76GV/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIG9VJMGV/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIXG7M4V9/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIOBQZ4A8/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIUOEZ8JD/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI8F6ZE85/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIIU9CAU1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIXXYHI65/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI3ZYP14W/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFILPK3TYU/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIAEFP2BT/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFI72LPVYW/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXR716AG4/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXR716AG4/ has uploaded subobject /files-processed/4DNFIJOC9T9W/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIE9C1N4J/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIQLOKMQK/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI7YB6LJ9/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFII5HTUBS/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIOVU95S4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIS1MVSAH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIA5AJGUG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI213NR6I/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFITR9R8CE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI3387AHV/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIXBZ7ORR/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI1TVDL7B/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIJPBVWD3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIR91Y2OX/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIXO2ZRCS/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIQSFCHML/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIR9UWPFG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIHNA2ZJQ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIDPBQSUD/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI7H8BTBM/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIC4VHUGV/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFICUME31M/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIIQ7FJS1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI2BBO95U/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIVG36P33/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFICGGIUSR/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIJJRP4BA/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIFZBCN5T/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIQ5X4YOV/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIAYM5VOE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFINYMJG8X/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFINE79TZ1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI69GYDMT/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIKZPVN2P/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFII579RD8/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIA5CHLHD/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIRUUWDHI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIEBKUA79/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI7S5EK5P/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFISWSSY2Q/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIXBTEZVI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI8EITI5T/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFICJMUAIG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFII6JUGN4/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI58PBSCQ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIONMDS8R/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIR5YDYSL/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIB3282ZX/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIZAKGGYF/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIDDJTAX3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIJ4RB2IJ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI3QI958D/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIYRXZYI5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIVOUCMHL/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIE6GZRGH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIAWP1935/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIACYDDSY/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI5S6G939/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIC55R4Y6/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIYQQZTJP/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI9TTH2Y9/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI5INDI65/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIQBTKCZ6/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI7QWRTIS/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFI2SZX561/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiments-damid/4DNEXMST3CDE/', 'category': 'mismatched status', 'detail': 'released /experiments-damid/4DNEXMST3CDE/ has uploaded subobject /files-processed/4DNFIAYFRIL7/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIJFD4TWE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIKSEZOJR/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI2H9QMZU/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIAKUT6VX/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIUNJUNBC/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIXNBG8L1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIJVMELGG/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI7NP9IVR/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFICT88Y52/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIU5LSP7P/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIXRI6UIH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFITUID4VI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFILWLO74H/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI5RCIF4F/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIOFREOYI/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI8KB9H8R/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIOCIY5BQ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIEOK4OOX/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIIFBEEEU/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI2J1OTL1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI1H5LWWB/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI7QVVZWA/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIOFN8R1O/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIIJJZHF2/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIGPBGODE/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIO2NKJRZ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIPYC31MH/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFITUR6YTN/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIP6N54B3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI6BH48Y3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIQ32VX9D/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIJXADI29/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIQQN9DWQ/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIRP8P9MC/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIVTMNBV5/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIQP16L8Y/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIMOQJ97R/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFICZGAM4H/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFITEE8XZK/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIBTQYNRA/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIIP7GJ1N/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIW6T3CC3/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFI6HQSEU1/', 'name': 'audit_item_status'}, {'level': 30, 'level_name': 'INTERNAL_ACTION', 'path': '/experiment-set-replicates/4DNESXKBPZKQ/', 'category': 'mismatched status', 'detail': 'released /experiment-set-replicates/4DNESXKBPZKQ/ has uploaded subobject /files-processed/4DNFIQOYO2PY/', 'name': 'audit_item_status'}]}, 'uuid': 'eae5ee86-b6ee-4112-941b-311d6026acb7', 'unique_keys': {'alias': ['vanSteensel-lab:H1_LmnB1'], 'accession': ['4DNESXKBPZKQ']}, 'sid': 555716, 'tid': '0b38cb6e-36a2-4023-9698-1a46714f67e4', 'properties': {'tags': ['4DN Joint Analysis 2018', '2018_01_public', '2018_01_internal', '2018_01_internal_JA'], 'other_processed_files': [{'files': ['d508a1c2-f4d4-4761-8257-aec6f5548a10', 'c70c6200-a51e-4747-b0c9-ae683eba9c6b'], 'type': 'preliminary', 'title': 'GATC binned files', 'description': 'The files associated with the GATC bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['24043c6b-25a5-4be3-a2aa-b467342e6e83', '28b12db0-1f11-4a6d-ac89-1a4a44218c16'], 'type': 'preliminary', 'title': 'Additional 1 kb binned files', 'description': 'The additional supplementary files associated with the 1 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['9bd01d51-3aee-4535-9071-752499ca6a70', '2c0c7103-5a56-410f-b6c4-d91dd0e16bcf', '2a4979c5-3f44-4d5d-8ffe-5f17ba15b793', '01fab0f2-e76a-4a14-a77e-e5239dd391dc'], 'type': 'preliminary', 'title': '2 kb binned files', 'description': 'The files associated with the 2 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['e6654895-fcff-4651-a7df-e20ddc455e40', '063961e2-891e-4b2f-be49-97a400f9a1e9', '9cd05f6f-d339-473d-a566-8eec65ce51e6', 'bb489025-b026-4384-9e0f-9657ccd1e273'], 'type': 'preliminary', 'title': '5 kb binned files', 'description': 'The files associated with the 5 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['eb470753-670d-4bcf-a99c-1b2bb8f73111', '1f4012c8-8b5e-4c5b-9604-c35d8a5e3c21', '3d43a37c-af1b-42ee-9d2c-d3843f63214c', '06ea837b-093e-4cc5-bc1d-87ac9f19f6c2'], 'type': 'preliminary', 'title': '10 kb binned files', 'description': 'The files associated with the 10 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['5402ee85-d568-4d40-b86d-fec615f5b419', 'e6b9e27b-7205-490b-a82c-b3e3b27113ae', 'ddec78d5-2cdb-47d2-b0f5-c88f84dab443', 'cc0b81df-b04d-4e52-b51b-90fe6c82f384'], 'type': 'preliminary', 'title': '20 kb binned files', 'description': 'The files associated with the 20 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['2bcb9dcc-7dbe-4c1e-b063-7c691db4ce22', '0e4bf888-08e9-40f2-a7da-a33efd00aeee', 'e1a326ae-70d8-4795-8ee0-b5f62e80c030', '55cb0f75-30c0-4493-9b5f-e561da2a5642'], 'type': 'preliminary', 'title': '25 kb binned files', 'description': 'The files associated with the 25 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['03f64d2f-9be4-47ea-aa53-25cece95b8a8', 'c02220c9-1f09-4e61-be2e-06364412d396', 'fe33cf1c-f323-4f96-9686-b66bd8044111', '8310a3b2-e421-4656-84dd-f7b0abfea638'], 'type': 'preliminary', 'title': '50 kb binned files', 'description': 'The files associated with the 50 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['6e378c15-bf09-4304-bd45-e8a444c2a5fa', 'a20ead51-2d13-4fb9-b781-c2b172beb4b0', 'bd8df9e5-4b82-4a8b-b3df-2eaa821b575c', '401e101d-55c5-45d1-942b-d036f7d3e70f'], 'type': 'preliminary', 'title': '80 kb binned files', 'description': 'The files associated with the 80 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['f00977e7-adba-4f58-87d7-ea705ad2108f', '91c4ce9e-1620-4cf6-9127-23107829bc60', '40b70f90-9f0e-464f-bfba-7616603a64ad', 'e395550a-c186-48a8-94ce-3519d8b6a7ea'], 'type': 'preliminary', 'title': '100 kb binned files', 'description': 'The files associated with the 100 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['e23b7427-a73e-428a-9ba9-758ef4692584', '8fe88d17-a0fd-4366-8f4a-ec931aea4a61', 'fca25ab6-ad07-470a-9462-236ceb03eb13', '7e45306a-ed29-4065-a25b-acc3aaf112d5'], 'type': 'preliminary', 'title': '250 kb binned files', 'description': 'The files associated with the 250 kb bin size processing of DAM ID seq data for the H1 LMNB1 Dataset.'}, {'files': ['fcfa884e-a5df-47b2-a708-6c34f03ceb8c', '8896846b-9c19-493a-ad78-b29162eea666'], 'type': 'preliminary', 'title': 'Supplementary files for H1 Dataset', 'description': 'Non-bin specific supplementary files for the H1 LMNB1 Dataset.'}], 'award': '91b694c3-f4d7-4ddd-8278-16f94e15c1c5', 'documents': [], 'status': 'released', 'public_release': '2018-02-14', 'aliases': ['vanSteensel-lab:H1_LmnB1'], 'submitted_by': '95deb527-6d19-45fa-ace9-4e5d4ed1e023', 'processed_files': ['85490436-b099-4760-9d4e-621e2dea4572', 'dceba897-c946-40b8-bb64-8776cf0d72d9'], 'replicate_exps': [{'bio_rep_no': 1, 'tec_rep_no': 1, 'replicate_exp': '22351015-90bb-4121-bc1f-5b5d99c65769'}, {'bio_rep_no': 2, 'tec_rep_no': 1, 'replicate_exp': '3e8087c9-1311-484d-9cea-8dac908b0aee'}, {'bio_rep_no': 3, 'tec_rep_no': 1, 'replicate_exp': '804d8f54-07b7-4f82-8c2b-71539b508aca'}], 'experiments_in_set': ['22351015-90bb-4121-bc1f-5b5d99c65769', '3e8087c9-1311-484d-9cea-8dac908b0aee', '804d8f54-07b7-4f82-8c2b-71539b508aca'], 'date_created': '2017-12-14T16:19:18.119382+00:00', 'alternate_accessions': [], 'description': 'LaminB1 DamID of H1-hESC Tier 1 cells – cells were transduced with virus expressing Dam-LaminB1, gDNA was harvested after 4 days and processed for DamID-seq', 'lab': '6343d57a-213e-4ac7-89ec-8b5e74c21bd2', 'schema_version': '2', 'experimentset_type': 'replicate', 'last_modified': {'modified_by': '986b362f-4eb6-4a9c-8173-3ab267227777', 'date_modified': '2018-07-31T19:19:50.851044+00:00'}, 'dbxrefs': [], 'accession': '4DNESXKBPZKQ'}, 'links': {'badges': [], 'other_processed_files~files': ['e6654895-fcff-4651-a7df-e20ddc455e40', '8310a3b2-e421-4656-84dd-f7b0abfea638', '40b70f90-9f0e-464f-bfba-7616603a64ad', 'e6b9e27b-7205-490b-a82c-b3e3b27113ae', '6e378c15-bf09-4304-bd45-e8a444c2a5fa', '5402ee85-d568-4d40-b86d-fec615f5b419', '063961e2-891e-4b2f-be49-97a400f9a1e9', '24043c6b-25a5-4be3-a2aa-b467342e6e83', 'eb470753-670d-4bcf-a99c-1b2bb8f73111', 'fcfa884e-a5df-47b2-a708-6c34f03ceb8c', 'd508a1c2-f4d4-4761-8257-aec6f5548a10', 'e1a326ae-70d8-4795-8ee0-b5f62e80c030', 'c02220c9-1f09-4e61-be2e-06364412d396', '8896846b-9c19-493a-ad78-b29162eea666', '9bd01d51-3aee-4535-9071-752499ca6a70', 'fe33cf1c-f323-4f96-9686-b66bd8044111', 'e23b7427-a73e-428a-9ba9-758ef4692584', '2bcb9dcc-7dbe-4c1e-b063-7c691db4ce22', '03f64d2f-9be4-47ea-aa53-25cece95b8a8', 'cc0b81df-b04d-4e52-b51b-90fe6c82f384', '55cb0f75-30c0-4493-9b5f-e561da2a5642', '01fab0f2-e76a-4a14-a77e-e5239dd391dc', 'bb489025-b026-4384-9e0f-9657ccd1e273', 'ddec78d5-2cdb-47d2-b0f5-c88f84dab443', 'bd8df9e5-4b82-4a8b-b3df-2eaa821b575c', '3d43a37c-af1b-42ee-9d2c-d3843f63214c', '91c4ce9e-1620-4cf6-9127-23107829bc60', '9cd05f6f-d339-473d-a566-8eec65ce51e6', '06ea837b-093e-4cc5-bc1d-87ac9f19f6c2', 'a20ead51-2d13-4fb9-b781-c2b172beb4b0', 'f00977e7-adba-4f58-87d7-ea705ad2108f', 'e395550a-c186-48a8-94ce-3519d8b6a7ea', '1f4012c8-8b5e-4c5b-9604-c35d8a5e3c21', '28b12db0-1f11-4a6d-ac89-1a4a44218c16', '401e101d-55c5-45d1-942b-d036f7d3e70f', '0e4bf888-08e9-40f2-a7da-a33efd00aeee', 'c70c6200-a51e-4747-b0c9-ae683eba9c6b', 'fca25ab6-ad07-470a-9462-236ceb03eb13', '2c0c7103-5a56-410f-b6c4-d91dd0e16bcf', '2a4979c5-3f44-4d5d-8ffe-5f17ba15b793', '7e45306a-ed29-4065-a25b-acc3aaf112d5', '8fe88d17-a0fd-4366-8f4a-ec931aea4a61'], 'contributing_labs': [], 'award': ['91b694c3-f4d7-4ddd-8278-16f94e15c1c5'], 'documents': [], 'processed_files': ['dceba897-c946-40b8-bb64-8776cf0d72d9', '85490436-b099-4760-9d4e-621e2dea4572'], 'lab': ['6343d57a-213e-4ac7-89ec-8b5e74c21bd2'], 'replicate_exps~replicate_exp': ['22351015-90bb-4121-bc1f-5b5d99c65769', '804d8f54-07b7-4f82-8c2b-71539b508aca', '3e8087c9-1311-484d-9cea-8dac908b0aee'], 'submitted_by': ['95deb527-6d19-45fa-ace9-4e5d4ed1e023'], 'experiments_in_set': ['22351015-90bb-4121-bc1f-5b5d99c65769', '804d8f54-07b7-4f82-8c2b-71539b508aca', '3e8087c9-1311-484d-9cea-8dac908b0aee'], 'last_modified~modified_by': ['986b362f-4eb6-4a9c-8173-3ab267227777']}, 'principals_allowed': {'audit': ['system.Everyone'], 'edit': ['group.admin'], 'view': ['system.Everyone']}}

In [22]:
ES_item = ff_utils.get_es_metadata(['88793582-d8d5-4cef-89c6-88a973441791'], key=my_auth)[0]


add_from_embedded = {'file_fastq': ['workflow_run_inputs', 'workflow_run_outputs'],
                     'file_processed': ['workflow_run_inputs', 'workflow_run_outputs']
                             }
uuid = ES_item['uuid']
object_resp = ES_item['object']
obj_type = object_resp['@type'][0]
obj_key = schema_name[obj_type]
    
#get linked items from es
uuids_to_check =[]
for key in ES_item['links']:
    print(key)
    #print(ES_item['links'][key])
    uuids_to_check.extend(ES_item['links'][key])
print(len(uuids_to_check))

# check if any field from the embedded frame is required
add_fields = add_from_embedded.get(obj_key)
if add_fields:
    emb_resp = ES_item['embedded']
    for a_field in add_fields:
        field_val = emb_resp.get(a_field)
        if field_val:
            #turn it into string
            field_val = str(field_val)
            # check if any of embedded uuids is in the field value
            for a_uuid in ES_item['embedded_uuids']:
                if a_uuid in field_val:
                    uuids_to_check.append(a_uuid)
                                                         
print(len(uuids_to_check)) 
uuid_list = list(set(uuids_to_check))
print(len(uuid_list))
ES_items = ff_utils.get_es_metadata(uuid_list, key=my_auth)
print(len(ES_items))
                                

submitted_by
award
last_modified~modified_by
contributing_labs
file_format_specifications
lab
related_files~file
quality_metric
6
25
22
10
